In [ ]:
!pip -q install gspread gspread-dataframe google-auth

In [ ]:
# =======================
# Config & imports (portable: Colab / GitHub / local)
# =======================
import io, threading, requests, pandas as pd, os, time, random
from concurrent.futures import ThreadPoolExecutor, as_completed
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from IPython.display import display

# ---- Secrets: Colab (userdata) o ENV ----
def _get_secret(name: str) -> str | None:
    try:
        from google.colab import userdata as _ud  # solo existe en Colab
        val = _ud.get(name)
        if val:
            return val
    except Exception:
        pass
    return os.environ.get(name)

USER_API   = _get_secret("USER_API")
SECRET_API = _get_secret("SECRET_API")
if not USER_API or not SECRET_API:
    raise ValueError(
        "Falta USER_API o SECRET_API. En Colab: agrega en 'Secretos'. "
        "En GitHub/local: define variables/Secrets de entorno."
    )

# =======================
# Parámetros "amables"
# =======================
BASE_URL   = "https://mutatio-api.gobravo.dev"
MAX_PAGES  = 2000

# Concurrencia general (facturaciones/reparadoras)
WINDOW_GENERAL      = 6      # tamaño de lote paralelo
MAX_WORKERS_GENERAL = 3      # hilos para paralelizar

# Concurrencia específica para collections (MUY grande)
WINDOW_COLLECTIONS      = 1  # totalmente secuencial
MAX_WORKERS_COLLECTIONS = 1

# Rate limiting y backoff
MIN_INTERVAL_S   = 0.35      # intervalo mínimo entre POST al mismo host
JITTER_S         = (0.05, 0.15)
BACKOFF_BASE_S   = 0.8
BACKOFF_CAP_S    = 8.0
MAX_RETRIES_SOFT = 4         # reintentos suaves por request (además de Retry del adapter)

ENDPOINTS = {
    "facturaciones": "/accounting/facturations/download",
    "reparadoras"  : "/accounting/repairs/download",
    "collections"  : "/accounting/collections/download",
}
TIPOS_COMISION = ["LIQUIDACION_COLOMBIA","MENSUALIDAD_COLOMBIA","INSCRIPCION_COLOMBIA"]

# =======================
# Sesión HTTP y auth
# =======================
def make_session():
    s = requests.Session()
    retry = Retry(
        total=5, connect=3, read=3, backoff_factor=1.2,
        status_forcelist=[408,429,500,502,503,504],
        allowed_methods={"POST"},
        raise_on_status=False,
        respect_retry_after_header=True
    )
    s.mount("https://", HTTPAdapter(max_retries=retry))
    s.mount("http://",  HTTPAdapter(max_retries=retry))
    s.headers.update({
        "User-Agent": "BravoDataClient/1.1 (+colab/local)",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
    })
    return s

session = make_session()

_token_lock = threading.Lock()
_token_cache = {"token": None, "user": USER_API, "secret": SECRET_API}

def _sleep_with_jitter(base_s: float):
    time.sleep(base_s + random.uniform(*JITTER_S))

def _get_new_token():
    # Backoff explícito para el auth (504/timeout)
    attempt = 0
    while True:
        attempt += 1
        try:
            r = session.post(
                f"{BASE_URL}/auth/generate-token",
                json={"user": _token_cache["user"], "secret": _token_cache["secret"]},
                headers={"Content-Type":"application/json","Accept":"application/json"},
                timeout=(10,45)
            )
            if r.status_code in (429, 500, 502, 503, 504):
                # respetar Retry-After si viene
                ra = r.headers.get("Retry-After")
                if ra:
                    try:
                        _sleep_with_jitter(float(ra))
                    except Exception:
                        _sleep_with_jitter(min(BACKOFF_CAP_S, BACKOFF_BASE_S*(2**(attempt-1))))
                    continue
            r.raise_for_status()
            tok = r.json().get("token")
            if not tok:
                raise RuntimeError("Auth OK pero no vino 'token'.")
            return tok
        except requests.RequestException:
            if attempt >= 5:
                raise
            _sleep_with_jitter(min(BACKOFF_CAP_S, BACKOFF_BASE_S*(2**(attempt-1))))

def get_token_cached(force=False):
    with _token_lock:
        if force or not _token_cache["token"]:
            _token_cache["token"] = _get_new_token()
        return _token_cache["token"]

# =======================
# Rate limiter global por host
# =======================
_last_call_lock = threading.Lock()
_last_call_ts = 0.0

def _polite_pause():
    global _last_call_ts
    with _last_call_lock:
        now = time.perf_counter()
        wait = _last_call_ts + MIN_INTERVAL_S - now
        if wait > 0:
            time.sleep(wait)
        _last_call_ts = time.perf_counter()

# =======================
# Utilidades
# =======================
def _read_csv_bytes(content: bytes) -> pd.DataFrame:
    if not content or not content.strip():
        return pd.DataFrame()
    first = content.split(b"\n", 1)[0]
    sep = b';' if first.count(b';') >= first.count(b',') else b','
    return pd.read_csv(io.BytesIO(content), sep=sep.decode(), dtype=str)

def _post_csv_once(resource: str, page: int, body):
    url = f"{BASE_URL}{resource}"
    tok = get_token_cached()
    headers = {
        "Authorization": f"Bearer {tok}",
        "Accept": "text/csv, text/plain",
        "Content-Type": "application/json",
    }
    params = {"pageToDownload": str(page)}
    _polite_pause()
    r = session.post(url, headers=headers, params=params, json=body, timeout=(12,90))
    # Manejo 401 -> refresh token una sola vez
    if r.status_code == 401:
        tok = get_token_cached(force=True)
        headers["Authorization"] = f"Bearer {tok}"
        _polite_pause()
        r = session.post(url, headers=headers, params=params, json=body, timeout=(12,90))
    return r

def _post_csv(resource: str, page: int, body):
    """
    POST con backoff suave + respeto de Retry-After/429.
    Devuelve bytes CSV o lanza excepción si realmente falla.
    """
    attempt = 0
    while True:
        attempt += 1
        r = _post_csv_once(resource, page, body)
        # Respetar Retry-After
        if r.status_code in (429, 500, 502, 503, 504):
            ra = r.headers.get("Retry-After")
            if ra:
                try:
                    _sleep_with_jitter(float(ra))
                    continue
                except Exception:
                    pass
        if r.status_code >= 400:
            if attempt <= MAX_RETRIES_SOFT and r.status_code in (408, 429, 500, 502, 503, 504):
                _sleep_with_jitter(min(BACKOFF_CAP_S, BACKOFF_BASE_S*(2**(attempt-1))))
                continue
            # Si seguimos fallando, levantar con resumen breve
            raise RuntimeError(f"[{resource}] Error {r.status_code} en page={page}. Resumen: {r.text[:300]}")
        return r.content

def _fetch_batch(resource: str, start_page: int, body, workers, window):
    pages = list(range(start_page, start_page + window))
    out = {}
    with ThreadPoolExecutor(max_workers=workers) as ex:
        futs = {ex.submit(_post_csv, resource, p, body): p for p in pages}
        for f in as_completed(futs):
            p = futs[f]
            try:
                out[p] = _read_csv_bytes(f.result())
            except Exception:
                out[p] = pd.DataFrame()
    # Devuelve ordenadas
    return [out[p] for p in pages]

def _download_parallel(resource: str, body, max_pages, workers, window):
    """
    Descarga por lotes paralelos con corte en primer vacío.
    Usa fallback a 1-based si la 0-based arranca vacía.
    """
    frames = []
    # --- 0-based ---
    cur = 0
    while cur < max_pages:
        batch = _fetch_batch(resource, cur, body, workers, window)
        if cur == 0 and all(df.empty for df in batch):  # intentar 1-based
            break
        for df in batch:
            if df.empty:
                return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
            frames.append(df)
        cur += window
    if frames:
        return pd.concat(frames, ignore_index=True)

    # --- 1-based ---
    frames = []
    cur = 1
    while cur < max_pages+1:
        batch = _fetch_batch(resource, cur, body, workers, window)
        if cur == 1 and all(df.empty for df in batch):
            return pd.DataFrame()
        for df in batch:
            if df.empty:
                return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
            frames.append(df)
        cur += window
    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

def _download_sequential(resource: str, body, max_pages):
    """
    Descarga SECUENCIAL (muy respetuosa). Fallback a 1-based.
    Pausa corta entre páginas para reducir presión.
    """
    frames = []
    # 0-based
    for p in range(max_pages):
        try:
            content = _post_csv(resource, p, body)
            df = _read_csv_bytes(content)
        except Exception:
            df = pd.DataFrame()
        if df.empty:
            if p == 0:
                # 1-based fallback
                for p1 in range(1, max_pages+1):
                    try:
                        content1 = _post_csv(resource, p1, body)
                        df1 = _read_csv_bytes(content1)
                    except Exception:
                        df1 = pd.DataFrame()
                    if df1.empty:
                        break
                    frames.append(df1)
                    _sleep_with_jitter(0.12)  # pausa amable
                break
            break
        frames.append(df)
        _sleep_with_jitter(0.12)  # pausa amable
    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

# =======================
# Descargadores
# =======================
def descargar_facturaciones_tres_tipos():
    dfs = []
    for tipo in TIPOS_COMISION:
        body = [{
            "column": "Comision",
            "attribute": "commissionType",
            "columnType": "ENUM",
            "table": "facturation",
            "operation": "IGUAL_A",
            "value": tipo
        }]
        df = _download_parallel(
            ENDPOINTS["facturaciones"], body,
            max_pages=MAX_PAGES,
            workers=MAX_WORKERS_GENERAL,
            window=WINDOW_GENERAL
        )
        if not df.empty:
            dfs.append(df)
        print(f"[Facturaciones/{tipo}] filas: {0 if df is None else len(df)}")
        _sleep_with_jitter(0.3)  # pausa entre tipos
    if not dfs:
        return pd.DataFrame()
    return pd.concat(dfs, ignore_index=True).drop_duplicates()

def descargar_reparadoras():
    # Reparadoras: tráfico moderado, pero seguro y amable
    return _download_sequential(ENDPOINTS["reparadoras"], body=[], max_pages=MAX_PAGES)

def _norm(s: str) -> str:
    import unicodedata as _ud
    return ''.join(c for c in _ud.normalize('NFKD', str(s)) if not _ud.combining(c)).lower().strip()

def filtrar_collections_cobrado(df: pd.DataFrame) -> pd.DataFrame:
    """Devuelve solo filas con Estado de cobro == Cobrado (case-insensitive)."""
    if df.empty:
        return df
    norm_cols = {col: _norm(col) for col in df.columns}
    cand = None
    for col, n in norm_cols.items():
        if "estado" in n and ("cobro" in n or "cobranza" in n or "collection" in n or "charge" in n):
            cand = col
            break
    if cand is None:
        for col in df.columns:
            if _norm(col) == "estado de cobro":
                cand = col
                break
    if cand is None:
        print("⚠️ No encontré columna 'Estado de cobro'; devuelvo Collections sin filtrar.")
        return df
    mask = df[cand].astype(str).str.lower().str.contains("cobrado")
    return df[mask].reset_index(drop=True)

def descargar_collections_filtrado():
    # Collections: extremo respeto → secuencial + pausas
    df = _download_sequential(ENDPOINTS["collections"], body=[], max_pages=MAX_PAGES)
    return filtrar_collections_cobrado(df)

# =======================
# Ejecutar todo
# =======================
get_token_cached(force=True)

c_facturaciones = descargar_facturaciones_tres_tipos()
reparadoras_df  = descargar_reparadoras()
collections_df  = descargar_collections_filtrado()  # solo 'Cobrado/COBRADO'

print(f"Facturaciones (3 tipos): {len(c_facturaciones)} filas | {len(c_facturaciones.columns)} cols")
print(f"Reparadoras           : {len(reparadoras_df)} filas | {len(reparadoras_df.columns)} cols")
print(f"Collections (Cobrado) : {len(collections_df)} filas | {len(collections_df.columns)} cols")

display(c_facturaciones.head(3))
display(reparadoras_df.head(3))
display(collections_df.head(3))

[Facturaciones/LIQUIDACION_COLOMBIA] filas: 80163
[Facturaciones/MENSUALIDAD_COLOMBIA] filas: 816915
[Facturaciones/INSCRIPCION_COLOMBIA] filas: 92385
Facturaciones (3 tipos): 989463 filas | 37 cols
Reparadoras           : 0 filas | 0 cols
Collections (Cobrado) : 724938 filas | 54 cols


,Id,Empresa Emisora,Credito,Status facturacion,Tipo de comision,Status reparadora,Referencia,Monto,Fecha de facturacion,Fecha de envio de cobro,...,Folio Nota Credito,Prefijo Resolucion,CUFE,Descripcion,Referencia credito,Tiene Nota Credito Electronica,Fecha de Nota Credito Electronica,Status NC Sistema Contable,Subido NC a Sistema Contable por,Fecha NC subida a sistema contable
0,1186301,RESUELVE_TU_DEUDA_COLOMBIA_SAS,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,NaN,3208712958,294118.0,4/10/2023,2/1/2024,...,Sin Folio NC,Sin Prefijo Resolucion,Sin Cufe,Sin Descripcion,Sin ref credito,No,Sin Status,NO_CARGADO,Sin data,Sin fecha
1,1186303,RESUELVE_TU_DEUDA_COLOMBIA_SAS,false,COBRADO,LIQUIDACION_COLOMBIA,NaN,3182588339,100000.0,3/10/2023,23/10/2023,...,Sin Folio NC,Sin Prefijo Resolucion,Sin Cufe,Sin Descripcion,Sin ref credito,No,Sin Status,NO_CARGADO,Sin data,Sin fecha
2,1186306,RESUELVE_TU_DEUDA_COLOMBIA_SAS,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,NaN,3505172416,111566.0,3/10/2023,4/12/2023,...,Sin Folio NC,Sin Prefijo Resolucion,Sin Cufe,Sin Descripcion,Sin ref credito,No,Sin Status,NO_CARGADO,Sin data,Sin fecha


""


,Id,Monto,Estado de cobro,Fecha de cobro,Banco receptor,Folio del cobro,Folio de Nota Credito,Id Factura Relacionada,Tiene Nota Credito Electronica,Fecha de Nota Credito Electronica,...,Folio Nota Credito,Prefijo Resolucion,CUFE,Descripcion,Referencia credito,Tiene Nota Credito Electronica.1,Fecha de Nota Credito Electronica.1,Status NC Sistema Contable.1,Subido NC a Sistema Contable por.1,Fecha NC subida a sistema contable.1
0,1,11890.0,COBRADO,15/8/2023,BANCOLOMBIA,NaN,NaN,993984,No,Sin Status,...,Sin Folio NC,Sin Prefijo Resolucion,Sin Cufe,Sin Descripcion,Sin ref credito,No,Sin Status,NO_CARGADO,Sin data,Sin fecha
1,2,23136.0,COBRADO,15/8/2023,BANCOLOMBIA,NaN,NaN,966290,No,Sin Status,...,Sin Folio NC,Sin Prefijo Resolucion,Sin Cufe,Sin Descripcion,Sin ref credito,No,Sin Status,NO_CARGADO,Sin data,Sin fecha
2,3,12191.0,COBRADO,15/8/2023,BANCOLOMBIA,NaN,NaN,978079,No,Sin Status,...,Sin Folio NC,Sin Prefijo Resolucion,Sin Cufe,Sin Descripcion,Sin ref credito,No,Sin Status,NO_CARGADO,Sin data,Sin fecha


In [ ]:
# ========= Imports =========
import io, os, requests, pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from IPython.display import display

# ========= Secrets (Colab o ENV) =========
def _get_secret(name: str):
    # 1) Colab
    try:
        from google.colab import userdata as _ud
        val = _ud.get(name)
        if val:
            return val
    except Exception:
        pass
    # 2) GitHub/local
    return os.environ.get(name)

USER_API   = _get_secret("USER_API")
SECRET_API = _get_secret("SECRET_API")
if not USER_API or not SECRET_API:
    raise ValueError("Falta USER_API o SECRET_API. En Colab: agrégalo en 'Secretos'. "
                     "En GitHub/local: define variables/Secrets de entorno.")

# ========= Constantes =========
BASE_URL  = "https://mutatio-api.gobravo.dev"
RESOURCE  = "/accounting/repairs/download"   # <-- reparadoras

# ========= Sesión con reintentos =========
def make_session():
    s = requests.Session()
    retry = Retry(
        total=3, connect=2, read=2, backoff_factor=1.2,
        status_forcelist=[502,503,504], allowed_methods={"POST"},
        raise_on_status=False
    )
    s.mount("https://", HTTPAdapter(max_retries=retry))
    s.mount("http://",  HTTPAdapter(max_retries=retry))
    return s

session = make_session()

# ========= Auth =========
def get_token(user: str, secret: str) -> str:
    r = session.post(f"{BASE_URL}/auth/generate-token",
                     json={"user": user, "secret": secret},
                     headers={"Content-Type":"application/json","Accept":"application/json"},
                     timeout=(10,45))
    r.raise_for_status()
    tok = r.json().get("token")
    if not tok:
        raise RuntimeError("Auth OK pero no vino 'token'.")
    return tok

# ========= Util: leer CSV con ; o , =========
def _leer_csv_flexible(resp) -> pd.DataFrame:
    resp.raise_for_status()
    txt = resp.text or ""
    if not txt.strip():
        return pd.DataFrame()
    first = txt.splitlines()[0]
    sep = ';' if first.count(';') >= first.count(',') else ','
    return pd.read_csv(io.StringIO(txt), sep=sep, dtype=str)

# ========= Descarga (una página) con auto-refresh de token =========
def descargar_pagina_repairs(user: str, secret: str, page: int, filtros=None) -> pd.DataFrame:
    if filtros is None:
        filtros = []  # este endpoint funciona sin body
    url = f"{BASE_URL}{RESOURCE}"

    def _do(tok):
        return session.post(
            url,
            headers={
                "Authorization": f"Bearer {tok}",
                "Accept": "text/csv, text/plain",
                "Content-Type": "application/json",
            },
            params={"pageToDownload": str(page)},
            json=filtros,
            timeout=(12,90)
        )

    token = get_token(user, secret)
    r = _do(token)
    if r.status_code == 401:
        token = get_token(user, secret)
        r = _do(token)

    if r.status_code >= 400:
        raise RuntimeError(f"Error {r.status_code} en page={page}. Resumen: {r.text[:300]}")

    return _leer_csv_flexible(r)

# ========= Descarga total (paginando) =========
def descargar_todo_repairs(user: str, secret: str, max_pages: int = 1000, filtros=None) -> pd.DataFrame:
    frames = []
    # 0-based
    for page in range(max_pages):
        df = descargar_pagina_repairs(user, secret, page=page, filtros=filtros)
        if df.empty:
            if page == 0:
                # fallback 1-based
                frames1 = []
                for p1 in range(1, max_pages+1):
                    df1 = descargar_pagina_repairs(user, secret, page=p1, filtros=filtros)
                    if df1.empty:
                        break
                    frames1.append(df1)
                    if p1 % 25 == 0:
                        print(f"[1-based] páginas leídas: {p1}")
                return pd.concat(frames1, ignore_index=True) if frames1 else pd.DataFrame()
            break
        frames.append(df)
        if page % 25 == 0 and page > 0:
            print(f"[0-based] páginas leídas: {page+1}")

    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

# ========= Ejecutar =========
reparadoras_df = descargar_todo_repairs(USER_API, SECRET_API, max_pages=1200)

print(f"Filas totales: {len(reparadoras_df)} | Columnas: {len(reparadoras_df.columns)}")
display(reparadoras_df.head())
print(reparadoras_df.columns.tolist())

Filas totales: 54425 | Columnas: 28


,Id,Referencia,Berex Id,Nombre Completo,Numero de Documento,Correo Electronico,Status,Fecha de Inicio,Tipo,Deuda Resuelve,...,Fecha de baja,Tipo de Pricing,Fecha de Reactivación,Porcentaje Liquidacion,Comision Inicial,Comision Mensual,Fecha de creacion,Fecha de actualizacion,Creado por,Actualizado por
0,3,3008127181,2766,ISABEL CRISTINA GOMEZ CORTES,52455178,LILIGRIS200@GMAIL.COM,GRADUADO,30/10/2018,4,1.9257551E7,...,NaN,Tradicional,NaN,0.15,404409.0,125174.08,10/1/2023 00:00:00,17/9/2024 10:37:24,Sin registro,sgonzalez
1,4,3046559962,6487,CARLOS ALBERTO ZARTA ISAZA,14242437,VIAJERO60-@HOTMAIL.COM,BAJA_2XC,18/1/2019,5,7127230.82,...,30/4/2025,Tradicional,NaN,0.15,149672.0,35636.15,10/1/2023 00:00:00,26/5/2025 16:48:14,Sin registro,sgonzalez
2,5,3133078002,6582,LIGIA MATEUS BENAVIDES,39763454,MATEUSLIGIA848@GMAIL.COM,ESTRUCTURADO_INCUMPLIDO,22/1/2019,5,1.3117212E7,...,31/10/2023,Tradicional,30/6/2020,0.15,275461.0,65586.06,10/1/2023 00:00:00,17/9/2024 10:37:24,Sin registro,sgonzalez
3,6,3023738871,6624,LIZ ANYELI SILVA ROMERO,52272068,ANGIESILVA.SPORT@HOTMAIL.COM,ESTRUCTURADO_INCUMPLIDO,23/1/2019,5,3.8208223E7,...,30/9/2023,Tradicional,NaN,0.15,802373.0,191041.12,10/1/2023 00:00:00,17/9/2024 10:37:24,Sin registro,sgonzalez
4,7,3167671311,6703,RICARDO NAVARRO VASQUEZ,79848858,USS_ENTERPRISE_CO@YAHOO.COM,GRADUADO,25/1/2019,5,1.1942724E7,...,NaN,Tradicional,NaN,0.15,250797.0,59713.62,10/1/2023 00:00:00,17/9/2024 10:37:24,Sin registro,sgonzalez


['Id', 'Referencia', 'Berex Id', 'Nombre Completo', 'Numero de Documento', 'Correo Electronico', 'Status', 'Fecha de Inicio', 'Tipo', 'Deuda Resuelve', 'Movimientos Mensual', 'Numero de Deudas', 'Termino de programa', 'Vehiculo de ahorro', 'Empresa', 'ARD', 'Fecha de graduación', 'Deuda inicial Fija', 'Fecha de baja', 'Tipo de Pricing', 'Fecha de Reactivación', 'Porcentaje Liquidacion', 'Comision Inicial', 'Comision Mensual', 'Fecha de creacion', 'Fecha de actualizacion', 'Creado por', 'Actualizado por']


In [ ]:
import unicodedata
import pandas as pd
from IPython.display import display

def _norm(s: str) -> str:
    s = '' if s is None else str(s)
    # normaliza acentos y espacios
    s = unicodedata.normalize("NFKD", s)
    s = ''.join(c for c in s if not unicodedata.combining(c))
    return ' '.join(s.lower().strip().split())

def subset_by_names(df: pd.DataFrame, wanted: list[str]) -> pd.DataFrame:
    """Selecciona columnas con match insensible a acentos/mayúsculas y en el orden pedido."""
    if df is None or df.empty:
        return pd.DataFrame()
    # mapa normalizado -> nombre real
    norm_to_real = {}
    for c in df.columns:
        nc = _norm(c)
        # si hay duplicados normalizados, conservamos el primero
        if nc not in norm_to_real:
            norm_to_real[nc] = c
    out_cols = []
    missing  = []
    for w in wanted:
        nc = _norm(w)
        if nc in norm_to_real:
            out_cols.append(norm_to_real[nc])
        else:
            missing.append(w)
    if missing:
        print("⚠️ No encontré en el DataFrame:", missing)
    if not out_cols:
        return pd.DataFrame()
    return df.loc[:, out_cols]

# ------------------------------
# 1) reparadoras_df (con rename)
# ------------------------------
rep_wanted = [
    'Referencia',
    'Berex Id',
    'Status',
    'Fecha de Inicio',
    'Deuda Resuelve',
    'Movimientos Mensual',   # será renombrada a "Apartado Mensual"
    'Comision Inicial',
    'Comision Mensual',
    'Vehiculo de ahorro',
    'Fecha de graduación',
    'Deuda inicial Fija',
    'Fecha de baja',
    'Tipo de Pricing',
]

rep_sel = subset_by_names(reparadoras_df, rep_wanted).copy()

# renombrar "Movimientos Mensual" -> "Apartado Mensual" (resistente a acentos/mayúsculas)
for col in list(rep_sel.columns):
    if _norm(col) == _norm('Movimientos Mensual'):
        rep_sel.rename(columns={col: 'Apartado Mensual'}, inplace=True)
        break

print(f"reparadoras_df -> {rep_sel.shape[0]} filas x {rep_sel.shape[1]} cols")
display(rep_sel.head())

# -----------------------------------
# 2) c_facturaciones (solo columnas)
# -----------------------------------
fac_wanted = [
    'Id',
    'Credito',
    'Status facturacion',
    'Tipo de comision',
    'Referencia',
    'Monto',
    'Fecha de facturacion',
    'Fecha de cobro',
]
fac_sel = subset_by_names(c_facturaciones, fac_wanted).copy()
print(f"c_facturaciones -> {fac_sel.shape[0]} filas x {fac_sel.shape[1]} cols")
display(fac_sel.head())

# --------------------------------
# 3) collections_df (solo columnas)
# --------------------------------
col_wanted = [
    'Id',
    'Monto',
    'Estado de cobro',
    'Fecha de cobro',
    'Fecha de facturacion',
    'Status facturacion',
    'Tipo de comision',
    'Referencia',

]
col_sel = subset_by_names(collections_df, col_wanted).copy()
print(f"collections_df -> {col_sel.shape[0]} filas x {col_sel.shape[1]} cols")
display(col_sel.head())

# (Opcional) reasignar a las variables originales si quieres seguir trabajando con las versiones reducidas
# reparadoras_df  = rep_sel
# c_facturaciones = fac_sel
# collections_df  = col_sel

reparadoras_df -> 54425 filas x 13 cols


/usr/local/lib/python3.12/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  cast_date_col = pd.to_datetime(column, errors="coerce")


,Referencia,Berex Id,Status,Fecha de Inicio,Deuda Resuelve,Apartado Mensual,Comision Inicial,Comision Mensual,Vehiculo de ahorro,Fecha de graduación,Deuda inicial Fija,Fecha de baja,Tipo de Pricing
0,3008127181,2766,GRADUADO,30/10/2018,1.9257551E7,459054.0,404409.0,125174.08,POWWI,30/11/2023,1.9257551E7,NaN,Tradicional
1,3046559962,6487,BAJA_2XC,18/1/2019,7127230.82,160453.06,149672.0,35636.15,POWWI,NaN,7127230.82,30/4/2025,Tradicional
2,3133078002,6582,ESTRUCTURADO_INCUMPLIDO,22/1/2019,1.3117212E7,192903.38,275461.0,65586.06,POWWI,NaN,1.3117212E7,31/10/2023,Tradicional
3,3023738871,6624,ESTRUCTURADO_INCUMPLIDO,23/1/2019,3.8208223E7,1125998.54,802373.0,191041.12,POWWI,NaN,3.8208223E7,30/9/2023,Tradicional
4,3167671311,6703,GRADUADO,25/1/2019,1.1942724E7,267093.47,250797.0,59713.62,POWWI,31/3/2024,1.1942724E7,NaN,Tradicional


c_facturaciones -> 989463 filas x 8 cols


,Id,Credito,Status facturacion,Tipo de comision,Referencia,Monto,Fecha de facturacion,Fecha de cobro
0,1186301,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,3208712958,294118.0,4/10/2023,3/1/2024
1,1186303,false,COBRADO,LIQUIDACION_COLOMBIA,3182588339,100000.0,3/10/2023,24/10/2023
2,1186306,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,3505172416,111566.0,3/10/2023,5/12/2023
3,1186311,true,COBRADO,LIQUIDACION_COLOMBIA,3233270136,853382.5499999999,4/10/2023,4/10/2023
4,1186314,false,COBRADO,LIQUIDACION_COLOMBIA,3223109455,10162.5,4/10/2023,11/10/2023


collections_df -> 724938 filas x 8 cols


,Id,Monto,Estado de cobro,Fecha de cobro,Fecha de facturacion,Status facturacion,Tipo de comision,Referencia
0,1,11890.0,COBRADO,15/8/2023,24/7/2023,COBRO_PARCIAL_COBRADO,INTERESES,6007350
1,2,23136.0,COBRADO,15/8/2023,11/7/2023,COBRO_PARCIAL_COBRADO,INTERESES,6006933
2,3,12191.0,COBRADO,15/8/2023,17/7/2023,COBRO_PARCIAL_COBRADO,INTERESES,6003875
3,4,11671.0,COBRADO,15/8/2023,17/7/2023,COBRO_PARCIAL_COBRADO,INTERESES,6002993R
4,5,31133.0,COBRADO,15/8/2023,11/7/2023,COBRO_PARCIAL_COBRADO,INTERESES,6006986


In [ ]:
def filter_eq(df: pd.DataFrame, col_like: str, value: str = "COBRADO") -> pd.DataFrame:
    if df is None or df.empty:
        return pd.DataFrame(columns=[])
    target = None
    for c in df.columns:
        if _norm(c) == _norm(col_like):
            target = c
            break
    if target is None:
        print(f"⚠️ No encontré la columna '{col_like}'.")
        return pd.DataFrame(columns=df.columns)
    mask = df[target].astype(str).str.upper().str.strip().eq(value.upper())
    return df.loc[mask].reset_index(drop=True)

# Filtrar según lo que pediste:
fac_sel = filter_eq(fac_sel, "Status facturacion", "COBRADO")
col_sel = filter_eq(col_sel, "Estado de cobro", "COBRADO")

print(f"fac_sel (Status facturacion=COBRADO): {fac_sel.shape}")
print(f"col_sel (Estado de cobro=COBRADO): {col_sel.shape}")
display(fac_sel.head())
display(col_sel.head())

fac_sel (Status facturacion=COBRADO): (692247, 8)
col_sel (Estado de cobro=COBRADO): (724938, 8)


,Id,Credito,Status facturacion,Tipo de comision,Referencia,Monto,Fecha de facturacion,Fecha de cobro
0,1186303,false,COBRADO,LIQUIDACION_COLOMBIA,3182588339,100000.0,3/10/2023,24/10/2023
1,1186311,true,COBRADO,LIQUIDACION_COLOMBIA,3233270136,853382.5499999999,4/10/2023,4/10/2023
2,1186314,false,COBRADO,LIQUIDACION_COLOMBIA,3223109455,10162.5,4/10/2023,11/10/2023
3,1186316,false,COBRADO,LIQUIDACION_COLOMBIA,3142688054,2506.0499999999997,4/10/2023,6/10/2023
4,1186318,false,COBRADO,LIQUIDACION_COLOMBIA,3214595216,218440.0,4/10/2023,6/10/2023


,Id,Monto,Estado de cobro,Fecha de cobro,Fecha de facturacion,Status facturacion,Tipo de comision,Referencia
0,1,11890.0,COBRADO,15/8/2023,24/7/2023,COBRO_PARCIAL_COBRADO,INTERESES,6007350
1,2,23136.0,COBRADO,15/8/2023,11/7/2023,COBRO_PARCIAL_COBRADO,INTERESES,6006933
2,3,12191.0,COBRADO,15/8/2023,17/7/2023,COBRO_PARCIAL_COBRADO,INTERESES,6003875
3,4,11671.0,COBRADO,15/8/2023,17/7/2023,COBRO_PARCIAL_COBRADO,INTERESES,6002993R
4,5,31133.0,COBRADO,15/8/2023,11/7/2023,COBRO_PARCIAL_COBRADO,INTERESES,6006986


In [ ]:
import pandas as pd

# Supongamos que ya tienes tu DataFrame col_sel cargado
# Filtrar las filas
col_sel_filtrado = col_sel[col_sel['Tipo de comision'].isin([
    'LIQUIDACION_COLOMBIA',
    'MENSUALIDAD_COLOMBIA',
    'INSCRIPCION_COLOMBIA'
])]

# Si quieres reemplazar el DataFrame original
col_sel = col_sel_filtrado

In [ ]:
col_sel

,Id,Monto,Estado de cobro,Fecha de cobro,Fecha de facturacion,Status facturacion,Tipo de comision,Referencia
76353,83511,77149.0,COBRADO,27/9/2023,1/10/2023,COBRO_PARCIAL_COBRADO,MENSUALIDAD_COLOMBIA,3203035694
76354,83512,25675.0,COBRADO,27/9/2023,1/10/2023,COBRO_PARCIAL_COBRADO,MENSUALIDAD_COLOMBIA,3152249590
76355,83513,10464.0,COBRADO,27/9/2023,1/10/2023,COBRO_PARCIAL_COBRADO,MENSUALIDAD_COLOMBIA,3154444468
76356,83514,25527.0,COBRADO,27/9/2023,1/10/2023,COBRO_PARCIAL_INCOBRABLE,MENSUALIDAD_COLOMBIA,3125466749
76357,83515,27526.0,COBRADO,27/9/2023,1/10/2023,COBRO_PARCIAL_INCOBRABLE,MENSUALIDAD_COLOMBIA,3167088085
...,...,...,...,...,...,...,...,...
724933,791699,7254.0,COBRADO,24/10/2025,5/6/2025,COBRO_PARCIAL_COBRADO,MENSUALIDAD_COLOMBIA,3043726636
724934,791700,31580.0,COBRADO,24/10/2025,3/9/2025,COBRO_PARCIAL_COBRADO,MENSUALIDAD_COLOMBIA,3134845881
724935,791701,3827.0,COBRADO,24/10/2025,3/7/2025,COBRO_PARCIAL,MENSUALIDAD_COLOMBIA,3166190365
724936,791702,603841.0,COBRADO,24/10/2025,24/9/2025,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,3043886922


In [ ]:
import pandas as pd
import unicodedata

def _norm(s: str) -> str:
    s = '' if s is None else str(s)
    s = unicodedata.normalize("NFKD", s)
    s = ''.join(c for c in s if not unicodedata.combining(c))
    return ' '.join(s.lower().strip().split())

def align_to_template(df: pd.DataFrame, template_cols: list[str], defaults: dict | None = None) -> pd.DataFrame:
    """Devuelve un DF con EXACTAMENTE las columnas de template_cols.
       Si falta alguna, la crea con defaults[col] o NaN."""
    defaults = defaults or {}
    norm_map = {_norm(c): c for c in df.columns}
    cols = {}
    for t in template_cols:
        nt = _norm(t)
        if nt in norm_map:
            cols[t] = df[norm_map[nt]]
        else:
            cols[t] = pd.Series([defaults.get(t, pd.NA)] * len(df), index=df.index, dtype="object")
    return pd.DataFrame(cols, index=df.index)

# 1) Plantilla = columnas de fac_sel (en tu screenshot son 8 columnas)
template = fac_sel.columns.tolist()

# 2) Valor por defecto para 'Credito' en col_sel (respetando el tipo de fac_sel)
cred_default = 'false' if str(fac_sel['Credito'].dtype) == 'object' else False
defaults_col_sel = {'Credito': cred_default}

# 3) Alinear col_sel a la plantilla (esto también crea 'Fecha de facturacion' si no existe)
col_sel_aligned = align_to_template(col_sel, template, defaults=defaults_col_sel)

# 4) Concatenar
unificado = pd.concat([fac_sel, col_sel_aligned], ignore_index=True)

print(f"fac_sel: {fac_sel.shape}  |  col_sel alineado: {col_sel_aligned.shape}")
print(f"unificado: {unificado.shape}")
display(unificado.head(10))

fac_sel: (692247, 8)  |  col_sel alineado: (421645, 8)
unificado: (1113892, 8)


,Id,Credito,Status facturacion,Tipo de comision,Referencia,Monto,Fecha de facturacion,Fecha de cobro
0,1186303,false,COBRADO,LIQUIDACION_COLOMBIA,3182588339,100000.0,3/10/2023,24/10/2023
1,1186311,true,COBRADO,LIQUIDACION_COLOMBIA,3233270136,853382.5499999999,4/10/2023,4/10/2023
2,1186314,false,COBRADO,LIQUIDACION_COLOMBIA,3223109455,10162.5,4/10/2023,11/10/2023
3,1186316,false,COBRADO,LIQUIDACION_COLOMBIA,3142688054,2506.0499999999997,4/10/2023,6/10/2023
4,1186318,false,COBRADO,LIQUIDACION_COLOMBIA,3214595216,218440.0,4/10/2023,6/10/2023
5,1186321,false,COBRADO,LIQUIDACION_COLOMBIA,3115705677,290452.5,4/10/2023,6/10/2023
6,1186322,false,COBRADO,LIQUIDACION_COLOMBIA,3223109455,38951.1,4/10/2023,11/10/2023
7,1186323,false,COBRADO,LIQUIDACION_COLOMBIA,3137009218,4804.8,4/10/2023,6/10/2023
8,1186324,false,COBRADO,LIQUIDACION_COLOMBIA,3168124099,49280.0,4/10/2023,10/10/2023
9,1186326,false,COBRADO,LIQUIDACION_COLOMBIA,3003971213,570380.0,4/10/2023,6/10/2023


In [ ]:
unificado

,Id,Credito,Status facturacion,Tipo de comision,Referencia,Monto,Fecha de facturacion,Fecha de cobro
0,1186303,false,COBRADO,LIQUIDACION_COLOMBIA,3182588339,100000.0,3/10/2023,24/10/2023
1,1186311,true,COBRADO,LIQUIDACION_COLOMBIA,3233270136,853382.5499999999,4/10/2023,4/10/2023
2,1186314,false,COBRADO,LIQUIDACION_COLOMBIA,3223109455,10162.5,4/10/2023,11/10/2023
3,1186316,false,COBRADO,LIQUIDACION_COLOMBIA,3142688054,2506.0499999999997,4/10/2023,6/10/2023
4,1186318,false,COBRADO,LIQUIDACION_COLOMBIA,3214595216,218440.0,4/10/2023,6/10/2023
...,...,...,...,...,...,...,...,...
1113887,791699,false,COBRO_PARCIAL_COBRADO,MENSUALIDAD_COLOMBIA,3043726636,7254.0,5/6/2025,24/10/2025
1113888,791700,false,COBRO_PARCIAL_COBRADO,MENSUALIDAD_COLOMBIA,3134845881,31580.0,3/9/2025,24/10/2025
1113889,791701,false,COBRO_PARCIAL,MENSUALIDAD_COLOMBIA,3166190365,3827.0,3/7/2025,24/10/2025
1113890,791702,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,3043886922,603841.0,24/9/2025,24/10/2025


In [ ]:
import pandas as pd
import re

def _to_number_safe(x):
    s = str(x).strip()
    if not s:
        return pd.NA

    # Limpia moneda/espacios y convierte (1.234,56) -> -1.234,56
    s = s.replace("$", "").replace(" ", "")
    s = re.sub(r'^\((.*)\)$', r'-\1', s)

    # Encuentra el último separador
    last_dot = s.rfind('.')
    last_com = s.rfind(',')

    if last_dot == -1 and last_com == -1:
        return pd.to_numeric(s, errors="coerce")

    # El separador decimal es el que aparece más a la derecha
    if last_dot > last_com:
        dec = '.'
        thousands = ','
    else:
        dec = ','
        thousands = '.'

    # Elimina separador de miles
    s = s.replace(thousands, '')

    # Normaliza decimal a punto
    if dec == ',':
        s = s.replace(',', '.')

    return pd.to_numeric(s, errors="coerce")


# ----- 1) Partimos del DF unificado -----
df = unificado.copy()

for c in ["Id"]:
    if c in df.columns:
        df.drop(columns=c, inplace=True)

df["Credito"] = df["Credito"].astype(str).str.lower().str.strip()
df["Monto_num"] = df["Monto"].map(_to_number_safe).fillna(0)

# Parseo de fechas
df["Fecha de cobro"] = pd.to_datetime(df["Fecha de cobro"], dayfirst=True, errors="coerce")
df["Fecha de cobro"] = df["Fecha de cobro"].dt.to_period("M").dt.to_timestamp("M")

df["Fecha de facturacion"] = pd.to_datetime(df["Fecha de facturacion"], dayfirst=True, errors="coerce")
# (opcional) si quieres también a fin de mes:
# df["Fecha de facturacion"] = df["Fecha de facturacion"].dt.to_period("M").dt.to_timestamp("M")

# Claves originales (no incluimos 'Fecha de facturacion' para no cambiar la granularidad)
keys = ["Referencia", "Credito", "Status facturacion", "Tipo de comision", "Fecha de cobro"]

# Agregamos Monto y “pegamos” Fecha de facturacion (mínima no nula del grupo)
resultado = (
    df.groupby(keys, dropna=False, as_index=False)
      .agg(
          Monto=("Monto_num", "sum"),
          **{"Fecha de facturacion": ("Fecha de facturacion", "min")}
      )
      .sort_values(["Referencia", "Fecha de cobro"])
      .reset_index(drop=True)
)

# Orden opcional
resultado = resultado.sort_values(["Referencia", "Fecha de cobro"]).reset_index(drop=True)

print(f"Resultado: {resultado.shape}")
display(resultado.head(10))

Resultado: (786395, 7)


,Referencia,Credito,Status facturacion,Tipo de comision,Fecha de cobro,Monto,Fecha de facturacion
0,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2025-06-30,602031.000000,2025-06-26
1,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2025-07-31,627346.000000,2025-06-26
2,1000018033,false,COBRADO,MENSUALIDAD_COLOMBIA,2025-08-31,227237.000000,2025-08-05
3,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2025-08-31,3852.000000,2025-06-26
4,1000018033,false,COBRADO,MENSUALIDAD_COLOMBIA,2025-09-30,227237.000000,2025-09-03
5,1000018033,false,COBRO_PARCIAL,MENSUALIDAD_COLOMBIA,2025-10-31,54606.000000,2025-10-06
6,1000046281,false,COBRO_PARCIAL_INCOBRABLE,INSCRIPCION_COLOMBIA,2024-05-31,392457.000000,2024-05-03
7,1000120635,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2024-02-29,251033.000000,2024-02-13
8,1000120635,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2024-04-30,249418.016954,2024-02-13
9,1000120635,false,COBRADO,MENSUALIDAD_COLOMBIA,2024-05-31,140546.000000,2024-04-03


In [ ]:
resultado

,Referencia,Credito,Status facturacion,Tipo de comision,Fecha de cobro,Monto,Fecha de facturacion
0,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2025-06-30,602031.0,2025-06-26
1,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2025-07-31,627346.0,2025-06-26
2,1000018033,false,COBRADO,MENSUALIDAD_COLOMBIA,2025-08-31,227237.0,2025-08-05
3,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2025-08-31,3852.0,2025-06-26
4,1000018033,false,COBRADO,MENSUALIDAD_COLOMBIA,2025-09-30,227237.0,2025-09-03
...,...,...,...,...,...,...,...
786390,99990,true,COBRADO,LIQUIDACION_COLOMBIA,2021-12-31,1210878.0,2021-12-23
786391,99990,true,COBRADO,MENSUALIDAD_COLOMBIA,2021-12-31,1575770.0,2021-12-23
786392,99990,true,COBRADO,LIQUIDACION_COLOMBIA,2022-07-31,112390.0,2022-07-30
786393,99990,true,COBRADO,LIQUIDACION_COLOMBIA,2022-10-31,692027.0,2022-10-24


In [ ]:
import pandas as pd

# Asegúrate de que la clave de unión tenga el mismo tipo en ambos DataFrames
resultado['Referencia'] = resultado['Referencia'].astype(str)
rep_sel['Referencia']   = rep_sel['Referencia'].astype(str)

# Hacemos un merge left: se quedan las filas de resultado y se añade la columna 'Fecha de Inicio'
resultado = resultado.merge(
    rep_sel[['Referencia', 'Fecha de Inicio']],
    on='Referencia',
    how='left'
)

In [ ]:
import pandas as pd
import numpy as np

# =========================
# 0) Normalización y utilidades
# =========================
def _norm_ref_base(s: pd.Series) -> pd.Series:
    """Referencia base: quita espacios, '.0' final y sufijos '-n'."""
    s = s.astype(str).str.strip()
    s = s.str.replace(r'\.0$', '', regex=True)    # 12345.0 -> 12345
    s = s.str.replace(r'-\d+$', '', regex=True)   # quita -1, -2, ...
    return s

def _is_empty(x: pd.Series) -> pd.Series:
    return x.isna() | (x.astype(str).str.strip() == '')

# =========================
# 1) Copias y claves base
# =========================
res = resultado.copy()
src = rep_sel.copy()

# Normalizar clave
res['__ref_base__'] = _norm_ref_base(res['Referencia'])
src['__ref_base__'] = _norm_ref_base(src['Referencia'])

# Fechas
res['Fecha de cobro']  = pd.to_datetime(res['Fecha de cobro'],  dayfirst=True, errors='coerce')
res['Fecha de Inicio'] = pd.to_datetime(res['Fecha de Inicio'], dayfirst=True, errors='coerce')
src['Fecha de Inicio'] = pd.to_datetime(src['Fecha de Inicio'], dayfirst=True, errors='coerce')

# =========================
# 2) Columnas a rellenar (mismo nombre en rep_sel y resultado)
# =========================
cols_objetivo = [
    'Fecha de Inicio',
    'Apartado Mensual',
    'Comision Inicial',
    'Comision Mensual',
    'Vehiculo de ahorro',
    'Deuda inicial Fija'
]

# Asegurar que existan en 'res'
for c in cols_objetivo:
    if c not in res.columns:
        res[c] = pd.NA

# =========================
# 3) Separar referencias únicas vs. repetidas en rep_sel
# =========================
dup_mask = src.duplicated('__ref_base__', keep=False)
src_unique = src.loc[~dup_mask].set_index('__ref_base__')
src_dups   = src.loc[dup_mask].copy()

# =========================
# 4) Relleno para referencias ÚNICAS (merge directo por map)
# =========================
for dcol in cols_objetivo:
    if dcol not in src_unique.columns:
        continue
    was_empty = _is_empty(res[dcol])
    res.loc[was_empty, dcol] = res.loc[was_empty, '__ref_base__'].map(src_unique[dcol])

# =========================
# 5) Reglas para referencias REPETIDAS
#    - "old": fila con Fecha de Inicio más antigua
#    - "new": fila con Fecha de Inicio más reciente
#    - En resultado:
#        si Fecha de cobro < FechaInicio_new → usar valores "old"
#        si Fecha de cobro ≥ FechaInicio_new O es NaT → usar valores "new"
#    - Solo rellenamos donde esté vacío
# =========================
if not src_dups.empty:
    # ordenar por fecha
    src_dups = src_dups.sort_values(['__ref_base__', 'Fecha de Inicio'])
    rules = []

    for base, g in src_dups.groupby('__ref_base__', sort=False):
        g = g.dropna(subset=['Fecha de Inicio'])
        if g.empty:
            continue
        row_old = g.iloc[0]   # más antigua
        row_new = g.iloc[-1]  # más reciente
        item = {
            '__ref_base__': base,
            'threshold': row_new['Fecha de Inicio'],  # fecha que separa old/new
        }
        for dcol in cols_objetivo:
            item[f'old:{dcol}'] = row_old.get(dcol, pd.NA)
            item[f'new:{dcol}'] = row_new.get(dcol, pd.NA)
        rules.append(item)

    if rules:
        rules_df = pd.DataFrame(rules).set_index('__ref_base__')

        # Aplicar por referencia
        groups = res.groupby('__ref_base__').groups
        for base, idx_list in groups.items():
            if base not in rules_df.index:
                continue  # esta ref no es repetida en rep_sel
            thr = rules_df.at[base, 'threshold']

            idx = pd.Index(idx_list)
            rows = res.loc[idx]

            # Máscaras por Fecha de cobro
            mask_new = (rows['Fecha de cobro'].notna() & (rows['Fecha de cobro'] >= thr)) | rows['Fecha de cobro'].isna()
            mask_old = rows['Fecha de cobro'].notna() & (rows['Fecha de cobro'] <  thr)

            # Rellenar SOLO vacíos con valores new/old
            for dcol in cols_objetivo:
                # NEW
                was_empty_new = _is_empty(rows.loc[mask_new, dcol])
                if was_empty_new.any():
                    res.loc[idx[mask_new][was_empty_new], dcol] = rules_df.at[base, f'new:{dcol}']
                # OLD
                was_empty_old = _is_empty(rows.loc[mask_old, dcol])
                if was_empty_old.any():
                    res.loc[idx[mask_old][was_empty_old], dcol] = rules_df.at[base, f'old:{dcol}']

# =========================
# 6) Salida final
# =========================
resultado_actualizado = res.drop(columns='__ref_base__').reset_index(drop=True)

# (Opcional) verificar que no cambió el número de filas ni el orden principal
assert len(resultado_actualizado) == len(resultado), "El proceso cambió el número de filas."

print("Relleno completado. Filas:", len(resultado_actualizado))

Relleno completado. Filas: 786395


In [ ]:
# Total de referencias únicas en todo el DataFrame
total_referencias = resultado['Referencia'].nunique()

# Filtrar filas con Fecha de Inicio nula
sin_fecha = resultado[resultado['Fecha de Inicio'].isna()]

# Referencias únicas sin Fecha de Inicio
referencias_sin_fecha = sin_fecha['Referencia'].unique()
cantidad_sin_fecha = len(referencias_sin_fecha)

print("Total de referencias únicas en resultado:", total_referencias)
print("Cantidad de referencias únicas sin Fecha de Inicio:", cantidad_sin_fecha)
print("Listado de referencias sin Fecha de Inicio:")
print(referencias_sin_fecha)

Total de referencias únicas en resultado: 115688
Cantidad de referencias únicas sin Fecha de Inicio: 62038
Listado de referencias sin Fecha de Inicio:
['100017' '100076' '100086' ... '99961' '99990' 'Por Cobrar']


In [ ]:
import os, json, pandas as pd, gspread

# ========= 1) Obtener el secreto =========
def _get_secret(name: str):
    # Intenta primero en Colab
    try:
        from google.colab import userdata as _ud
        val = _ud.get(name)
        if val:
            return val
    except Exception:
        pass
    # Luego en entorno normal (GitHub / local)
    return os.environ.get(name)

creds_json = _get_secret("MI_JSON")
if not creds_json:
    raise RuntimeError(
        "No encontré el secreto 'MI_JSON'. "
        "En Colab, agrégalo en 'Secretos'. "
        "En GitHub/local, define la variable de entorno MI_JSON con el JSON del Service Account."
    )

creds_dict = json.loads(creds_json)

# ========= 2) Autorizar gspread =========
scopes = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive",
]
gc = gspread.service_account_from_dict(creds_dict, scopes=scopes)

# ========= 3) Abrir la hoja por ID y gid =========
sheet_id = "1JB8m0xVAJYhP2e57DcIe-MujFEXIbyRO2x76C30-9Pk"
gid      = 655613670  # pestaña específica

ws = gc.open_by_key(sheet_id).get_worksheet_by_id(gid)

# ========= 4) Traer todos los valores y recortar a primeras 23 columnas =========
valores = ws.get_all_values()
df = pd.DataFrame(valores).iloc[:, :23]   # solo columnas 0–22

# ========= 5) Mostrar muestra y shape =========
print(df.head())
print(f"Filas: {df.shape[0]}  |  Columnas: {df.shape[1]}")

         0                1                              2   \
0  ID Berex  # de Referencia             Correo electronico   
1    565565       1067969495         luis.mestrac@gmail.com   
2    568776       3023181377      miguelpaipa2735@gmail.com   
3    570328       1073681400          eliza18cs@hotmail.com   
4    571153       3022640512  rafaelhernandez2527@gmail.com   

                  3           4        5             6                7   \
0  Tipo de Documento      Cédula  Celular  Deuda Cierre  SUMA(N3:N17932)   
1                 CC  1067969495            130.981.450        130981450   
2                 CC    80132159             16.109.100         16109100   
3                 CC  1073681400              9.705.150          9705150   
4                 CC  1007680232             37.398.113         37398113   

         8                9   ...       13        14      15  \
0  Apartado  Fecha de Inicio  ...  C Powwi  Vehículo  Status   
1   1539412         3/8/2025  ...   

In [ ]:
# Usar la primera fila como encabezados y eliminarla del cuerpo
df.columns = df.iloc[0]           # asigna la fila 0 como nombres de columnas
df = df.drop(index=0).reset_index(drop=True)  # quita la fila 0 y rein

In [ ]:
import pandas as pd
import numpy as np

# =========================
# 0) Normalización y utilidades
# =========================
def _norm_ref_base(s: pd.Series) -> pd.Series:
    s = s.astype(str).str.strip()
    s = s.str.replace(r'\.0$', '', regex=True)    # 12345.0 -> 12345
    s = s.str.replace(r'-\d+$', '', regex=True)   # quita -1, -2, ...
    return s

def _is_empty(x: pd.Series) -> pd.Series:
    return x.isna() | (x.astype(str).str.strip() == '')

# =========================
# 1) Copias y claves base
# =========================
res = resultado_actualizado.copy()
src = df.copy()

res['__ref_base__'] = _norm_ref_base(res['Referencia'])
src['__ref_base__'] = _norm_ref_base(src['# de Referencia'])

# Asegurar fechas
res['Fecha de cobro']  = pd.to_datetime(res['Fecha de cobro'],  dayfirst=True, errors='coerce')
res['Fecha de Inicio'] = pd.to_datetime(res['Fecha de Inicio'], dayfirst=True, errors='coerce')
src['Fecha de Inicio'] = pd.to_datetime(src['Fecha de Inicio'], dayfirst=True, errors='coerce')

# Columnas origen (en df) y destino (en resultado_actualizado)
col_mensualidad = ' Mensualidad Total' if ' Mensualidad Total' in src.columns else 'Mensualidad Total'
pairs = [
    ('Fecha de Inicio',    'Fecha de Inicio'),
    ('Apartado Mensual',   'Apartado'),
    ('Comision Mensual',   col_mensualidad),
    ('Vehiculo de ahorro', 'Vehículo'),
    ('Deuda inicial Fija', 'Deuda Cierre'),
]

# Asegurar que columnas destino existan en res
for dcol, _ in pairs:
    if dcol not in res.columns:
        res[dcol] = pd.NA

# =========================
# 2) Separar referencias únicas vs repetidas en df
# =========================
dup_mask = src.duplicated('__ref_base__', keep=False)
src_unique = src.loc[~dup_mask].set_index('__ref_base__')
src_dups   = src.loc[dup_mask].copy()

# =========================
# 3) Relleno para referencias ÚNICAS (merge directo por map)
# =========================
for dcol, scol in pairs:
    if scol not in src_unique.columns:
        continue
    was_empty = _is_empty(res[dcol])
    res.loc[was_empty, dcol] = res.loc[was_empty, '__ref_base__'].map(src_unique[scol])

# =========================
# 4) Reglas para referencias REPETIDAS:
#    - "old": fila con Fecha de Inicio más antigua
#    - "new": fila con Fecha de Inicio más reciente
# =========================
if not src_dups.empty:
    # ordenar por fecha
    src_dups = src_dups.sort_values(['__ref_base__', 'Fecha de Inicio'])
    rules = []

    for base, g in src_dups.groupby('__ref_base__', sort=False):
        g = g.dropna(subset=['Fecha de Inicio'])
        if g.empty:
            continue
        row_old = g.iloc[0]
        row_new = g.iloc[-1]
        item = {
            '__ref_base__': base,
            'threshold': row_new['Fecha de Inicio'],  # fecha que separa old/new
        }
        # guardar valores old/new para cada destino
        for dcol, scol in pairs:
            item[f'old:{dcol}'] = row_old.get(scol, pd.NA)
            item[f'new:{dcol}'] = row_new.get(scol, pd.NA)
        rules.append(item)

    if rules:
        rules_df = pd.DataFrame(rules).set_index('__ref_base__')

        # aplicar por referencia
        for base, sub_idx in res.groupby('__ref_base__').groups.items():
            if base not in rules_df.index:
                continue  # esta ref no es repetida en df
            thr = rules_df.at[base, 'threshold']

            # filas de esta ref en res
            idx = pd.Index(sub_idx)
            rows = res.loc[idx]

            # submáscaras por fecha de cobro
            mask_new = (rows['Fecha de cobro'].notna() & (rows['Fecha de cobro'] >= thr)) | rows['Fecha de cobro'].isna()
            mask_old = rows['Fecha de cobro'].notna() & (rows['Fecha de cobro'] <  thr)

            # rellenar SOLO vacíos con valores new/old
            for dcol, _ in pairs:
                # NEW
                was_empty_new = _is_empty(rows.loc[mask_new, dcol])
                if was_empty_new.any():
                    res.loc[idx[mask_new][was_empty_new], dcol] = rules_df.at[base, f'new:{dcol}']
                # OLD
                was_empty_old = _is_empty(rows.loc[mask_old, dcol])
                if was_empty_old.any():
                    res.loc[idx[mask_old][was_empty_old], dcol] = rules_df.at[base, f'old:{dcol}']

# =========================
# 5) Salida final
# =========================
resultado_actualizado = res.drop(columns='__ref_base__').reset_index(drop=True)
print("Relleno completado. Filas:", len(resultado_actualizado))

Relleno completado. Filas: 786395


In [ ]:
import pandas as pd

# 1️⃣ Filtrar filas: mantener solo las referencias que sean enteramente numéricas
#    isdigit() devuelve True solo si la cadena son dígitos del 0-9
resultado_actualizado = resultado_actualizado[resultado_actualizado['Referencia'].astype(str).str.isdigit()].copy()

# 2️⃣ Eliminar la columna 'Comision Inicial' si existe
if 'Comision Inicial' in resultado_actualizado.columns:
    resultado_actualizado.drop(columns='Comision Inicial', inplace=True)

# Verificación
print("Filas restantes:", resultado_actualizado.shape[0])
print("Columnas actuales:", list(resultado_actualizado.columns))

Filas restantes: 786205
Columnas actuales: ['Referencia', 'Credito', 'Status facturacion', 'Tipo de comision', 'Fecha de cobro', 'Monto', 'Fecha de facturacion', 'Fecha de Inicio', 'Apartado Mensual', 'Comision Mensual', 'Vehiculo de ahorro', 'Deuda inicial Fija']


In [ ]:
import pandas as pd
import numpy as np

def to_number_mixed_dot(series: pd.Series) -> pd.Series:
    # Forzamos string dtype robusto
    s = series.astype("string").str.strip()

    # Negativos con paréntesis
    s = s.str.replace(r'^\((.*)\)$', r'-\1', regex=True)

    # Notación científica -> directo
    m_sci = s.str.contains(r'^[+-]?\d*\.?\d+(e|E)[+-]?\d+$', na=False)
    out = pd.Series(np.nan, index=s.index, dtype="float64")
    if m_sci.any():
        out.loc[m_sci] = pd.to_numeric(s[m_sci], errors="coerce")

    # Resto: limpiamos todo lo que no sea dígito, punto o signo; quitamos comas (miles)
    t = s[~m_sci].copy()
    t = t.str.replace(r'[^0-9\.\-]', '', regex=True)   # quita símbolos, espacios, etc.
    t = t.str.replace(',', '', regex=False)            # (por si había)

    # Contar puntos
    dot_count = t.str.count(r'\.')

    # >1 punto => todos son miles (quitar todos los puntos)
    idx_multi = dot_count > 1
    if idx_multi.any():
        out.loc[idx_multi.index[idx_multi]] = pd.to_numeric(
            t[idx_multi].str.replace('.', '', regex=False),
            errors='coerce'
        )

    # 1 punto => decidir por nº de dígitos antes del punto
    idx_one = dot_count == 1
    if idx_one.any():
        one = t[idx_one]
        # parte antes del punto sin signo
        before = one.str.split('.', n=1).str[0].str.replace('-', '', regex=False)
        m_dec = before.str.len().astype("Int64") >= 4  # ≥4 dígitos antes => decimal
        if m_dec.any():
            out.loc[one.index[m_dec]] = pd.to_numeric(one[m_dec], errors='coerce')
        if (~m_dec).any():
            out.loc[one.index[~m_dec]] = pd.to_numeric(
                one[~m_dec].str.replace('.', '', regex=False),
                errors='coerce'
            )

    # 0 puntos => número directo
    idx_zero = dot_count == 0
    if idx_zero.any():
        out.loc[t.index[idx_zero]] = pd.to_numeric(t[idx_zero], errors='coerce')

    return out

# Aplica a tus columnas
cols_a_convertir = ['Apartado Mensual', 'Comision Mensual', 'Deuda inicial Fija']
for c in cols_a_convertir:
    if c in resultado_actualizado.columns:
        resultado_actualizado[c] = to_number_mixed_dot(resultado_actualizado[c])

# Verificación rápida
print(resultado_actualizado[cols_a_convertir].dtypes)
print(resultado_actualizado[cols_a_convertir].head(10))

/tmp/ipython-input-705226084.py:12: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  m_sci = s.str.contains(r'^[+-]?\d*\.?\d+(e|E)[+-]?\d+$', na=False)
/tmp/ipython-input-705226084.py:12: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  m_sci = s.str.contains(r'^[+-]?\d*\.?\d+(e|E)[+-]?\d+$', na=False)
/tmp/ipython-input-705226084.py:12: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  m_sci = s.str.contains(r'^[+-]?\d*\.?\d+(e|E)[+-]?\d+$', na=False)


Apartado Mensual      float64
Comision Mensual      float64
Deuda inicial Fija    float64
dtype: object
   Apartado Mensual  Comision Mensual  Deuda inicial Fija
0         611362.66         227237.00          43346600.0
1         611362.66         227237.00          43346600.0
2         611362.66         227237.00          43346600.0
3         611362.66         227237.00          43346600.0
4         611362.66         227237.00          43346600.0
5         611362.66         227237.00          43346600.0
6         389316.26          67611.23          11421445.0
7         246716.02          70272.50          11953700.0
8         246716.02          70272.50          11953700.0
9         246716.02          70272.50          11953700.0


In [ ]:
# Total de referencias únicas en todo el DataFrame
total_referencias = resultado_actualizado['Referencia'].nunique()

# Filtrar filas con Fecha de Inicio nula
sin_fecha = resultado_actualizado[resultado_actualizado['Fecha de Inicio'].isna()]

# Referencias únicas sin Fecha de Inicio
referencias_sin_fecha = sin_fecha['Referencia'].unique()
cantidad_sin_fecha = len(referencias_sin_fecha)

print("Total de referencias únicas en resultado:", total_referencias)
print("Cantidad de referencias únicas sin Fecha de Inicio:", cantidad_sin_fecha)
print("Listado de referencias sin Fecha de Inicio:")
print(referencias_sin_fecha)

Total de referencias únicas en resultado: 115651
Cantidad de referencias únicas sin Fecha de Inicio: 16837
Listado de referencias sin Fecha de Inicio:
['117857' '3000004' '3000006' ... '3174152116' '46827' '52310468']


In [ ]:
import pandas as pd
import re

# Aseguramos que sea string
ref_series = df['# de Referencia'].astype(str)

# 1️⃣ Patrón: al menos un dígito y al menos un carácter que NO sea dígito
mask = ref_series.str.contains(r'\d') & ref_series.str.contains(r'\D')

df_con_caracteres = df[mask].copy()

print(df_con_caracteres.head())
print("Total de referencias con números + otros caracteres:", df_con_caracteres.shape[0])

0     ID Berex # de Referencia              Correo electronico  \
13930   491756    3106441511-1       LLERENAPADILLAG@GMAIL.COM   
15027   500231    3222169837-1    UYABANANDRES130910@GMAIL.COM   
15725   486992    3005732101-1    LINDA.DUARTE1995@OUTLOOK.COM   
16723   493041    3132075879-1  VALENTINASANCHEZ1107@GMAIL.COM   
16993   492082      79369661-1           ANDRES0106V@YAHOO.COM   

0     Tipo de Documento      Cédula     Celular Deuda Cierre SUMA(N3:N17932)  \
13930                CC    45371576  3106441511    7.345.800         7345800   
15027                CC  1057548267  3222169837   20.189.500        20189500   
15725                CC  1065819988  3005732101   13.502.500        13502500   
16723                CC  1073714622  3132075879    7.981.050         7981050   
16993                CC    79369661  3142071231   14.635.348        14635348   

0     Apartado Fecha de Inicio  ... C Powwi Vehículo    Status  \
13930   179317       2/10/2024  ...   10504    COINK  Ba

In [ ]:
# Total de referencias únicas en todo el DataFrame
total_referencias = resultado_actualizado['Referencia'].nunique()

# Filtrar filas con Fecha de Inicio nula
sin_fecha = resultado_actualizado[resultado_actualizado['Fecha de Inicio'].isna()]

# Referencias únicas sin Fecha de Inicio
referencias_sin_fecha = sin_fecha['Referencia'].unique()
cantidad_sin_fecha = len(referencias_sin_fecha)

print("Total de referencias únicas en resultado:", total_referencias)
print("Cantidad de referencias únicas sin Fecha de Inicio:", cantidad_sin_fecha)
print("Listado de referencias sin Fecha de Inicio:")
print(referencias_sin_fecha)

Total de referencias únicas en resultado: 115651
Cantidad de referencias únicas sin Fecha de Inicio: 16837
Listado de referencias sin Fecha de Inicio:
['117857' '3000004' '3000006' ... '3174152116' '46827' '52310468']


In [ ]:
# Asegúrate de que 'Fecha de Inicio' sea de tipo datetime
resultado_actualizado['Fecha de Inicio'] = pd.to_datetime(resultado_actualizado['Fecha de Inicio'], errors='coerce')

# Crear la nueva columna 'Mes_Año' con el formato "MM-YYYY"
resultado_actualizado['Mes_Año'] = resultado_actualizado['Fecha de Inicio'].dt.strftime('%m-%Y')

In [ ]:
import numpy as np
import pandas as pd

# Asegúrate de que ambas columnas sean datetime
resultado_actualizado['Fecha de Inicio'] = pd.to_datetime(resultado_actualizado['Fecha de Inicio'], errors='coerce')
resultado_actualizado['Fecha de cobro'] = pd.to_datetime(resultado_actualizado['Fecha de cobro'], errors='coerce')

# Calcular diferencia en meses entre Fecha de Inicio y Fecha de cobro
# (año de cobro - año de inicio)*12 + (mes de cobro - mes de inicio)
resultado_actualizado['Mes_Cobro'] = (
    (resultado_actualizado['Fecha de cobro'].dt.year - resultado_actualizado['Fecha de Inicio'].dt.year) * 12 +
    (resultado_actualizado['Fecha de cobro'].dt.month - resultado_actualizado['Fecha de Inicio'].dt.month)
)

# Si Fecha de Inicio o Fecha de cobro son nulos, poner NaN
resultado_actualizado['Mes_Cobro'] = np.where(
    resultado_actualizado['Fecha de Inicio'].notna() & resultado_actualizado['Fecha de cobro'].notna(),
    resultado_actualizado['Mes_Cobro'],
    np.nan
)

In [ ]:
# Filtrar manteniendo solo los Mes_Cobro no nulos y >= 0
resultado_actualizado = resultado_actualizado[
    resultado_actualizado['Mes_Cobro'].notna() & (resultado_actualizado['Mes_Cobro'] >= 0)
].reset_index(drop=True)

In [ ]:
resultado_actualizado

,Referencia,Credito,Status facturacion,Tipo de comision,Fecha de cobro,Monto,Fecha de facturacion,Fecha de Inicio,Apartado Mensual,Comision Mensual,Vehiculo de ahorro,Deuda inicial Fija,Mes_Año,Mes_Cobro
0,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2025-06-30,602031.0,2025-06-26,2025-06-16,611362.66,227237.0,SKANDIA,43346600.0,06-2025,0.0
1,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2025-07-31,627346.0,2025-06-26,2025-06-16,611362.66,227237.0,SKANDIA,43346600.0,06-2025,1.0
2,1000018033,false,COBRADO,MENSUALIDAD_COLOMBIA,2025-08-31,227237.0,2025-08-05,2025-06-16,611362.66,227237.0,SKANDIA,43346600.0,06-2025,2.0
3,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2025-08-31,3852.0,2025-06-26,2025-06-16,611362.66,227237.0,SKANDIA,43346600.0,06-2025,2.0
4,1000018033,false,COBRADO,MENSUALIDAD_COLOMBIA,2025-09-30,227237.0,2025-09-03,2025-06-16,611362.66,227237.0,SKANDIA,43346600.0,06-2025,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765853,99990,false,COBRADO,MENSUALIDAD_COLOMBIA,2021-12-31,157577.0,2021-12-05,2021-06-16,519374.00,157577.0,A&V,29414564.0,06-2021,6.0
765854,99990,true,COBRADO,LIQUIDACION_COLOMBIA,2021-12-31,1210878.0,2021-12-23,2021-06-16,519374.00,157577.0,A&V,29414564.0,06-2021,6.0
765855,99990,true,COBRADO,MENSUALIDAD_COLOMBIA,2021-12-31,1575770.0,2021-12-23,2021-06-16,519374.00,157577.0,A&V,29414564.0,06-2021,6.0
765856,99990,true,COBRADO,LIQUIDACION_COLOMBIA,2022-07-31,112390.0,2022-07-30,2021-06-16,519374.00,157577.0,A&V,29414564.0,06-2021,13.0


#Arreglo

In [ ]:
rep_sel = rep_sel[["Referencia", "Apartado Mensual", "Deuda Resuelve", "Deuda inicial Fija"]]

In [ ]:
df = df[["# de Referencia", "Apartado", "Deuda Cierre"]]

In [ ]:
# Elimina puntos y convierte a float
df["Deuda Cierre"] = (
    df["Deuda Cierre"]
      .astype(str)           # asegúrate de que sea string
      .str.replace('.', '', regex=False)  # quita separadores de miles
      .str.replace(',', '.', regex=False) # por si acaso hay comas decimales
      .astype(float)
)

In [ ]:
# 1️⃣ Primero, asegúrate de que las columnas clave tengan el mismo tipo
df['# de Referencia'] = df['# de Referencia'].astype(str)
resultado_actualizado['Referencia'] = resultado_actualizado['Referencia'].astype(str)

# 2️⃣ Crea un diccionario de referencia→Deuda Cierre para hacer el mapeo rápido
mapa_cierre = df.set_index('# de Referencia')['Deuda Cierre']

# 3️⃣ Reemplaza solo donde Deuda inicial Fija < 5,000,000
mask = resultado_actualizado['Deuda inicial Fija'] < 5_000_000
resultado_actualizado.loc[mask, 'Deuda inicial Fija'] = (
    resultado_actualizado.loc[mask, 'Referencia'].map(mapa_cierre)
)

# 4️⃣ (opcional) verifica
print(resultado_actualizado['Deuda inicial Fija'].head())

0    43346600.0
1    43346600.0
2    43346600.0
3    43346600.0
4    43346600.0
Name: Deuda inicial Fija, dtype: float64


In [ ]:
df["Deuda Cierre"] = pd.to_numeric(df["Deuda Cierre"], errors="coerce")
resultado_actualizado["Deuda inicial Fija"] = pd.to_numeric(
    resultado_actualizado["Deuda inicial Fija"], errors="coerce"
)

# Unimos por referencia
merged = resultado_actualizado.merge(
    df[["# de Referencia", "Deuda Cierre"]],
    left_on="Referencia",
    right_on="# de Referencia",
    how="left",
    suffixes=("", "_df")
)

# Creamos máscara donde la diferencia sea mayor a 50,000
mask = (merged["Deuda Cierre"].notna() &
        (merged["Deuda Cierre"] - merged["Deuda inicial Fija"]).abs() > 50000)

# Reemplazamos en resultado_actualizado solo para esas referencias
resultado_actualizado.loc[mask, "Deuda inicial Fija"] = merged.loc[mask, "Deuda Cierre"]

In [ ]:
columnas_eliminar = ['Comision Mensual']

resultado_actualizado = resultado_actualizado.drop(columns=columnas_eliminar)

In [ ]:
resultado_actualizado

,Referencia,Credito,Status facturacion,Tipo de comision,Fecha de cobro,Monto,Fecha de facturacion,Fecha de Inicio,Apartado Mensual,Vehiculo de ahorro,Deuda inicial Fija,Mes_Año,Mes_Cobro
0,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2025-06-30,602031.0,2025-06-26,2025-06-16,611362.66,SKANDIA,43346600.0,06-2025,0.0
1,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2025-07-31,627346.0,2025-06-26,2025-06-16,611362.66,SKANDIA,43346600.0,06-2025,1.0
2,1000018033,false,COBRADO,MENSUALIDAD_COLOMBIA,2025-08-31,227237.0,2025-08-05,2025-06-16,611362.66,SKANDIA,43346600.0,06-2025,2.0
3,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2025-08-31,3852.0,2025-06-26,2025-06-16,611362.66,SKANDIA,43346600.0,06-2025,2.0
4,1000018033,false,COBRADO,MENSUALIDAD_COLOMBIA,2025-09-30,227237.0,2025-09-03,2025-06-16,611362.66,SKANDIA,43346600.0,06-2025,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
765853,99990,false,COBRADO,MENSUALIDAD_COLOMBIA,2021-12-31,157577.0,2021-12-05,2021-06-16,519374.00,A&V,29414564.0,06-2021,6.0
765854,99990,true,COBRADO,LIQUIDACION_COLOMBIA,2021-12-31,1210878.0,2021-12-23,2021-06-16,519374.00,A&V,29414564.0,06-2021,6.0
765855,99990,true,COBRADO,MENSUALIDAD_COLOMBIA,2021-12-31,1575770.0,2021-12-23,2021-06-16,519374.00,A&V,29414564.0,06-2021,6.0
765856,99990,true,COBRADO,LIQUIDACION_COLOMBIA,2022-07-31,112390.0,2022-07-30,2021-06-16,519374.00,A&V,29414564.0,06-2021,13.0


In [ ]:
resultado_actualizado.loc[
    resultado_actualizado['Referencia'] == '3223168906',
    'Apartado Mensual'
] = resultado_actualizado.loc[
    resultado_actualizado['Referencia'] == '3223168906',
    'Apartado Mensual'
] / 100

#Dealer

In [ ]:
import os, json, base64, pandas as pd, gspread
from google.oauth2.service_account import Credentials

# ========= 1) Cargar secreto MI_JSON (Colab o entorno normal) =========
def _get_secret(name: str):
    try:
        from google.colab import userdata as _ud
        val = _ud.get(name)
        if val:
            return val
    except Exception:
        pass
    return os.environ.get(name)

raw_json = _get_secret("MI_JSON")
if not raw_json:
    raise RuntimeError(
        "No se encontró el secreto MI_JSON. "
        "En Colab, agrégalo en 'Secretos'. "
        "En GitHub/local, define la variable de entorno MI_JSON con el JSON del Service Account."
    )

# Puede venir en texto plano o Base64
try:
    sa_info = json.loads(raw_json)
except json.JSONDecodeError:
    sa_info = json.loads(base64.b64decode(raw_json).decode("utf-8"))

# ========= 2) Autorizar gspread =========
creds = Credentials.from_service_account_info(
    sa_info,
    scopes=[
        "https://www.googleapis.com/auth/spreadsheets.readonly",
        "https://www.googleapis.com/auth/drive.readonly",
    ],
)
client = gspread.authorize(creds)

# ========= 3) Abrir la hoja de cálculo y pestaña específica =========
spreadsheet_id = "1v7Bg9ss5ZTAIA5J4DJ8THmSMHs5ILrUeLgBZkE3NCMs"
worksheet_gid  = 678352849  # GID de la pestaña
ws = client.open_by_key(spreadsheet_id).get_worksheet_by_id(worksheet_gid)

# ========= 4) Convertir a DataFrame =========
toco_dealer = pd.DataFrame(ws.get_all_records())

print("Filas:", toco_dealer.shape[0], "| Columnas:", toco_dealer.shape[1])
print(toco_dealer.head())

Filas: 116340 | Columnas: 7
  bank_reference     cm      am       dbt       vh status fecha_de_inicio
0     1140875425  79947  233090  11335500  skandia   drop      2025-04-15
1                 38641  176268   6494250            drop      2019-07-31
2     3022623789  70487  218395   9745600    powwi   drop      2025-02-21
3              1  58402  287790   9815400            drop      2019-07-26
4              1  35317  173981   5935600            drop      2019-07-19


In [ ]:
# Seleccionar y renombrar columnas
toco_dealer = toco_dealer[['am', 'dbt', 'bank_reference']].rename(
    columns={
        'am': 'Apartado Mensual',
        'dbt': 'Deuda inicial Fija',
        'bank_reference': 'Referencia'
    }
)

# Verificar resultado
print(toco_dealer.shape)

(116340, 3)


In [ ]:
# Apartado Mensual
toco_dealer['Apartado Mensual'] = (
    toco_dealer['Apartado Mensual']
    .astype(str)
    .str.replace(r'[^\d.]', '', regex=True)
    .replace('', pd.NA)                   # Reemplaza cadenas vacías por NA
    .pipe(pd.to_numeric, errors='coerce') # Convierte a float y deja NaN donde no pueda
)

# Deuda inicial Fija
toco_dealer['Deuda inicial Fija'] = (
    toco_dealer['Deuda inicial Fija']
    .astype(str)
    .str.replace(r'[^\d.]', '', regex=True)
    .replace('', pd.NA)
    .pipe(pd.to_numeric, errors='coerce')
)

# Comprobar tipos y algunos valores
print(toco_dealer.dtypes)

Apartado Mensual      float64
Deuda inicial Fija    float64
Referencia             object
dtype: object


In [ ]:
# Asegurarnos de que las columnas sean numéricas
resultado_actualizado['Apartado Mensual']   = pd.to_numeric(resultado_actualizado['Apartado Mensual'], errors='coerce')
resultado_actualizado['Deuda inicial Fija'] = pd.to_numeric(resultado_actualizado['Deuda inicial Fija'], errors='coerce')

# Calcular AM/DB (con protección ante divisiones por 0 o NaN)
resultado_actualizado['AM/DB'] = (
    resultado_actualizado['Apartado Mensual'] /
    resultado_actualizado['Deuda inicial Fija']
)

In [ ]:
resultado_actualizado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 765858 entries, 0 to 765857
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   Referencia            765858 non-null  object        
 1   Credito               765858 non-null  object        
 2   Status facturacion    765858 non-null  object        
 3   Tipo de comision      765858 non-null  object        
 4   Fecha de cobro        765858 non-null  datetime64[ns]
 5   Monto                 765858 non-null  float64       
 6   Fecha de facturacion  765858 non-null  datetime64[ns]
 7   Fecha de Inicio       765858 non-null  datetime64[ns]
 8   Apartado Mensual      765858 non-null  float64       
 9   Vehiculo de ahorro    765858 non-null  object        
 10  Deuda inicial Fija    765857 non-null  float64       
 11  Mes_Año               765858 non-null  object        
 12  Mes_Cobro             765858 non-null  float64       
 13 

In [ ]:
import numpy as np

# 1) Asegurar tipos para la clave
resultado_actualizado['Referencia'] = resultado_actualizado['Referencia'].astype(str)
toco_dealer['Referencia']           = toco_dealer['Referencia'].astype(str)

# 2) Máscara de filas a considerar
mask = (
    (resultado_actualizado['AM/DB'] > 0.045) |
    (resultado_actualizado['AM/DB'] < 0.0099)
)

# 3) Mapas desde toco_dealer
map_apartado = dict(zip(toco_dealer['Referencia'], toco_dealer['Apartado Mensual']))
map_deuda    = dict(zip(toco_dealer['Referencia'], toco_dealer['Deuda inicial Fija']))

# 4) Series mapeadas (pueden venir con NaN si no existe la referencia)
mapped_apartado = resultado_actualizado['Referencia'].map(map_apartado)
mapped_deuda    = resultado_actualizado['Referencia'].map(map_deuda)

# 5) Reemplazar solo si: (mask) y (hay valor mapeado no nulo)
resultado_actualizado['Apartado Mensual'] = np.where(
    mask & mapped_apartado.notna(),
    mapped_apartado,
    resultado_actualizado['Apartado Mensual']
)

resultado_actualizado['Deuda inicial Fija'] = np.where(
    mask & mapped_deuda.notna(),
    mapped_deuda,
    resultado_actualizado['Deuda inicial Fija']
)

In [ ]:
# Lista de años permitidos
anios_validos = [str(a) for a in range(2021, 2031)]

# Filtrar resultado_actualizado
resultado_actualizado = resultado_actualizado[
    resultado_actualizado['Mes_Año'].str[-4:].isin(anios_validos)
].copy()

In [ ]:
import numpy as np

# Calcular cuartiles e IQR
Q1 = resultado_actualizado['AM/DB'].quantile(0.25)
Q3 = resultado_actualizado['AM/DB'].quantile(0.75)
IQR = Q3 - Q1

# Límites para definir atípicos
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Crear la columna: 1 si es atípico, 0 si no
resultado_actualizado['AM/DB_outlier'] = np.where(
    (resultado_actualizado['AM/DB'] < lower_bound) |
    (resultado_actualizado['AM/DB'] > upper_bound),
    1,   # Es atípico
    0    # No es atípico
)


In [ ]:
import numpy as np
import pandas as pd

# Definir los límites en fracción (no en %)
bins = [ -np.inf, 0.015, 0.0199, 0.025, np.inf ]
labels = [
    "< 1.5%",
    "1.5% – 1.99%",
    "2% – 2.5%",
    "> 2.5%"
]

# Crear la columna de rango
resultado_actualizado['Rango_AM_DB'] = pd.cut(
    resultado_actualizado['AM/DB'],
    bins=bins,
    labels=labels,
    right=True,   # incluye el límite superior
    include_lowest=True
)

In [ ]:
import numpy as np
import pandas as pd

# Límites en pesos (ajusta si tu columna ya está en millones)
bins_deuda   = [-np.inf, 35_000_000, 60_000_000, 100_000_000, np.inf]
labels_deuda = [
    "< 35 M",
    "35 – 60 M",
    "60 – 100 M",
    "> 100 M"
]

# Crear la nueva columna de rango
resultado_actualizado["Rango_de_deuda"] = pd.cut(
    resultado_actualizado["Deuda inicial Fija"],
    bins=bins_deuda,
    labels=labels_deuda,
    right=True,
    include_lowest=True
)

In [ ]:
# Asegurarnos de que las columnas sean numéricas
resultado_actualizado['Apartado Mensual']   = pd.to_numeric(resultado_actualizado['Apartado Mensual'], errors='coerce')
resultado_actualizado['Deuda inicial Fija'] = pd.to_numeric(resultado_actualizado['Deuda inicial Fija'], errors='coerce')

# Calcular AM/DB (con protección ante divisiones por 0 o NaN)
resultado_actualizado['AM/DB'] = (
    resultado_actualizado['Apartado Mensual'] /
    resultado_actualizado['Deuda inicial Fija']
)

In [ ]:
resultado_actualizado

,Referencia,Credito,Status facturacion,Tipo de comision,Fecha de cobro,Monto,Fecha de facturacion,Fecha de Inicio,Apartado Mensual,Vehiculo de ahorro,Deuda inicial Fija,Mes_Año,Mes_Cobro,AM/DB,AM/DB_outlier,Rango_AM_DB,Rango_de_deuda
0,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2025-06-30,602031.0,2025-06-26,2025-06-16,611362.66,SKANDIA,43346600.0,06-2025,0.0,0.014104,0,< 1.5%,35 – 60 M
1,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2025-07-31,627346.0,2025-06-26,2025-06-16,611362.66,SKANDIA,43346600.0,06-2025,1.0,0.014104,0,< 1.5%,35 – 60 M
2,1000018033,false,COBRADO,MENSUALIDAD_COLOMBIA,2025-08-31,227237.0,2025-08-05,2025-06-16,611362.66,SKANDIA,43346600.0,06-2025,2.0,0.014104,0,< 1.5%,35 – 60 M
3,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2025-08-31,3852.0,2025-06-26,2025-06-16,611362.66,SKANDIA,43346600.0,06-2025,2.0,0.014104,0,< 1.5%,35 – 60 M
4,1000018033,false,COBRADO,MENSUALIDAD_COLOMBIA,2025-09-30,227237.0,2025-09-03,2025-06-16,611362.66,SKANDIA,43346600.0,06-2025,3.0,0.014104,0,< 1.5%,35 – 60 M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765853,99990,false,COBRADO,MENSUALIDAD_COLOMBIA,2021-12-31,157577.0,2021-12-05,2021-06-16,519374.00,A&V,29414564.0,06-2021,6.0,0.017657,0,1.5% – 1.99%,< 35 M
765854,99990,true,COBRADO,LIQUIDACION_COLOMBIA,2021-12-31,1210878.0,2021-12-23,2021-06-16,519374.00,A&V,29414564.0,06-2021,6.0,0.017657,0,1.5% – 1.99%,< 35 M
765855,99990,true,COBRADO,MENSUALIDAD_COLOMBIA,2021-12-31,1575770.0,2021-12-23,2021-06-16,519374.00,A&V,29414564.0,06-2021,6.0,0.017657,0,1.5% – 1.99%,< 35 M
765856,99990,true,COBRADO,LIQUIDACION_COLOMBIA,2022-07-31,112390.0,2022-07-30,2021-06-16,519374.00,A&V,29414564.0,06-2021,13.0,0.017657,0,1.5% – 1.99%,< 35 M


In [ ]:
import pandas as pd

df = resultado_actualizado.copy()

# 0) Quitar AM/DB si existe
for c in df.columns:
    if c.strip().lower().replace(" ", "") in {"am/db", "amdb"}:
        df = df.drop(columns=[c])
        break

# 1) Detectar nombre real de la columna Mes_Año / Mes_Ano
mes_anio_col = None
for cand in ["Mes_Año", "Mes_Ano", "Mes_AÑO", "Mes_AÑO", "Mes_Ano "]:
    if cand in df.columns:
        mes_anio_col = cand
        break
if mes_anio_col is None:
    raise KeyError("No encontré la columna 'Mes_Año' / 'Mes_Ano' en el DataFrame.")

# 2) Llaves de agrupación
keys = ["Credito", "Tipo de comision", "Vehiculo de ahorro", mes_anio_col, "Mes_Cobro"]

# 3) Verificación mínima de columnas requeridas
for k in keys + ["Referencia", "Monto"]:
    if k not in df.columns:
        raise KeyError(f"Falta la columna requerida: {k}")

# 4) Agregación final: sum(Monto) y conteo de referencias únicas
df_final = (
    df.groupby(keys, dropna=False)
      .agg(
          Monto=("Monto", "sum"),
          Referencias_unicas=("Referencia", pd.Series.nunique),
      )
      .reset_index()
)

# (Opcional) reordenar columnas
df_final = df_final[keys + ["Referencias_unicas", "Monto"]]

print("Filas originales:", len(df))
print("Filas consolidadas:", len(df_final))
df_final.head()

Filas originales: 678640
Filas consolidadas: 12352


,Credito,Tipo de comision,Vehiculo de ahorro,Mes_Año,Mes_Cobro,Referencias_unicas,Monto
0,false,INSCRIPCION_COLOMBIA,A&V,01-2021,0.0,626,263814136.0
1,false,INSCRIPCION_COLOMBIA,A&V,01-2021,1.0,999,248805465.0
2,false,INSCRIPCION_COLOMBIA,A&V,01-2021,2.0,535,3819623.0
3,false,INSCRIPCION_COLOMBIA,A&V,01-2021,3.0,63,440980.0
4,false,INSCRIPCION_COLOMBIA,A&V,01-2021,4.0,17,85820.0


In [ ]:
df_final

,Credito,Tipo de comision,Vehiculo de ahorro,Mes_Año,Mes_Cobro,Referencias_unicas,Monto
0,false,INSCRIPCION_COLOMBIA,A&V,01-2021,0.0,626,263814136.0
1,false,INSCRIPCION_COLOMBIA,A&V,01-2021,1.0,999,248805465.0
2,false,INSCRIPCION_COLOMBIA,A&V,01-2021,2.0,535,3819623.0
3,false,INSCRIPCION_COLOMBIA,A&V,01-2021,3.0,63,440980.0
4,false,INSCRIPCION_COLOMBIA,A&V,01-2021,4.0,17,85820.0
...,...,...,...,...,...,...,...
12347,true,MENSUALIDAD_COLOMBIA,powwi,05-2022,7.0,3,3667968.0
12348,true,MENSUALIDAD_COLOMBIA,powwi,05-2022,8.0,1,520536.0
12349,true,MENSUALIDAD_COLOMBIA,powwi,05-2022,13.0,1,639864.0
12350,true,MENSUALIDAD_COLOMBIA,powwi,06-2022,7.0,1,818595.0


In [ ]:
# Columnas que queremos conservar
cols = [
    "Referencia",
    "Mes_Año",
    "Apartado Mensual",
    "Deuda inicial Fija",
    "AM/DB",
    "Rango_AM_DB",
    "Rango_de_deuda"
]

# Crear el nuevo DataFrame solo con esas columnas
subset_df = resultado_actualizado[cols].copy()

# Verificamos
print("Filas:", len(subset_df))

Filas: 678640


In [ ]:
# Elimina filas duplicadas conservando la primera aparición
subset_df = subset_df.drop_duplicates().reset_index(drop=True)

In [ ]:
# 1️⃣ Ver si hay referencias duplicadas (True/False)
hay_repetidas = subset_df['Referencia'].duplicated().any()
print("¿Hay referencias repetidas?:", hay_repetidas)

# 2️⃣ Contar cuántas referencias están repetidas
num_repetidas = subset_df['Referencia'].duplicated().sum()
print("Cantidad de referencias repetidas:", num_repetidas)

# 3️⃣ Ver exactamente cuáles se repiten (con su número de apariciones)
repetidas = (
    subset_df['Referencia']
    .value_counts()
    .loc[lambda x: x > 1]
)
print("Referencias que se repiten y cuántas veces:\n", repetidas)

¿Hay referencias repetidas?: True
Cantidad de referencias repetidas: 86
Referencias que se repiten y cuántas veces:
 Referencia
3102023112    2
3008657733    2
3205687759    2
3104138087    2
3133298211    2
             ..
3013200237    2
3135092482    2
3118812017    2
3053259183    2
3002746132    2
Name: count, Length: 86, dtype: int64


In [ ]:
import pandas as pd
import re

# Copia de trabajo
tmp = subset_df.copy()

# Detectar la columna Mes_Año (acepta variantes comunes)
mes_candidates = [c for c in tmp.columns
                  if re.sub(r'[\s_]', '', c).lower() in {'mesaño','mesano'}]
if not mes_candidates:
    raise KeyError("No encontré la columna 'Mes_Año' / 'Mes_Ano'.")
MES_COL = mes_candidates[0]

# Asegurar tipos numéricos (por si vienen como texto con símbolos)
tmp['Apartado Mensual']   = pd.to_numeric(tmp['Apartado Mensual'], errors='coerce')
tmp['Deuda inicial Fija'] = pd.to_numeric(tmp['Deuda inicial Fija'], errors='coerce')

# Agrupar por Mes_Año:
# - sumar Apartado Mensual y Deuda inicial Fija
# - contar referencias únicas
resumen_mes = (
    tmp.groupby(MES_COL, dropna=False)
       .agg({
           'Apartado Mensual': 'sum',
           'Deuda inicial Fija': 'sum',
           'Referencia': pd.Series.nunique
       })
       .rename(columns={'Referencia': 'Referencias_unicas'})
       .reset_index()
)

# Orden opcional de columnas
resumen_mes = resumen_mes[[MES_COL, 'Referencias_unicas', 'Apartado Mensual', 'Deuda inicial Fija']]

resumen_mes.head()

,Mes_Año,Referencias_unicas,Apartado Mensual,Deuda inicial Fija
0,01-2021,1481,7.329912e+08,3.975783e+10
1,01-2022,1379,6.983784e+08,4.070526e+10
2,01-2023,2260,1.263548e+09,7.836527e+10
3,01-2024,1616,9.165936e+08,5.700115e+10
4,01-2025,1644,9.243889e+08,5.380762e+10


In [ ]:
resumen_mes

,Mes_Año,Referencias_unicas,Apartado Mensual,Deuda inicial Fija
0,01-2021,1481,7.329912e+08,3.975783e+10
1,01-2022,1379,6.983784e+08,4.070526e+10
2,01-2023,2260,1.263548e+09,7.836527e+10
3,01-2024,1616,9.165936e+08,5.700115e+10
4,01-2025,1644,9.243889e+08,5.380762e+10
5,02-2021,1500,7.589782e+08,4.279622e+10
6,02-2022,1492,7.403932e+08,4.246839e+10
7,02-2023,2400,1.327191e+09,8.121980e+10
8,02-2024,1706,9.619553e+08,6.028811e+10
9,02-2025,1476,8.193488e+08,4.833615e+10


In [ ]:
subset_df

,Referencia,Mes_Año,Apartado Mensual,Deuda inicial Fija,AM/DB,Rango_AM_DB,Rango_de_deuda
0,1000018033,06-2025,611362.66,43346600.0,0.014104,< 1.5%,35 – 60 M
1,1000046281,04-2024,389316.26,11421445.0,0.034086,> 2.5%,< 35 M
2,1000120635,02-2024,246716.02,11953700.0,0.020639,2% – 2.5%,< 35 M
3,1000136703,06-2024,272569.17,12278900.0,0.022198,2% – 2.5%,< 35 M
4,1000137024,10-2024,498600.59,25457250.0,0.019586,1.5% – 1.99%,< 35 M
...,...,...,...,...,...,...,...
86406,99942,06-2021,529899.00,31496350.0,0.016824,1.5% – 1.99%,< 35 M
86407,99944,06-2021,436679.00,22096800.0,0.019762,1.5% – 1.99%,< 35 M
86408,99961,06-2021,383931.00,17062100.0,0.022502,2% – 2.5%,< 35 M
86409,99969,06-2021,611331.56,37778650.0,0.016182,1.5% – 1.99%,35 – 60 M


In [ ]:
# Asegura que Monto sea numérico y la fecha en datetime
resultado_actualizado["Monto"] = pd.to_numeric(resultado_actualizado["Monto"], errors="coerce")
resultado_actualizado["Fecha de cobro"] = pd.to_datetime(resultado_actualizado["Fecha de cobro"], dayfirst=True, errors="coerce")

# Filtra por año 2025 y tipo de comisión
filtro = (
    (resultado_actualizado["Fecha de cobro"].dt.year == 2025) &
    (resultado_actualizado["Tipo de comision"] == "LIQUIDACION_COLOMBIA")
)

# Suma de Monto
total_monto_2025 = resultado_actualizado.loc[filtro, "Monto"].sum()
print(total_monto_2025)

10869985435.371824


In [ ]:
Columnas_eliminar = ['Fecha de cobro', 'Fecha de Inicio']
resultado_actualizado = resultado_actualizado.drop(columns=Columnas_eliminar)

In [ ]:
resultado_actualizado

,Referencia,Credito,Status facturacion,Tipo de comision,Monto,Fecha de facturacion,Apartado Mensual,Vehiculo de ahorro,Deuda inicial Fija,Mes_Año,Mes_Cobro,AM/DB,AM/DB_outlier,Rango_AM_DB,Rango_de_deuda
0,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,602031.0,2025-06-26,611362.66,SKANDIA,43346600.0,06-2025,0.0,0.014104,0,< 1.5%,35 – 60 M
1,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,627346.0,2025-06-26,611362.66,SKANDIA,43346600.0,06-2025,1.0,0.014104,0,< 1.5%,35 – 60 M
2,1000018033,false,COBRADO,MENSUALIDAD_COLOMBIA,227237.0,2025-08-05,611362.66,SKANDIA,43346600.0,06-2025,2.0,0.014104,0,< 1.5%,35 – 60 M
3,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,3852.0,2025-06-26,611362.66,SKANDIA,43346600.0,06-2025,2.0,0.014104,0,< 1.5%,35 – 60 M
4,1000018033,false,COBRADO,MENSUALIDAD_COLOMBIA,227237.0,2025-09-03,611362.66,SKANDIA,43346600.0,06-2025,3.0,0.014104,0,< 1.5%,35 – 60 M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765853,99990,false,COBRADO,MENSUALIDAD_COLOMBIA,157577.0,2021-12-05,519374.00,A&V,29414564.0,06-2021,6.0,0.017657,0,1.5% – 1.99%,< 35 M
765854,99990,true,COBRADO,LIQUIDACION_COLOMBIA,1210878.0,2021-12-23,519374.00,A&V,29414564.0,06-2021,6.0,0.017657,0,1.5% – 1.99%,< 35 M
765855,99990,true,COBRADO,MENSUALIDAD_COLOMBIA,1575770.0,2021-12-23,519374.00,A&V,29414564.0,06-2021,6.0,0.017657,0,1.5% – 1.99%,< 35 M
765856,99990,true,COBRADO,LIQUIDACION_COLOMBIA,112390.0,2022-07-30,519374.00,A&V,29414564.0,06-2021,13.0,0.017657,0,1.5% – 1.99%,< 35 M


#Traemos berex

In [ ]:
import pandas as pd
import re

# =========================
# Config
# =========================
SPREADSHEET_ID = "13Vf32LzRI2V95dIUqfevzm-ZmsDR3d17UTre_7XJ-UU"
GID_ESTRUCTURADO = 0
GID_TRADICIONAL  = 285942728
FECHA_COLS_EXACTAS = ["payment_date", "Originado"]


# =========================
# Utilidades
# =========================
def load_sheet_csv(spreadsheet_id: str, gid: int) -> pd.DataFrame:
    """Lee una hoja pública de Google Sheets exportándola como CSV."""
    url = f"https://docs.google.com/spreadsheets/d/{spreadsheet_id}/export?format=csv&gid={gid}"
    df = pd.read_csv(url, dtype=str)
    df.columns = [c.strip() for c in df.columns]
    return df


def get_ref_col(df: pd.DataFrame) -> str:
    """Detecta el nombre de la columna 'reference'."""
    candidatos = ['reference', 'Referencia', 'REFERENCIA', 'Reference', 'referencia']
    for c in candidatos:
        if c in df.columns:
            return c
    for c in df.columns:
        if 'refer' in c.lower():
            return c
    raise KeyError("No se encontró la columna de referencia en el DataFrame.")


def normaliza_ref(s: str) -> str:
    """
    Normaliza referencias para comparación:
    - '42871.0' o '00042871.000' -> '42871'
    - '00042871' -> '42871'
    - quita separadores y pone mayúsculas para alfanuméricos.
    """
    if pd.isna(s):
        return None

    raw = str(s).strip()
    up  = raw.upper()

    try:

        fv = float(raw.replace(",", ""))
        if fv.is_integer():
            return str(int(fv))
    except Exception:
        pass

    if raw.isdigit():
        try:
            return str(int(raw))
        except Exception:
            return raw.lstrip("0") or "0"

    t = re.sub(r"[^0-9A-Z]", "", up)
    if t.isdigit():
        try:
            t = str(int(t))
        except Exception:
            pass
    return t or None


def to_datetime_ddmmyyyy(series: pd.Series) -> pd.Series:
    """Convierte una serie a datetime asumiendo formato día/mes/año."""
    return pd.to_datetime(series.astype(str).str.strip(), dayfirst=True, errors="coerce")


# =========================
# Cargar datos
# =========================
df_estructurado = load_sheet_csv(SPREADSHEET_ID, GID_ESTRUCTURADO)
df_tradicional  = load_sheet_csv(SPREADSHEET_ID, GID_TRADICIONAL)

ref_est  = get_ref_col(df_estructurado)
ref_trad = get_ref_col(df_tradicional)

df_estructurado[ref_est] = df_estructurado[ref_est].apply(normaliza_ref)
df_tradicional[ref_trad] = df_tradicional[ref_trad].apply(normaliza_ref)

# =========================
# Diagnóstico ANTES del filtrado
# =========================
refs_validas = set(df_estructurado[ref_est].dropna().unique())
trad_refs   = set(df_tradicional[ref_trad].dropna().unique())
inter = refs_validas & trad_refs

print(f"Refs únicas Estructurado: {len(refs_validas)}")
print(f"Refs únicas Tradicional : {len(trad_refs)}")
print(f"Intersección            : {len(inter)}")

no_match = list(trad_refs - refs_validas)[:10]
if no_match:
    print("Ejemplos Tradicional NO presentes en Estructurado:", no_match)

# =========================
# Conversión de fechas (dd/mm/YYYY) en ambos dataframes
# =========================
for col in FECHA_COLS_EXACTAS:
    if col in df_estructurado.columns:
        df_estructurado[col] = to_datetime_ddmmyyyy(df_estructurado[col])
    if col in df_tradicional.columns:
        df_tradicional[col] = to_datetime_ddmmyyyy(df_tradicional[col])

# =========================
# Marcar origen
# =========================
df_estructurado['Estructurado'] = 1
df_tradicional['Estructurado']  = 0

# =========================
# Filtrado Tradicional (regla de negocio):
# - Mantener TODO lo de Estructurado (gid=0).
# - De Tradicional (gid=285942728), conservar solo filas cuya reference está en Estructurado.
# =========================
df_trad_filtrado = df_tradicional[df_tradicional[ref_trad].isin(refs_validas)].copy()

# =========================
# Alinear nombre de columna de referencia y concatenar
# =========================
if ref_trad != ref_est:
    df_trad_filtrado.rename(columns={ref_trad: ref_est}, inplace=True)

berex_estruct = pd.concat([df_estructurado, df_trad_filtrado], ignore_index=True)

# =========================
# Chequeos finales
# =========================
eliminadas = len(df_tradicional) - len(df_trad_filtrado)
print(f"Filas eliminadas de 'Tradicional con Estructurado': {eliminadas}")
print("Filas finales berex_estruct:", len(berex_estruct))
print("Referencias únicas finales:", berex_estruct[ref_est].nunique())
print("Columnas convertidas a fecha:", [c for c in FECHA_COLS_EXACTAS if c in berex_estruct.columns])

Refs únicas Estructurado: 8127
Refs únicas Tradicional : 3672
Intersección            : 3000
Ejemplos Tradicional NO presentes en Estructurado: ['3118596545', '3177286045', '3207436587', '1126788185', '3118369989', '3006856349', '3136237842', '3144909135', '3102889053', '3166240965']
Filas eliminadas de 'Tradicional con Estructurado': 1900
Filas finales berex_estruct: 62038
Referencias únicas finales: 8127
Columnas convertidas a fecha: ['payment_date', 'Originado']


In [ ]:
berex_estruct

,reference,id,amount,bank,debt_id,destination,payment_date,payment_number,requester,Originado,Unnamed: 10,Unnamed: 11,Unnamed: 12,Estructurado
0,3183831101,3295,350824,Alkosto,544721,commission,2021-01-12,2,Natalia Valentina Castro Jimenez,2021-01-12,318383110144208,NaN,294810.084,1
1,3183831101,3294,500000,Alkosto,544721,bank,2021-01-12,1,Natalia Valentina Castro Jimenez,2021-01-12,318383110144208,NaN,420168.0672,1
2,3183831101,3296,436000,Alkosto,544721,bank,2021-02-19,3,Natalia Valentina Castro Jimenez,2021-01-12,318383110144208,NaN,366386.5546,1
3,3183831101,3297,436000,Alkosto,544721,bank,2021-03-19,4,Natalia Valentina Castro Jimenez,2021-01-12,318383110144208,NaN,366386.5546,1
4,3504389669,3941,131198,Alkosto,510212,commission,2021-01-30,2,No Asignado,2021-01-30,350438966944226,NaN,110250.4202,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62033,3132820285,NaN,1045113,Davivienda,1097209,commission,2025-10-03,1,Diego Alejandro Sanchez Fonseca,2025-10-03,NaN,NaN,NaN,0
62034,79377506,NaN,62609,Falabella,1605967,commission,2025-10-03,1,Juan Sebastian Lopez Martinez,2025-10-03,NaN,NaN,NaN,0
62035,3102587912,NaN,514361,Davivienda,1422192,commission,2025-10-06,1,Nina Mayerly Gomez Guerrero,2025-10-06,NaN,NaN,NaN,0
62036,3102587912,NaN,25704,Davivienda,1422190,commission,2025-10-07,1,Nina Mayerly Gomez Guerrero,2025-10-07,NaN,NaN,NaN,0


In [ ]:
referencia_buscada = '3002426421'

resultado = berex_estruct[berex_estruct['reference'] == referencia_buscada]

resultado

,reference,id,amount,bank,debt_id,destination,payment_date,payment_number,requester,Originado,Unnamed: 10,Unnamed: 11,Unnamed: 12,Estructurado
26700,3002426421,151997,202000,CREDIVALORES,907614,bank,2024-06-11,1,Nicolle Saray Catica Trujillo,2024-06-11,300242642145454,NaN,169747.8992,1
26701,3002426421,151998,500000,CREDIVALORES,907614,commission,2024-06-11,2,Nicolle Saray Catica Trujillo,2024-06-11,300242642145454,NaN,420168.0672,1
26702,3002426421,151999,2002000,CREDIVALORES,907614,bank,2024-06-26,3,Nicolle Saray Catica Trujillo,2024-06-11,300242642145454,NaN,1682352.941,1
26703,3002426421,152000,255000,CREDIVALORES,907614,commission,2024-08-05,4,Nicolle Saray Catica Trujillo,2024-06-11,300242642145454,NaN,214285.7143,1
26704,3002426421,152001,254310,CREDIVALORES,907614,commission,2024-09-05,5,Nicolle Saray Catica Trujillo,2024-06-11,300242642145454,NaN,213705.8824,1
54368,3002426421,NaN,2610000,Serfinanza,907612,bank,2024-04-27,1,Angela Yara Mayorga,2024-04-27,NaN,NaN,NaN,0
60132,3002426421,NaN,97140,Serfinanza,907612,commission,2024-04-27,2,Angela Yara Mayorga,2024-04-27,NaN,NaN,NaN,0


In [ ]:
berex_estruct.info()
resultado_actualizado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62038 entries, 0 to 62037
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   reference       62038 non-null  object        
 1   id              50296 non-null  object        
 2   amount          61384 non-null  object        
 3   bank            62038 non-null  object        
 4   debt_id         62038 non-null  object        
 5   destination     62038 non-null  object        
 6   payment_date    62004 non-null  datetime64[ns]
 7   payment_number  62038 non-null  object        
 8   requester       62038 non-null  object        
 9   Originado       62004 non-null  datetime64[ns]
 10  Unnamed: 10     50302 non-null  object        
 11  Unnamed: 11     0 non-null      object        
 12  Unnamed: 12     50302 non-null  object        
 13  Estructurado    62038 non-null  int64         
dtypes: datetime64[ns](2), int64(1), object(11)
memory usag

In [ ]:
# 1) Obtener las referencias únicas de berex_estruct (columna 'reference')
refs_berex = (
    berex_estruct['reference']
    .dropna()                # por si hay NaN
    .astype(str)             # homogeneizar tipo
    .str.strip()             # quitar espacios
    .unique()
)

# (opcional) si quieres mejor rendimiento para el filtro:
refs_berex_set = set(refs_berex)

# 2) Filtrar resultado_actualizado dejando solo filas cuya 'Referencia' esté en las únicas de berex
resultado_filtrado = resultado_actualizado[
    resultado_actualizado['Referencia'].astype(str).str.strip().isin(refs_berex_set)
].copy()

# 3) (opcional) verificación rápida
print(f"Refs únicas en berex_estruct: {len(refs_berex_set):,}")
print(f"Filas resultantes en resultado_filtrado: {len(resultado_filtrado):,}")
print(resultado_filtrado['Referencia'].nunique(), "referencias únicas en el filtrado")

Refs únicas en berex_estruct: 8,127
Filas resultantes en resultado_filtrado: 132,496
7758 referencias únicas en el filtrado


In [ ]:
resultado_filtrado

,Referencia,Credito,Status facturacion,Tipo de comision,Monto,Fecha de facturacion,Apartado Mensual,Vehiculo de ahorro,Deuda inicial Fija,Mes_Año,Mes_Cobro,AM/DB,AM/DB_outlier,Rango_AM_DB,Rango_de_deuda
72,1000185400,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,523175.000000,2025-03-04,520791.11,SKANDIA,22718600.0,02-2025,1.0,0.022924,0,2% – 2.5%,< 35 M
73,1000185400,false,COBRADO,MENSUALIDAD_COLOMBIA,124097.000000,2025-04-04,520791.11,SKANDIA,22718600.0,02-2025,2.0,0.022924,0,2% – 2.5%,< 35 M
74,1000185400,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,528911.000000,2025-03-04,520791.11,SKANDIA,22718600.0,02-2025,2.0,0.022924,0,2% – 2.5%,< 35 M
75,1000185400,false,COBRADO,MENSUALIDAD_COLOMBIA,124097.000000,2025-05-06,520791.11,SKANDIA,22718600.0,02-2025,3.0,0.022924,0,2% – 2.5%,< 35 M
76,1000185400,false,COBRADO,LIQUIDACION_COLOMBIA,19405.915966,2025-06-20,520791.11,SKANDIA,22718600.0,02-2025,4.0,0.022924,0,2% – 2.5%,< 35 M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765712,9957452,false,COBRADO,MENSUALIDAD_COLOMBIA,74016.000000,2025-03-06,465252.24,SKANDIA,14306600.0,07-2023,20.0,0.032520,0,> 2.5%,< 35 M
765713,9957452,false,COBRADO,MENSUALIDAD_COLOMBIA,74016.000000,2025-04-04,465252.24,SKANDIA,14306600.0,07-2023,21.0,0.032520,0,> 2.5%,< 35 M
765714,9957452,false,COBRADO,MENSUALIDAD_COLOMBIA,74016.000000,2025-05-06,465252.24,SKANDIA,14306600.0,07-2023,22.0,0.032520,0,> 2.5%,< 35 M
765715,9957452,false,COBRADO,MENSUALIDAD_COLOMBIA,10504.000000,2025-06-05,465252.24,SKANDIA,14306600.0,07-2023,23.0,0.032520,0,> 2.5%,< 35 M


In [ ]:
# Normalizamos la columna para evitar problemas de mayúsculas/espacios/NaN
tipo_norm = (
    resultado_filtrado['Tipo de comision']
    .fillna('')
    .astype(str)
    .str.strip()
    .str.upper()
)

# 1) Todo lo que diga LIQUIDACION_COLOMBIA
resultado_liquidacion = resultado_filtrado.loc[
    tipo_norm.eq('LIQUIDACION_COLOMBIA')
].copy()

# 2) Todo lo que diga MENSUALIDAD_COLOMBIA
resultado_mensualidad= resultado_filtrado.loc[
    tipo_norm.eq('MENSUALIDAD_COLOMBIA')
].copy()

# (Opcional) chequeos rápidos
print("LIQUIDACION_COLOMBIA:", len(resultado_liquidacion))
print("MENSUALIDAD_COLOMBIA:", len(resultado_mensualidad))


LIQUIDACION_COLOMBIA: 29378
MENSUALIDAD_COLOMBIA: 88077


In [ ]:
import numpy as np
import pandas as pd

# 0) Copia de trabajo y filtro por destination == 'commission'
be = berex_estruct.copy()

dest_norm = be['destination'].fillna('').astype(str).str.strip().str.lower()
be = be.loc[dest_norm.eq('commission')].copy()

# 1) Nos quedamos con las columnas relevantes
cols = ['reference', 'Originado', 'requester', 'bank', 'amount']
be = be[cols].copy()

# 2) Normalización básica
be['reference'] = be['reference'].astype(str).str.strip()
be['requester'] = be['requester'].astype(str).str.strip()
be['bank'] = be['bank'].astype(str).str.strip()

# 3) Limpiar 'amount' (viene como object) -> numérico
#    Quita símbolos comunes, puntos de miles y deja el decimal
be['amount'] = (
    be['amount']
    .astype(str)
    .str.replace(r'[^\d,.\-]', '', regex=True)     # remueve símbolos no numéricos
    .str.replace(r'(?<=\d)\.(?=\d{3}(\D|$))', '', regex=True)  # quita puntos de miles
    .str.replace(',', '.', regex=False)            # coma -> punto decimal
)
be['amount_num'] = pd.to_numeric(be['amount'], errors='coerce')

# 4) (Opcional) Si una fila no tiene 'Originado', no se puede agrupar por fecha -> la descartamos
be = be[~be['Originado'].isna()].copy()

# 5) Función para tomar el "responsable" y "bank" por grupo:
#    - Primero el modo (valor más frecuente)
#    - Si hay empate o no hay modo claro, toma el primer no nulo en el orden original
def pick_label(s: pd.Series):
    s_nonnull = s.dropna()
    if s_nonnull.empty:
        return np.nan
    modes = s_nonnull.mode()
    if not modes.empty:
        return modes.iloc[0]
    return s_nonnull.iloc[0]

# 6) Agregación por (reference, Originado)
df_referencia_fecha = (
    be.groupby(['reference', 'Originado'], as_index=False)
      .agg(
          requester=('requester', pick_label),
          bank=('bank', pick_label),
          amount_promedio=('amount_num', 'mean'),
          amount_total    = ('amount_num', 'sum'),
          n_pagos=('reference', 'size')
      )
)

# 7) Orden opcional
df_referencia_fecha = df_referencia_fecha.sort_values(['reference', 'Originado']).reset_index(drop=True)

# 8) (Opcional) Redondeo del promedio
df_referencia_fecha['amount_promedio'] = df_referencia_fecha['amount_promedio'].round(2)
df_referencia_fecha['amount_total'] = df_referencia_fecha['amount_total'].round(2)

# Chequeo rápido
print(df_referencia_fecha.head())
print(f"Total filas (reference x Originado): {len(df_referencia_fecha):,}")

    reference  Originado                          requester        bank  \
0  1000185400 2025-06-19           Hector Elian Lacera Vega  Rapicredit   
1  1000591190 2025-09-11          Steven Mateo Aroca Garzon   Falabella   
2      100076 2022-12-23          David Steban Pineda Gallo  Davivienda   
3      100076 2023-01-11                        No Asignado  Davivienda   
4  1001082965 2024-11-18  Daniel Alejandro Umbarila Bolaños   Compensar   

   amount_promedio  amount_total  n_pagos  
0          53546.5      107093.0        2  
1          76543.5      153087.0        2  
2         208809.0      208809.0        1  
3         243659.0      487318.0        2  
4         410419.0      820838.0        2  
Total filas (reference x Originado): 13,596


In [ ]:
df_referencia_fecha

,reference,Originado,requester,bank,amount_promedio,amount_total,n_pagos
0,1000185400,2025-06-19,Hector Elian Lacera Vega,Rapicredit,53546.50,107093.0,2
1,1000591190,2025-09-11,Steven Mateo Aroca Garzon,Falabella,76543.50,153087.0,2
2,100076,2022-12-23,David Steban Pineda Gallo,Davivienda,208809.00,208809.0,1
3,100076,2023-01-11,No Asignado,Davivienda,243659.00,487318.0,2
4,1001082965,2024-11-18,Daniel Alejandro Umbarila Bolaños,Compensar,410419.00,820838.0,2
...,...,...,...,...,...,...,...
13591,98702966,2025-01-30,Jana Milena Lopez Buitrago,Bancolombia,1140268.44,18244295.0,16
13592,98709164,2025-08-29,Vivian Caterin Rodriguez Verano,Bancolombia,332080.50,1328322.0,4
13593,99534,2022-02-14,Maria Paula Balaguera Penagos,Scotiabank Colpatria,390861.00,390861.0,1
13594,99534,2022-08-10,Natalia Valentina Castro Jimenez,Colsubsidio,145242.33,435727.0,3


In [ ]:
be_pab = berex_estruct.copy()
dest_norm2 = be_pab['destination'].fillna('').astype(str).str.strip().str.lower()
be_pab = be_pab.loc[dest_norm2.eq('bank')].copy()

be_pab = be_pab[['reference', 'Originado', 'amount']].copy()
be_pab['reference'] = be_pab['reference'].astype(str).str.strip()

be_pab['amount'] = (
    be_pab['amount'].astype(str)
    .str.replace(r'[^\d,.\-]', '', regex=True)
    .str.replace(r'(?<=\d)\.(?=\d{3}(\D|$))', '', regex=True)
    .str.replace(',', '.', regex=False)
)
be_pab['amount_num'] = pd.to_numeric(be_pab['amount'], errors='coerce')
be_pab = be_pab[~be_pab['Originado'].isna()].copy()

df_pab = (
    be_pab.groupby(['reference', 'Originado'], as_index=False)
    .agg(
        PaB_total=('amount_num', 'sum'),
        n_pagos_PaB=('reference', 'size')
    )
)

df_pab['PaB_total'] = df_pab['PaB_total'].round(2)

# ========= Merge a df_referencia_fecha =========
df_referencia_fecha_pab = df_referencia_fecha.merge(
    df_pab, on=['reference', 'Originado'], how='left'
)

# Si no hay registros 'bank' para ese (reference, Originado) -> 0
df_referencia_fecha_pab['PaB_total'] = df_referencia_fecha_pab['PaB_total'].fillna(0).round(2)
df_referencia_fecha_pab['n_pagos_PaB'] = df_referencia_fecha_pab['n_pagos_PaB'].fillna(0).astype(int)

In [ ]:
df_referencia_fecha_pab

,reference,Originado,requester,bank,amount_promedio,amount_total,n_pagos,PaB_total,n_pagos_PaB
0,1000185400,2025-06-19,Hector Elian Lacera Vega,Rapicredit,53546.50,107093.0,2,552839.0,1
1,1000591190,2025-09-11,Steven Mateo Aroca Garzon,Falabella,76543.50,153087.0,2,1930000.0,1
2,100076,2022-12-23,David Steban Pineda Gallo,Davivienda,208809.00,208809.0,1,2942000.0,1
3,100076,2023-01-11,No Asignado,Davivienda,243659.00,487318.0,2,10200000.0,1
4,1001082965,2024-11-18,Daniel Alejandro Umbarila Bolaños,Compensar,410419.00,820838.0,2,1390000.0,1
...,...,...,...,...,...,...,...,...,...
13591,98702966,2025-01-30,Jana Milena Lopez Buitrago,Bancolombia,1140268.44,18244295.0,16,8954000.0,3
13592,98709164,2025-08-29,Vivian Caterin Rodriguez Verano,Bancolombia,332080.50,1328322.0,4,1800000.0,1
13593,99534,2022-02-14,Maria Paula Balaguera Penagos,Scotiabank Colpatria,390861.00,390861.0,1,1800000.0,1
13594,99534,2022-08-10,Natalia Valentina Castro Jimenez,Colsubsidio,145242.33,435727.0,3,2600000.0,1


In [ ]:
resultado_liquidacion

,Referencia,Credito,Status facturacion,Tipo de comision,Monto,Fecha de facturacion,Apartado Mensual,Vehiculo de ahorro,Deuda inicial Fija,Mes_Año,Mes_Cobro,AM/DB,AM/DB_outlier,Rango_AM_DB,Rango_de_deuda
76,1000185400,false,COBRADO,LIQUIDACION_COLOMBIA,19405.915966,2025-06-20,520791.11,SKANDIA,22718600.0,02-2025,4.0,0.022924,0,2% – 2.5%,< 35 M
78,1000185400,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,15060.000000,2025-06-19,520791.11,SKANDIA,22718600.0,02-2025,4.0,0.022924,0,2% – 2.5%,< 35 M
80,1000185400,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,55528.000000,2025-06-19,520791.11,SKANDIA,22718600.0,02-2025,5.0,0.022924,0,2% – 2.5%,< 35 M
156,1000591190,false,COBRADO,LIQUIDACION_COLOMBIA,342925.000000,2025-04-11,344986.00,SKANDIA,12465987.0,12-2024,4.0,0.027674,0,> 2.5%,< 35 M
162,1000591190,false,COBRADO,LIQUIDACION_COLOMBIA,128644.546218,2025-09-12,344986.00,SKANDIA,12465987.0,12-2024,9.0,0.027674,0,> 2.5%,< 35 M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765680,99534,false,COBRADO,LIQUIDACION_COLOMBIA,328455.000000,2022-02-14,287861.00,A&V,9030750.0,06-2021,8.0,0.031876,0,> 2.5%,< 35 M
765687,99534,false,COBRADO,LIQUIDACION_COLOMBIA,73231.000000,2022-08-10,287861.00,A&V,9030750.0,06-2021,14.0,0.031876,0,> 2.5%,< 35 M
765689,99534,false,COBRADO,LIQUIDACION_COLOMBIA,146463.000000,2022-09-12,287861.00,A&V,9030750.0,06-2021,15.0,0.031876,0,> 2.5%,< 35 M
765691,99534,false,COBRADO,LIQUIDACION_COLOMBIA,146464.000000,2022-09-28,287861.00,A&V,9030750.0,06-2021,16.0,0.031876,0,> 2.5%,< 35 M


In [ ]:
import pandas as pd
import numpy as np

df = resultado_liquidacion.copy()

# 1) Aseguramos formatos
mes_ano_str = df['Mes_Año'].astype(str).str.strip()

# Si Mes_Cobro viene como 4.0, 5.0, etc. -> entero; NaN -> 0 (puedes cambiar este comportamiento si lo prefieres)
offset_meses = (
    pd.to_numeric(df['Mes_Cobro'], errors='coerce')
      .fillna(0)
      .astype(int)
)

# 2) Convertimos Mes_Año a periodo mensual y sumamos el offset en meses
base_period = pd.to_datetime(mes_ano_str, format='%m-%Y', errors='coerce').dt.to_period('M')

# 3) mes_exacto = primer día del mes (start of month) + offset de meses
df['mes_exacto'] = (base_period + offset_meses).dt.to_timestamp(how='start')

# (Opcional) si deseas asegurarte que sea exactamente día 1 sin hora
df['mes_exacto'] = df['mes_exacto'].dt.normalize()

# Chequeo rápido
print(df[['Mes_Año', 'Mes_Cobro', 'mes_exacto']].head())


     Mes_Año  Mes_Cobro mes_exacto
76   02-2025        4.0 2025-06-01
78   02-2025        4.0 2025-06-01
80   02-2025        5.0 2025-07-01
156  12-2024        4.0 2025-04-01
162  12-2024        9.0 2025-09-01


In [ ]:
df

,Referencia,Credito,Status facturacion,Tipo de comision,Monto,Fecha de facturacion,Apartado Mensual,Vehiculo de ahorro,Deuda inicial Fija,Mes_Año,Mes_Cobro,AM/DB,AM/DB_outlier,Rango_AM_DB,Rango_de_deuda,mes_exacto
76,1000185400,false,COBRADO,LIQUIDACION_COLOMBIA,19405.915966,2025-06-20,520791.11,SKANDIA,22718600.0,02-2025,4.0,0.022924,0,2% – 2.5%,< 35 M,2025-06-01
78,1000185400,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,15060.000000,2025-06-19,520791.11,SKANDIA,22718600.0,02-2025,4.0,0.022924,0,2% – 2.5%,< 35 M,2025-06-01
80,1000185400,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,55528.000000,2025-06-19,520791.11,SKANDIA,22718600.0,02-2025,5.0,0.022924,0,2% – 2.5%,< 35 M,2025-07-01
156,1000591190,false,COBRADO,LIQUIDACION_COLOMBIA,342925.000000,2025-04-11,344986.00,SKANDIA,12465987.0,12-2024,4.0,0.027674,0,> 2.5%,< 35 M,2025-04-01
162,1000591190,false,COBRADO,LIQUIDACION_COLOMBIA,128644.546218,2025-09-12,344986.00,SKANDIA,12465987.0,12-2024,9.0,0.027674,0,> 2.5%,< 35 M,2025-09-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765680,99534,false,COBRADO,LIQUIDACION_COLOMBIA,328455.000000,2022-02-14,287861.00,A&V,9030750.0,06-2021,8.0,0.031876,0,> 2.5%,< 35 M,2022-02-01
765687,99534,false,COBRADO,LIQUIDACION_COLOMBIA,73231.000000,2022-08-10,287861.00,A&V,9030750.0,06-2021,14.0,0.031876,0,> 2.5%,< 35 M,2022-08-01
765689,99534,false,COBRADO,LIQUIDACION_COLOMBIA,146463.000000,2022-09-12,287861.00,A&V,9030750.0,06-2021,15.0,0.031876,0,> 2.5%,< 35 M,2022-09-01
765691,99534,false,COBRADO,LIQUIDACION_COLOMBIA,146464.000000,2022-09-28,287861.00,A&V,9030750.0,06-2021,16.0,0.031876,0,> 2.5%,< 35 M,2022-10-01


In [ ]:
import pandas as pd
import numpy as np

# 1) Normalizamos llaves para empatar referencias
df = df.copy()
df['__ref_norm'] = df['Referencia'].astype(str).str.strip()

tmp = df_referencia_fecha.copy()
tmp['__ref_norm'] = tmp['reference'].astype(str).str.strip()

# (Recomendado) Asegurar tipos de fecha
df['mes_exacto'] = pd.to_datetime(df['mes_exacto'], errors='coerce')
tmp['Originado']  = pd.to_datetime(tmp['Originado'], errors='coerce')

# 2) Para cada reference, tomar la fecha Originado más antigua
cutoff_por_ref = (
    tmp.loc[~tmp['Originado'].isna(), ['__ref_norm', 'Originado']]
       .groupby('__ref_norm', as_index=True)['Originado']
       .min()
)

# 3) Mapear fecha de corte a df
df['__fecha_corte'] = df['__ref_norm'].map(cutoff_por_ref)

# 4) Filtrar:
#   - Solo referencias que existen en df_referencia_fecha (tienen __fecha_corte no nula)
#   - Comparar por MES-AÑO: mes_exacto_M >= fecha_corte_M  (mismo mes se conserva)
mask_valid = df['__fecha_corte'].notna()

mes_period   = df['mes_exacto'].dt.to_period('M')
corte_period = pd.to_datetime(df['__fecha_corte']).dt.to_period('M')

mask_fecha = mes_period.notna() & corte_period.notna() & (mes_period >= corte_period)

df_filtrado = df.loc[mask_valid & mask_fecha].copy()

# 5) Limpieza columnas auxiliares
df_filtrado.drop(columns=['__ref_norm', '__fecha_corte'], inplace=True)

# (Opcional) Chequeo
print("Filas antes:", len(df))
print("Filas después:", len(df_filtrado))
print("Referencias únicas conservadas:", df_filtrado['Referencia'].nunique())

Filas antes: 29378
Filas después: 29293
Referencias únicas conservadas: 7675


#Merge

In [ ]:
df_filtrado

,Referencia,Credito,Status facturacion,Tipo de comision,Monto,Fecha de facturacion,Apartado Mensual,Vehiculo de ahorro,Deuda inicial Fija,Mes_Año,Mes_Cobro,AM/DB,AM/DB_outlier,Rango_AM_DB,Rango_de_deuda,mes_exacto
76,1000185400,false,COBRADO,LIQUIDACION_COLOMBIA,19405.915966,2025-06-20,520791.11,SKANDIA,22718600.0,02-2025,4.0,0.022924,0,2% – 2.5%,< 35 M,2025-06-01
78,1000185400,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,15060.000000,2025-06-19,520791.11,SKANDIA,22718600.0,02-2025,4.0,0.022924,0,2% – 2.5%,< 35 M,2025-06-01
80,1000185400,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,55528.000000,2025-06-19,520791.11,SKANDIA,22718600.0,02-2025,5.0,0.022924,0,2% – 2.5%,< 35 M,2025-07-01
162,1000591190,false,COBRADO,LIQUIDACION_COLOMBIA,128644.546218,2025-09-12,344986.00,SKANDIA,12465987.0,12-2024,9.0,0.027674,0,> 2.5%,< 35 M,2025-09-01
164,1000591190,true,COBRADO,LIQUIDACION_COLOMBIA,214365.000000,2025-09-30,344986.00,SKANDIA,12465987.0,12-2024,10.0,0.027674,0,> 2.5%,< 35 M,2025-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765680,99534,false,COBRADO,LIQUIDACION_COLOMBIA,328455.000000,2022-02-14,287861.00,A&V,9030750.0,06-2021,8.0,0.031876,0,> 2.5%,< 35 M,2022-02-01
765687,99534,false,COBRADO,LIQUIDACION_COLOMBIA,73231.000000,2022-08-10,287861.00,A&V,9030750.0,06-2021,14.0,0.031876,0,> 2.5%,< 35 M,2022-08-01
765689,99534,false,COBRADO,LIQUIDACION_COLOMBIA,146463.000000,2022-09-12,287861.00,A&V,9030750.0,06-2021,15.0,0.031876,0,> 2.5%,< 35 M,2022-09-01
765691,99534,false,COBRADO,LIQUIDACION_COLOMBIA,146464.000000,2022-09-28,287861.00,A&V,9030750.0,06-2021,16.0,0.031876,0,> 2.5%,< 35 M,2022-10-01


In [ ]:
df_referencia_fecha_pab

,reference,Originado,requester,bank,amount_promedio,amount_total,n_pagos,PaB_total,n_pagos_PaB
0,1000185400,2025-06-19,Hector Elian Lacera Vega,Rapicredit,53546.50,107093.0,2,552839.0,1
1,1000591190,2025-09-11,Steven Mateo Aroca Garzon,Falabella,76543.50,153087.0,2,1930000.0,1
2,100076,2022-12-23,David Steban Pineda Gallo,Davivienda,208809.00,208809.0,1,2942000.0,1
3,100076,2023-01-11,No Asignado,Davivienda,243659.00,487318.0,2,10200000.0,1
4,1001082965,2024-11-18,Daniel Alejandro Umbarila Bolaños,Compensar,410419.00,820838.0,2,1390000.0,1
...,...,...,...,...,...,...,...,...,...
13591,98702966,2025-01-30,Jana Milena Lopez Buitrago,Bancolombia,1140268.44,18244295.0,16,8954000.0,3
13592,98709164,2025-08-29,Vivian Caterin Rodriguez Verano,Bancolombia,332080.50,1328322.0,4,1800000.0,1
13593,99534,2022-02-14,Maria Paula Balaguera Penagos,Scotiabank Colpatria,390861.00,390861.0,1,1800000.0,1
13594,99534,2022-08-10,Natalia Valentina Castro Jimenez,Colsubsidio,145242.33,435727.0,3,2600000.0,1


In [ ]:
df_filtrado.info()
df_referencia_fecha_pab.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29293 entries, 76 to 765704
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Referencia            29293 non-null  object        
 1   Credito               29293 non-null  object        
 2   Status facturacion    29293 non-null  object        
 3   Tipo de comision      29293 non-null  object        
 4   Monto                 29293 non-null  float64       
 5   Fecha de facturacion  29293 non-null  datetime64[ns]
 6   Apartado Mensual      29293 non-null  float64       
 7   Vehiculo de ahorro    29293 non-null  object        
 8   Deuda inicial Fija    29293 non-null  float64       
 9   Mes_Año               29293 non-null  object        
 10  Mes_Cobro             29293 non-null  float64       
 11  AM/DB                 29293 non-null  float64       
 12  AM/DB_outlier         29293 non-null  int64         
 13  Rango_AM_DB        

In [ ]:
print("df_filtrado cols:", list(df_filtrado.columns))
print("df_referencia_fecha_pab cols:", list(df_referencia_fecha_pab.columns))

df_filtrado cols: ['Referencia', 'Credito', 'Status facturacion', 'Tipo de comision', 'Monto', 'Fecha de facturacion', 'Apartado Mensual', 'Vehiculo de ahorro', 'Deuda inicial Fija', 'Mes_Año', 'Mes_Cobro', 'AM/DB', 'AM/DB_outlier', 'Rango_AM_DB', 'Rango_de_deuda', 'mes_exacto']
df_referencia_fecha_pab cols: ['reference', 'Originado', 'requester', 'bank', 'amount_promedio', 'amount_total', 'n_pagos', 'PaB_total', 'n_pagos_PaB']


In [ ]:
import pandas as pd
import numpy as np

def agregar_originado_por_mes_anio(
    df_filtrado: pd.DataFrame,
    df_referencia_fecha_pab: pd.DataFrame,
    *,
    # Nombres de columnas (ajústalos si difieren)
    col_ref_L: str = "Referencia",       # en df_filtrado
    col_mes_L: str = "mes_exacto",       # en df_filtrado
    col_ref_R: str = "reference",        # en df_referencia_fecha_pab
    col_origin_R: str = "Originado"      # en df_referencia_fecha_pab
) -> pd.DataFrame:
    """
    Agrega la columna 'Originado' a df_filtrado con estas reglas:
      - Si un reference tiene SOLO UNA fecha de Originado -> se copia a todas sus filas.
      - Si tiene VARIAS -> para cada mes_exacto se toma la última Originado cuyo MES/AÑO <= MES/AÑO de mes_exacto.
      - Si no hay Originado <= mes_exacto, queda NaT.

    La comparación se hace por MES/AÑO (primer día del mes), por lo que el "mismo mes" cuenta como válido.

    Parámetros:
      df_filtrado: DataFrame con columnas [col_ref_L, col_mes_L]
      df_referencia_fecha_pab: DataFrame con columnas [col_ref_R, col_origin_R]
      col_ref_L, col_mes_L, col_ref_R, col_origin_R: nombres de columnas.
    """
    # Copias de trabajo
    dfL = df_filtrado.copy()
    dfR = df_referencia_fecha_pab.copy()

    # Normaliza nombre de referencia en dfR para unir con dfL
    if col_ref_R != col_ref_L:
        dfR = dfR.rename(columns={col_ref_R: col_ref_L})

    # Asegurar tipos de fecha
    dfL[col_mes_L] = pd.to_datetime(dfL[col_mes_L], errors="coerce")
    dfR[col_origin_R] = pd.to_datetime(dfR[col_origin_R], errors="coerce")

    # Claves por MES/AÑO (primer día de mes) para que "mismo mes" cuente como válido
    dfL["_mes_floor"] = dfL[col_mes_L].dt.to_period("M").dt.to_timestamp()
    dfR["_originado_floor"] = dfR[col_origin_R].dt.to_period("M").dt.to_timestamp()

    # Filtrar dfR a filas válidas (Referencia y fecha válidas)
    dfR_valid = dfR.dropna(subset=[col_ref_L, "_originado_floor"]).copy()

    # Precalcular por referencia: arrays ORDENADOS de fechas originadas y sus valores
    ref_to_origin_keys = {}
    ref_to_origin_vals = {}

    for ref, sub in dfR_valid.groupby(col_ref_L, sort=False):
        sub_sorted = sub.sort_values("_originado_floor")
        ref_to_origin_keys[ref] = sub_sorted["_originado_floor"].to_numpy(dtype="datetime64[ns]")
        ref_to_origin_vals[ref] = sub_sorted[col_origin_R].to_numpy(dtype="datetime64[ns]")

    # Resolver para cada referencia del dfL (sin merge_asof)
    def resolver_para_referencia(grp: pd.DataFrame) -> pd.Series:
        ref = grp.name
        idx = grp.index

        # Si esta referencia no tiene originados en dfR -> todo NaT
        if ref not in ref_to_origin_keys:
            return pd.Series(pd.NaT, index=idx, dtype="datetime64[ns]")

        keys = ref_to_origin_keys[ref]
        vals = ref_to_origin_vals[ref]

        # Si solo hay una fecha -> copiarla completa
        if len(keys) == 1:
            unica = pd.to_datetime(vals[0])
            return pd.Series(unica, index=idx)

        # Múltiples fechas: usar searchsorted con la clave de mes/año
        left_keys = grp["_mes_floor"].to_numpy(dtype="datetime64[ns]")
        out = np.full(shape=len(grp), fill_value=np.datetime64("NaT"), dtype="datetime64[ns]")

        # Para filas con fecha válida:
        mask_ok = ~pd.isna(grp["_mes_floor"])
        if mask_ok.any():
            # Posición del último originado_floor <= _mes_floor
            pos = np.searchsorted(keys, left_keys[mask_ok], side="right") - 1
            good = pos >= 0
            tmp = np.full(mask_ok.sum(), np.datetime64("NaT"), dtype="datetime64[ns]")
            tmp[good] = vals[pos[good]]
            out[mask_ok.to_numpy()] = tmp

        return pd.Series(pd.to_datetime(out), index=idx)

    # Aplicar por referencia y reinsertar en dfL preservando el orden
    resultado_series = (
        dfL.groupby(col_ref_L, sort=False, group_keys=False)
           .apply(resolver_para_referencia)  # no incluye la columna de grupo en grp
           .reindex(dfL.index)
    )

    # Agregar la columna 'Originado' calculada
    dfL["Originado"] = resultado_series.values

    # Limpiar columnas auxiliares
    dfL.drop(columns=["_mes_floor"], inplace=True, errors="ignore")

    return dfL

    # 2) Si ya tienes ambos DataFrames en memoria, simplemente:
df_resultado = agregar_originado_por_mes_anio(df_filtrado, df_referencia_fecha_pab)


/tmp/ipython-input-879417336.py:91: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(resolver_para_referencia)  # no incluye la columna de grupo en grp


In [ ]:
if 'Fecha_originacion' in df_resultado.columns:
    df_resultado.drop(columns=['Fecha_originacion'], inplace=True)

In [ ]:
df_resultado

,Referencia,Credito,Status facturacion,Tipo de comision,Monto,Fecha de facturacion,Apartado Mensual,Vehiculo de ahorro,Deuda inicial Fija,Mes_Año,Mes_Cobro,AM/DB,AM/DB_outlier,Rango_AM_DB,Rango_de_deuda,mes_exacto,Originado
76,1000185400,false,COBRADO,LIQUIDACION_COLOMBIA,19405.915966,2025-06-20,520791.11,SKANDIA,22718600.0,02-2025,4.0,0.022924,0,2% – 2.5%,< 35 M,2025-06-01,2025-06-19
78,1000185400,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,15060.000000,2025-06-19,520791.11,SKANDIA,22718600.0,02-2025,4.0,0.022924,0,2% – 2.5%,< 35 M,2025-06-01,2025-06-19
80,1000185400,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,55528.000000,2025-06-19,520791.11,SKANDIA,22718600.0,02-2025,5.0,0.022924,0,2% – 2.5%,< 35 M,2025-07-01,2025-06-19
162,1000591190,false,COBRADO,LIQUIDACION_COLOMBIA,128644.546218,2025-09-12,344986.00,SKANDIA,12465987.0,12-2024,9.0,0.027674,0,> 2.5%,< 35 M,2025-09-01,2025-09-11
164,1000591190,true,COBRADO,LIQUIDACION_COLOMBIA,214365.000000,2025-09-30,344986.00,SKANDIA,12465987.0,12-2024,10.0,0.027674,0,> 2.5%,< 35 M,2025-10-01,2025-09-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765680,99534,false,COBRADO,LIQUIDACION_COLOMBIA,328455.000000,2022-02-14,287861.00,A&V,9030750.0,06-2021,8.0,0.031876,0,> 2.5%,< 35 M,2022-02-01,2022-02-14
765687,99534,false,COBRADO,LIQUIDACION_COLOMBIA,73231.000000,2022-08-10,287861.00,A&V,9030750.0,06-2021,14.0,0.031876,0,> 2.5%,< 35 M,2022-08-01,2022-08-10
765689,99534,false,COBRADO,LIQUIDACION_COLOMBIA,146463.000000,2022-09-12,287861.00,A&V,9030750.0,06-2021,15.0,0.031876,0,> 2.5%,< 35 M,2022-09-01,2022-08-10
765691,99534,false,COBRADO,LIQUIDACION_COLOMBIA,146464.000000,2022-09-28,287861.00,A&V,9030750.0,06-2021,16.0,0.031876,0,> 2.5%,< 35 M,2022-10-01,2022-08-10


In [ ]:
# Asegurar que las columnas de fecha estén en formato datetime
df_resultado['Originado'] = pd.to_datetime(df_resultado['Originado'], errors='coerce')
df_referencia_fecha_pab['Originado'] = pd.to_datetime(df_referencia_fecha_pab['Originado'], errors='coerce')

# Hacemos una copia para no alterar el original
df_ref = df_referencia_fecha_pab.copy()

# Renombrar 'reference' a 'Referencia' para que coincidan las claves
df_ref = df_ref.rename(columns={'reference': 'Referencia'})

# Columnas que queremos traer
cols_a_agregar = ['requester', 'bank', 'amount_promedio', 'amount_total', 'n_pagos', 'PaB_total', 'n_pagos_PaB']

# Merge con las dos llaves
df_resultado = df_resultado.merge(
    df_ref[['Referencia', 'Originado'] + cols_a_agregar],
    on=['Referencia', 'Originado'],
    how='left'  # left mantiene todas las filas del df_resultado
)

# Resultado listo
print("Columnas nuevas agregadas correctamente ✅")

Columnas nuevas agregadas correctamente ✅


In [ ]:
df_resultado

,Referencia,Credito,Status facturacion,Tipo de comision,Monto,Fecha de facturacion,Apartado Mensual,Vehiculo de ahorro,Deuda inicial Fija,Mes_Año,...,Rango_de_deuda,mes_exacto,Originado,requester,bank,amount_promedio,amount_total,n_pagos,PaB_total,n_pagos_PaB
0,1000185400,false,COBRADO,LIQUIDACION_COLOMBIA,19405.915966,2025-06-20,520791.11,SKANDIA,22718600.0,02-2025,...,< 35 M,2025-06-01,2025-06-19,Hector Elian Lacera Vega,Rapicredit,53546.50,107093.0,2,552839.0,1
1,1000185400,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,15060.000000,2025-06-19,520791.11,SKANDIA,22718600.0,02-2025,...,< 35 M,2025-06-01,2025-06-19,Hector Elian Lacera Vega,Rapicredit,53546.50,107093.0,2,552839.0,1
2,1000185400,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,55528.000000,2025-06-19,520791.11,SKANDIA,22718600.0,02-2025,...,< 35 M,2025-07-01,2025-06-19,Hector Elian Lacera Vega,Rapicredit,53546.50,107093.0,2,552839.0,1
3,1000591190,false,COBRADO,LIQUIDACION_COLOMBIA,128644.546218,2025-09-12,344986.00,SKANDIA,12465987.0,12-2024,...,< 35 M,2025-09-01,2025-09-11,Steven Mateo Aroca Garzon,Falabella,76543.50,153087.0,2,1930000.0,1
4,1000591190,true,COBRADO,LIQUIDACION_COLOMBIA,214365.000000,2025-09-30,344986.00,SKANDIA,12465987.0,12-2024,...,< 35 M,2025-10-01,2025-09-11,Steven Mateo Aroca Garzon,Falabella,76543.50,153087.0,2,1930000.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29288,99534,false,COBRADO,LIQUIDACION_COLOMBIA,328455.000000,2022-02-14,287861.00,A&V,9030750.0,06-2021,...,< 35 M,2022-02-01,2022-02-14,Maria Paula Balaguera Penagos,Scotiabank Colpatria,390861.00,390861.0,1,1800000.0,1
29289,99534,false,COBRADO,LIQUIDACION_COLOMBIA,73231.000000,2022-08-10,287861.00,A&V,9030750.0,06-2021,...,< 35 M,2022-08-01,2022-08-10,Natalia Valentina Castro Jimenez,Colsubsidio,145242.33,435727.0,3,2600000.0,1
29290,99534,false,COBRADO,LIQUIDACION_COLOMBIA,146463.000000,2022-09-12,287861.00,A&V,9030750.0,06-2021,...,< 35 M,2022-09-01,2022-08-10,Natalia Valentina Castro Jimenez,Colsubsidio,145242.33,435727.0,3,2600000.0,1
29291,99534,false,COBRADO,LIQUIDACION_COLOMBIA,146464.000000,2022-09-28,287861.00,A&V,9030750.0,06-2021,...,< 35 M,2022-10-01,2022-08-10,Natalia Valentina Castro Jimenez,Colsubsidio,145242.33,435727.0,3,2600000.0,1


In [ ]:
import pandas as pd
import numpy as np

# Aseguramos que las fechas estén bien formateadas
df_resultado['Originado'] = pd.to_datetime(df_resultado['Originado'], errors='coerce')
df_resultado['mes_exacto'] = pd.to_datetime(df_resultado['mes_exacto'], errors='coerce')

# 1️⃣ Crear la columna con mes y año de la fecha de originación
df_resultado['Mes_año_Originacion'] = df_resultado['Originado'].dt.to_period('M').astype(str)
# Ejemplo: "2023-07"

# 2️⃣ Calcular los meses transcurridos entre Originado y mes_exacto
#    (año_diff * 12 + mes_diff)
df_resultado['Meses_despues_Originacion'] = (
    (df_resultado['mes_exacto'].dt.year - df_resultado['Originado'].dt.year) * 12 +
    (df_resultado['mes_exacto'].dt.month - df_resultado['Originado'].dt.month)
)

# Si alguna fecha es NaT → dejar como NaN
df_resultado['Meses_despues_Originacion'] = df_resultado['Meses_despues_Originacion'].where(
    ~df_resultado['Meses_despues_Originacion'].isna(), np.nan
)

# ✅ Resultado final
print(df_resultado[['Referencia', 'Originado', 'mes_exacto', 'Mes_año_Originacion', 'Meses_despues_Originacion']].head(10))

   Referencia  Originado mes_exacto Mes_año_Originacion  \
0  1000185400 2025-06-19 2025-06-01             2025-06   
1  1000185400 2025-06-19 2025-06-01             2025-06   
2  1000185400 2025-06-19 2025-07-01             2025-06   
3  1000591190 2025-09-11 2025-09-01             2025-09   
4  1000591190 2025-09-11 2025-10-01             2025-09   
5      100076 2022-12-23 2022-12-01             2022-12   
6      100076 2023-01-11 2023-01-01             2023-01   
7  1001082965 2024-11-18 2024-11-01             2024-11   
8  1001082965 2024-12-23 2024-12-01             2024-12   
9  1001082965 2025-01-31 2025-02-01             2025-01   

   Meses_despues_Originacion  
0                          0  
1                          0  
2                          1  
3                          0  
4                          1  
5                          0  
6                          0  
7                          0  
8                          0  
9                          1  


In [ ]:
import pandas as pd
import numpy as np

# Aseguramos que las fechas estén bien formateadas
df_resultado['Fecha de facturacion'] = pd.to_datetime(df_resultado['Fecha de facturacion'], errors='coerce')
df_resultado['mes_exacto'] = pd.to_datetime(df_resultado['mes_exacto'], errors='coerce')

# 1️⃣ Crear la columna con mes y año de la fecha de originación
df_resultado['Mes_año_Originacion'] = df_resultado['Fecha de facturacion'].dt.to_period('M').astype(str)
# Ejemplo: "2023-07"

# 2️⃣ Calcular los meses transcurridos entre Originado y mes_exacto
#    (año_diff * 12 + mes_diff)
df_resultado['Meses_despues_facturacion'] = (
    (df_resultado['mes_exacto'].dt.year - df_resultado['Fecha de facturacion'].dt.year) * 12 +
    (df_resultado['mes_exacto'].dt.month - df_resultado['Fecha de facturacion'].dt.month)
)

# Si alguna fecha es NaT → dejar como NaN
df_resultado['Meses_despues_facturacion'] = df_resultado['Meses_despues_facturacion'].where(
    ~df_resultado['Meses_despues_facturacion'].isna(), np.nan
)

# ✅ Resultado final
print(df_resultado[['Referencia', 'Fecha de facturacion', 'mes_exacto', 'Mes_año_Originacion', 'Meses_despues_facturacion']].head(10))

   Referencia Fecha de facturacion mes_exacto Mes_año_Originacion  \
0  1000185400           2025-06-20 2025-06-01             2025-06   
1  1000185400           2025-06-19 2025-06-01             2025-06   
2  1000185400           2025-06-19 2025-07-01             2025-06   
3  1000591190           2025-09-12 2025-09-01             2025-09   
4  1000591190           2025-09-30 2025-10-01             2025-09   
5      100076           2022-12-23 2022-12-01             2022-12   
6      100076           2023-01-11 2023-01-01             2023-01   
7  1001082965           2024-11-19 2024-11-01             2024-11   
8  1001082965           2024-12-23 2024-12-01             2024-12   
9  1001082965           2025-02-03 2025-02-01             2025-02   

   Meses_despues_facturacion  
0                          0  
1                          0  
2                          1  
3                          0  
4                          1  
5                          0  
6                      

In [ ]:
berex_estruct['amount'] = pd.to_numeric(berex_estruct['amount'], errors='coerce')
berex_estruct.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62038 entries, 0 to 62037
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   reference       62038 non-null  object        
 1   id              50296 non-null  object        
 2   amount          61384 non-null  float64       
 3   bank            62038 non-null  object        
 4   debt_id         62038 non-null  object        
 5   destination     62038 non-null  object        
 6   payment_date    62004 non-null  datetime64[ns]
 7   payment_number  62038 non-null  object        
 8   requester       62038 non-null  object        
 9   Originado       62004 non-null  datetime64[ns]
 10  Unnamed: 10     50302 non-null  object        
 11  Unnamed: 11     0 non-null      object        
 12  Unnamed: 12     50302 non-null  object        
 13  Estructurado    62038 non-null  int64         
dtypes: datetime64[ns](2), float64(1), int64(1), object(10)

In [ ]:
berex_estruct

,reference,id,amount,bank,debt_id,destination,payment_date,payment_number,requester,Originado,Unnamed: 10,Unnamed: 11,Unnamed: 12,Estructurado
0,3183831101,3295,350824.0,Alkosto,544721,commission,2021-01-12,2,Natalia Valentina Castro Jimenez,2021-01-12,318383110144208,NaN,294810.084,1
1,3183831101,3294,500000.0,Alkosto,544721,bank,2021-01-12,1,Natalia Valentina Castro Jimenez,2021-01-12,318383110144208,NaN,420168.0672,1
2,3183831101,3296,436000.0,Alkosto,544721,bank,2021-02-19,3,Natalia Valentina Castro Jimenez,2021-01-12,318383110144208,NaN,366386.5546,1
3,3183831101,3297,436000.0,Alkosto,544721,bank,2021-03-19,4,Natalia Valentina Castro Jimenez,2021-01-12,318383110144208,NaN,366386.5546,1
4,3504389669,3941,131198.0,Alkosto,510212,commission,2021-01-30,2,No Asignado,2021-01-30,350438966944226,NaN,110250.4202,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62033,3132820285,NaN,1045113.0,Davivienda,1097209,commission,2025-10-03,1,Diego Alejandro Sanchez Fonseca,2025-10-03,NaN,NaN,NaN,0
62034,79377506,NaN,62609.0,Falabella,1605967,commission,2025-10-03,1,Juan Sebastian Lopez Martinez,2025-10-03,NaN,NaN,NaN,0
62035,3102587912,NaN,514361.0,Davivienda,1422192,commission,2025-10-06,1,Nina Mayerly Gomez Guerrero,2025-10-06,NaN,NaN,NaN,0
62036,3102587912,NaN,25704.0,Davivienda,1422190,commission,2025-10-07,1,Nina Mayerly Gomez Guerrero,2025-10-07,NaN,NaN,NaN,0


In [ ]:
import pandas as pd

# ==============================
# Parámetros configurables
# ==============================
DENOMINADOR_MESES = 30.4375  # promedio gregoriano de días por mes
DECIMALES_PRI_ULT = 2        # decimales para 'Pri-ult'

# ==============================
# Claves y columnas base
# ==============================
keys = ['reference', 'Originado', 'destination']
cols = ['reference', 'amount', 'payment_date', 'Originado', 'destination']

# ==============================
# Partimos de berex_estruct
# ==============================
df = berex_estruct[cols].copy()

# Tipos
df['amount'] = pd.to_numeric(df['amount'], errors='coerce')
df['payment_date'] = pd.to_datetime(df['payment_date'], errors='coerce')

# =========================================
# A) COLAPSAR PAGOS DENTRO DEL MISMO MES
#    - Suma 'amount' por mes
#    - Conserva la payment_date más reciente de ese mes
# =========================================
df['_ym'] = df['payment_date'].dt.to_period('M')  # año-mes

df_mensual = (
    df.groupby(keys + ['_ym'], dropna=False)
      .agg(
          amount=('amount', 'sum'),              # suma de montos del mes
          payment_date=('payment_date', 'max')   # última fecha del mes
      )
      .reset_index()
)

# =========================================
# B) Calcular Pri-ult (en meses decimales) con min/max del grupo
#    *Ajuste: fecha_min = 1er día del mes; fecha_max = último día del mes*
#    *Se calcula sobre df_mensual, no sobre los registros diarios*
# =========================================
agg = (
    df_mensual.groupby(keys, dropna=False)['payment_date']
              .agg(fecha_min='min', fecha_max='max')
)

# Primer día del mes del primer pago (00:00:00)
fecha_min_floor = agg['fecha_min'].dt.to_period('M').dt.start_time

# Último día del mes del último pago (fin del día)
fecha_max_eom = agg['fecha_max'].dt.to_period('M').dt.end_time

# Diferencia exacta en días
dias = (fecha_max_eom - fecha_min_floor).dt.days

# Convertir a meses decimales
pri_ult = dias / DENOMINADOR_MESES

# Manejo de nulos y formato final
pri_ult = pri_ult.where(agg['fecha_min'].notna() & agg['fecha_max'].notna())
pri_ult = pri_ult.round(DECIMALES_PRI_ULT).astype('float')  # decimal con redondeo
pri_ult.name = 'Pri-ult'

# Unir Pri-ult al DF mensual
df_mensual = df_mensual.merge(pri_ult, left_on=keys, right_index=True, how='left')

# =========================================
# C) Ordenar y quedarnos con la payment_date más antigua por grupo
#    (ya no hay duplicados por mes, así que esto elige el mes más antiguo)
# =========================================
df_mensual_sorted = df_mensual.sort_values(keys + ['payment_date'], ascending=True)

df_unico_antiguo = (
    df_mensual_sorted
      .drop_duplicates(subset=keys, keep='first')  # toma el mes más antiguo por grupo
      .reset_index(drop=True)
)

# Limpieza opcional
df_unico_antiguo = df_unico_antiguo.drop(columns=['_ym'])

In [ ]:
referencia_buscada = '3002426421'

resultado = df_resultado[df_resultado['Referencia'] == referencia_buscada]

resultado

,Referencia,Credito,Status facturacion,Tipo de comision,Monto,Fecha de facturacion,Apartado Mensual,Vehiculo de ahorro,Deuda inicial Fija,Mes_Año,...,requester,bank,amount_promedio,amount_total,n_pagos,PaB_total,n_pagos_PaB,Mes_año_Originacion,Meses_despues_Originacion,Meses_despues_facturacion
1458,3002426421,false,COBRADO,LIQUIDACION_COLOMBIA,81630.0000,2024-04-27,405212.95,POWWI,17389250.0,03-2022,...,Angela Yara Mayorga,Serfinanza,97140.00,97140.0,1,2610000.0,1,2024-04,0,0
1459,3002426421,false,COBRADO,LIQUIDACION_COLOMBIA,420168.0672,2024-06-11,405212.95,POWWI,17389250.0,03-2022,...,Nicolle Saray Catica Trujillo,CREDIVALORES,336436.67,1009310.0,3,2204000.0,2,2024-06,0,0
1460,3002426421,false,COBRO_PARCIAL_INCOBRABLE,LIQUIDACION_COLOMBIA,80755.0000,2024-06-11,405212.95,POWWI,17389250.0,03-2022,...,Nicolle Saray Catica Trujillo,CREDIVALORES,336436.67,1009310.0,3,2204000.0,2,2024-06,1,1
1461,3002426421,false,COBRO_PARCIAL_INCOBRABLE,LIQUIDACION_COLOMBIA,170.0000,2024-06-11,405212.95,POWWI,17389250.0,03-2022,...,Nicolle Saray Catica Trujillo,CREDIVALORES,336436.67,1009310.0,3,2204000.0,2,2024-06,2,2


In [ ]:
df_unico_antiguo

,reference,Originado,destination,amount,payment_date,Pri-ult
0,1000185400,2025-06-19,bank,552839.0,2025-06-19,0.95
1,1000185400,2025-06-19,commission,23093.0,2025-06-19,1.97
2,1000591190,2025-09-11,bank,1930000.0,2025-09-11,0.95
3,1000591190,2025-09-11,commission,76000.0,2025-09-11,1.97
4,100076,2022-12-23,bank,2942000.0,2022-12-23,0.99
...,...,...,...,...,...,...
27250,99534,2022-02-14,commission,390861.0,2022-02-14,0.89
27251,99534,2022-08-10,bank,2600000.0,2022-08-10,0.99
27252,99534,2022-08-10,commission,261436.0,2022-08-10,1.97
27253,9957452,2024-08-13,bank,2800000.0,2024-08-13,0.99


#LLAMAMOS MENSUALIDADES DE DF_MORA_ESTADOS

In [ ]:
# === 1️⃣ Librerías necesarias ===
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials
from google.colab import userdata

# === 2️⃣ Credenciales seguras ===
mi_json = userdata.get("MI_JSON")

scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive"
]

creds = Credentials.from_service_account_info(
    eval(mi_json), scopes=scope
)
client = gspread.authorize(creds)

# === 3️⃣ Abrir el archivo y buscar la hoja por su GID ===
sheet_id = "1jcPPhtF2YK3Kr7P_A0Mgh2OqhOfnVWB2to3UPoSH5tE"
sheet = client.open_by_key(sheet_id)

# Buscar la hoja con el gid que mencionaste
target_gid = "702333742"

worksheet = None
for ws in sheet.worksheets():
    if str(ws.id) == target_gid:
        worksheet = ws
        break

if worksheet is None:
    raise ValueError(f"No se encontró una hoja con gid={target_gid}")

# === 4️⃣ Cargar los datos a un DataFrame ===
data = worksheet.get_all_records()
df_mensualidades = pd.DataFrame(data)

# === 5️⃣ Mostrar una vista previa ===
print(f"✅ Hoja '{worksheet.title}' cargada correctamente desde Google Sheets.")
print(f"Filas: {df_mensualidades.shape[0]}, Columnas: {df_mensualidades.shape[1]}")
df_mensualidades.head()

✅ Hoja 'Mensualidades' cargada correctamente desde Google Sheets.
Filas: 164380, Columnas: 8


,Referencia,Fecha de corte,Fecha de facturacion,X_COBRAR,C_Parcial,X_COBRAR_ACUMULADO,DIAS_EN_MORA,Status_Mora
0,1001082965,2025-02-28 0:00:00,2025-02-06 0:00:00,599181.0,0,599181.0,22,Mora 1
1,1001082965,2025-03-31 0:00:00,2025-03-06 0:00:00,599181.0,0,1198362.0,53,Mora 30
2,1001082965,2025-04-30 0:00:00,,0.0,,1198362.0,83,Mora 60
3,1001082965,2025-05-31 0:00:00,,0.0,,1198362.0,114,Mora 90
4,1001082965,2025-06-30 0:00:00,,0.0,,1198362.0,144,Mora 120


In [ ]:
# Convertir 'Referencia' a tipo object (string)
df_mensualidades['Referencia'] = df_mensualidades['Referencia'].astype(str)

# Convertir las fechas a datetime (maneja errores automáticamente)
df_mensualidades['Fecha de corte'] = pd.to_datetime(
    df_mensualidades['Fecha de corte'], errors='coerce', dayfirst=True
)
df_mensualidades['Fecha de facturacion'] = pd.to_datetime(
    df_mensualidades['Fecha de facturacion'], errors='coerce', dayfirst=True
)

# Verificar tipos finales
df_mensualidades.dtypes

/tmp/ipython-input-2878630590.py:5: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df_mensualidades['Fecha de corte'] = pd.to_datetime(


,0
Referencia,object
Fecha de corte,datetime64[ns]
Fecha de facturacion,datetime64[ns]
X_COBRAR,float64
C_Parcial,object
X_COBRAR_ACUMULADO,float64
DIAS_EN_MORA,int64
Status_Mora,object


In [ ]:
df_mensualidades

,Referencia,Fecha de corte,Fecha de facturacion,X_COBRAR,C_Parcial,X_COBRAR_ACUMULADO,DIAS_EN_MORA,Status_Mora
0,1001082965,2025-02-28,2025-06-02,599181.0,0,599181.0,22,Mora 1
1,1001082965,2025-03-31,2025-06-03,599181.0,0,1198362.0,53,Mora 30
2,1001082965,2025-04-30,NaT,0.0,,1198362.0,83,Mora 60
3,1001082965,2025-05-31,NaT,0.0,,1198362.0,114,Mora 90
4,1001082965,2025-06-30,NaT,0.0,,1198362.0,144,Mora 120
...,...,...,...,...,...,...,...,...
164375,98702966,2025-06-30,NaT,0.0,,1262183.0,388,Mora 180
164376,98702966,2025-07-31,NaT,0.0,,1262183.0,419,Mora 180
164377,98702966,2025-08-31,NaT,0.0,,1262183.0,450,Mora 180
164378,98702966,2025-09-30,2025-03-09,393314.0,0,1655497.0,480,Mora 180


In [ ]:
df_resultado.info()
df_mensualidades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29293 entries, 0 to 29292
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Referencia                 29293 non-null  object        
 1   Credito                    29293 non-null  object        
 2   Status facturacion         29293 non-null  object        
 3   Tipo de comision           29293 non-null  object        
 4   Monto                      29293 non-null  float64       
 5   Fecha de facturacion       29293 non-null  datetime64[ns]
 6   Apartado Mensual           29293 non-null  float64       
 7   Vehiculo de ahorro         29293 non-null  object        
 8   Deuda inicial Fija         29293 non-null  float64       
 9   Mes_Año                    29293 non-null  object        
 10  Mes_Cobro                  29293 non-null  float64       
 11  AM/DB                      29293 non-null  float64       
 12  AM/D

In [ ]:
import pandas as pd
import numpy as np

# Copias de trabajo
dfu = df_unico_antiguo.copy()
dfm = df_mensualidades.copy()

# Tipos consistentes
dfu['reference'] = dfu['reference'].astype(str)
dfm['Referencia'] = dfm['Referencia'].astype(str)

dfu['Originado'] = pd.to_datetime(dfu['Originado'], errors='coerce')
dfu['Pri-ult'] = pd.to_numeric(dfu['Pri-ult'], errors='coerce')  # permite NaN
dfm['Fecha de facturacion'] = pd.to_datetime(dfm['Fecha de facturacion'], errors='coerce')

# === 0) Pri-ult máximo por (reference, Originado), con ceil ===
priult_max = (
    dfu.groupby(['reference', 'Originado'], dropna=False)['Pri-ult']
       .max()                          # toma el mayor Pri-ult del grupo
       .reset_index(name='Pri_ult_max')
)
priult_max['Pri_ult_max'] = np.ceil(priult_max['Pri_ult_max'])

# === 1) Calcular fecha_fin_max usando ese Pri_ult_max (sin eliminar filas originales) ===
def calc_fecha_fin_max(row):
    originado = row['Originado']
    p = row['Pri_ult_max']
    if pd.isna(originado) or pd.isna(p):
        return pd.NaT
    return (originado + pd.DateOffset(months=int(p))) + pd.offsets.MonthEnd(0)

priult_max['fecha_fin_max'] = priult_max.apply(calc_fecha_fin_max, axis=1)

# === 2) Merge para traer facturas y filtrar por rango [Originado, fecha_fin_max] ===
tmp = priult_max.merge(
    dfm[['Referencia', 'Fecha de facturacion', 'X_COBRAR']],
    left_on='reference', right_on='Referencia', how='left'
)

mask = (
    tmp['Fecha de facturacion'].notna() &
    tmp['Originado'].notna() &
    tmp['fecha_fin_max'].notna() &
    (tmp['Fecha de facturacion'] >= tmp['Originado']) &
    (tmp['Fecha de facturacion'] <= tmp['fecha_fin_max'])
)
tmp_ok = tmp.loc[mask].copy()

# === 3) Sumar X_COBRAR dentro del rango por (reference, Originado) ===
suma_por_rango = (
    tmp_ok.groupby(['reference', 'Originado'], dropna=False)['X_COBRAR']
          .sum()
          .reset_index()
          .rename(columns={'X_COBRAR': 'Monto_Mensualidades'})
)

# === 4) Unir la suma al dfu ORIGINAL (sin eliminar sus filas) ===
dfu = dfu.merge(suma_por_rango, on=['reference', 'Originado'], how='left')

# Si no hay Originado o no hubo Pri-ult válido en el grupo, dejar NaN
falta_info = dfu['Originado'].isna() | dfu.groupby(['reference','Originado'])['Pri-ult'].transform('max').isna()
dfu.loc[falta_info, 'Monto_Mensualidades'] = np.nan

# === 5) Resultado final (no tocamos filas de dfu, solo agregamos la columna) ===
df_unico_antiguo = dfu  # ya incluye 'Monto_Mensualidades'

In [ ]:
df_unico_antiguo

,reference,Originado,destination,amount,payment_date,Pri-ult,Monto_Mensualidades
0,1000185400,2025-06-19,bank,552839.0,2025-06-19,0.95,NaN
1,1000185400,2025-06-19,commission,23093.0,2025-06-19,1.97,NaN
2,1000591190,2025-09-11,bank,1930000.0,2025-09-11,0.95,NaN
3,1000591190,2025-09-11,commission,76000.0,2025-09-11,1.97,NaN
4,100076,2022-12-23,bank,2942000.0,2022-12-23,0.99,NaN
...,...,...,...,...,...,...,...
27250,99534,2022-02-14,commission,390861.0,2022-02-14,0.89,NaN
27251,99534,2022-08-10,bank,2600000.0,2022-08-10,0.99,NaN
27252,99534,2022-08-10,commission,261436.0,2022-08-10,1.97,NaN
27253,9957452,2024-08-13,bank,2800000.0,2024-08-13,0.99,NaN


In [ ]:
referencia_buscada = '3233199692'

resultado = df_unico_antiguo[df_unico_antiguo['reference'] == referencia_buscada]

resultado

,reference,Originado,destination,amount,payment_date,Pri-ult,Monto_Mensualidades
24904,3233199692,2025-01-15,bank,2720000.0,2025-01-15,1.91,205413.0
24905,3233199692,2025-01-15,commission,59773.0,2025-01-15,3.91,205413.0


In [ ]:
# Renombrar la columna 'bank' a 'BANCO' en df_resultado
if 'bank' in df_resultado.columns:
    df_resultado.rename(columns={'bank': 'BANCO'}, inplace=True)

In [ ]:
import numpy as np
import pandas as pd

# =========================================
# 1) Insumos desde df_unico_antiguo (llave: ['reference','Originado'])
# =========================================
dfu = df_unico_antiguo.copy()

# Normalizaciones mínimas de columnas de valor (NO tocamos la llave)
dfu['destination'] = dfu['destination'].astype(str).str.strip().str.lower()
for c in ['amount', 'Pri-ult', 'Monto_Mensualidades']:
    dfu[c] = pd.to_numeric(dfu[c], errors='coerce')

# --- Pivot: amounts por destino (index = llave exacta) ---
pivot_amt = (
    dfu.pivot_table(
        index=['reference', 'Originado'],
        columns='destination',
        values='amount',
        aggfunc='first'
    )
    .reset_index()
    .rename(columns={'bank': 'bank_monto', 'commission': 'commission_monto'})
)

# Asegurar columnas numéricas aunque no existan
for col in ['bank_monto', 'commission_monto']:
    if col not in pivot_amt.columns:
        pivot_amt[col] = 0.0
pivot_amt[['bank_monto', 'commission_monto']] = pivot_amt[['bank_monto', 'commission_monto']].fillna(0.0)

# --- ÚNICO Pri-ult por llave: usar el MAYOR ---
priult_final = (
    dfu.groupby(['reference', 'Originado'], dropna=False)['Pri-ult']
       .max()
       .reset_index()
       .rename(columns={'Pri-ult': 'Pri_ult_final'})
)

# --- ÚNICO Monto_Mensualidades por llave: usar el MAYOR ---
mens_final = (
    dfu.groupby(['reference', 'Originado'], dropna=False)['Monto_Mensualidades']
       .max()
       .reset_index()
)

# --- Ensamblar insumos por llave (1 fila por llave) ---
insumos = (
    pivot_amt
    .merge(priult_final, on=['reference', 'Originado'], how='left', validate='one_to_one')
    .merge(mens_final,   on=['reference', 'Originado'], how='left', validate='one_to_one')
    [['reference', 'Originado', 'bank_monto', 'commission_monto', 'Pri_ult_final', 'Monto_Mensualidades']]
)

# Validación de unicidad en insumos
if insumos.duplicated(['reference', 'Originado']).any():
    dups = insumos[insumos.duplicated(['reference','Originado'], keep=False)].sort_values(['reference','Originado'])
    raise ValueError(
        "La tabla 'insumos' no es única por ['reference','Originado'].\n"
        f"Ejemplos de duplicados:\n{dups.head(10)}"
    )

# =========================================
# 2) Merge con df_resultado (llave: ['Referencia','Originado'] ↔ ['reference','Originado'])
# =========================================
dfr = df_resultado.copy()

# Eliminar C/A y Pri-ult previos para evitar arrastres
for col in ['C/A', 'Pri-ult']:
    if col in dfr.columns:
        dfr.drop(columns=[col], inplace=True)

# Merge estrictamente por (Referencia|reference, Originado)
dfr = dfr.merge(
    insumos,
    left_on=['Referencia', 'Originado'],
    right_on=['reference', 'Originado'],
    how='left',
    validate='many_to_one'
)

# Usar una sola columna Pri-ult para el cálculo
dfr.rename(columns={'Pri_ult_final': 'Pri-ult'}, inplace=True)

# Limpiar auxiliares
if 'reference' in dfr.columns:
    dfr.drop(columns=['reference'], inplace=True)

# =========================================
# 3) Asegurar columnas necesarias
# =========================================
necesarias = [
    'PaB_total', 'amount_total', 'Apartado Mensual',
    'Pri-ult', 'bank_monto', 'commission_monto', 'Monto_Mensualidades'
]
for col in necesarias:
    if col not in dfr.columns:
        dfr[col] = np.nan

# Conversión a numérico
for c in necesarias:
    dfr[c] = pd.to_numeric(dfr[c], errors='coerce')

# 🔒 AJUSTE SOLICITADO: Antes de cualquier cálculo, forzar Pri-ult <= 1 a 1
dfr['Pri-ult'] = np.where(dfr['Pri-ult'].notna() & (dfr['Pri-ult'] <= 1), 1, dfr['Pri-ult'])

# =========================================
# 4) Cálculo C/A
# =========================================
# Numerador = PaB_total + amount_total - bank_monto - commission_monto + (Monto_Mensualidades * 1.19)
numerador = (
    dfr['PaB_total'].fillna(0)
    + dfr['amount_total'].fillna(0)
    - dfr['bank_monto'].fillna(0)
    - dfr['commission_monto'].fillna(0)
    + dfr['Monto_Mensualidades'].fillna(0) * 1.19
)

# Pri-ult base (ya con ≤1 => 1): si es NaN permanece NaN
priult_base = dfr['Pri-ult'].to_numpy()

# SOLO EN EL CÁLCULO: restar 1 excepto si es exactamente 1
priult_ajustada = np.where(priult_base == 1, 1, priult_base - 1)

# Evitar divisiones inválidas
priult_ajustada = pd.to_numeric(priult_ajustada, errors='coerce')
intermedio = numerador / priult_ajustada

# C/A se calcula solo cuando hay Apartado Mensual válido
dfr['C/A'] = np.where(
    (dfr['Apartado Mensual'].notna()) & (dfr['Apartado Mensual'] != 0),
    intermedio / dfr['Apartado Mensual'],
    np.nan
)

# =========================================
# 5) Salida
# =========================================
df_resultado = dfr.copy()
print("✅ Listo. Columnas:", df_resultado.columns.tolist())
print("🔐 Pri-ult <= 1 se fuerza a 1 antes de todo; en el denominador se resta 1 salvo que sea 1.")

✅ Listo. Columnas: ['Referencia', 'Credito', 'Status facturacion', 'Tipo de comision', 'Monto', 'Fecha de facturacion', 'Apartado Mensual', 'Vehiculo de ahorro', 'Deuda inicial Fija', 'Mes_Año', 'Mes_Cobro', 'AM/DB', 'AM/DB_outlier', 'Rango_AM_DB', 'Rango_de_deuda', 'mes_exacto', 'Originado', 'requester', 'BANCO', 'amount_promedio', 'amount_total', 'n_pagos', 'PaB_total', 'n_pagos_PaB', 'Mes_año_Originacion', 'Meses_despues_Originacion', 'Meses_despues_facturacion', 'bank_monto', 'commission_monto', 'Pri-ult', 'Monto_Mensualidades', 'C/A']
🔐 Pri-ult <= 1 se fuerza a 1 antes de todo; en el denominador se resta 1 salvo que sea 1.


In [ ]:
referencia_buscada = '3002426421'

resultado = df_unico_antiguo[df_unico_antiguo['reference'] == referencia_buscada]

resultado

,reference,Originado,destination,amount,payment_date,Pri-ult,Monto_Mensualidades
1353,3002426421,2024-04-27,bank,2610000.0,2024-04-27,0.95,10504.0
1354,3002426421,2024-04-27,commission,97140.0,2024-04-27,0.95,10504.0
1355,3002426421,2024-06-11,bank,2204000.0,2024-06-26,0.95,10504.0
1356,3002426421,2024-06-11,commission,500000.0,2024-06-11,3.98,10504.0


In [ ]:
# Filtrar las referencias deseadas
filtro = df_resultado['Referencia'].isin(['79621572', '3233199692', '3214703121', '3002426421'])

# Seleccionar solo las columnas específicas
df_resultado_filtrado = dfr.loc[filtro, ['Referencia', 'Pri-ult', 'Monto_Mensualidades', 'C/A']]

# Mostrar el resultado
df_resultado_filtrado

,Referencia,Pri-ult,Monto_Mensualidades,C/A
1458,3002426421,1.00,10504.0,0.030847
1459,3002426421,3.98,10504.0,0.432128
1460,3002426421,3.98,10504.0,0.432128
1461,3002426421,3.98,10504.0,0.432128
25535,3214703121,2.00,NaN,12.992014
27059,3233199692,3.91,205413.0,0.463366
27060,3233199692,3.91,205413.0,0.463366
27061,3233199692,3.91,205413.0,0.463366
27062,3233199692,3.91,205413.0,0.463366
28943,79621572,3.98,NaN,0.254893


In [ ]:
# Eliminar columnas no deseadas de df_resultado
df_resultado.drop(
    columns=['reference', 'bank_y', 'commission', 'bank'],
    inplace=True,
    errors='ignore'  # evita error si alguna no existe
)

In [ ]:
df_resultado

,Referencia,Credito,Status facturacion,Tipo de comision,Monto,Fecha de facturacion,Apartado Mensual,Vehiculo de ahorro,Deuda inicial Fija,Mes_Año,...,PaB_total,n_pagos_PaB,Mes_año_Originacion,Meses_despues_Originacion,Meses_despues_facturacion,bank_monto,commission_monto,Pri-ult,Monto_Mensualidades,C/A
0,1000185400,false,COBRADO,LIQUIDACION_COLOMBIA,19405.915966,2025-06-20,520791.11,SKANDIA,22718600.0,02-2025,...,552839.0,1,2025-06,0,0,552839.0,23093.0,1.97,NaN,0.166282
1,1000185400,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,15060.000000,2025-06-19,520791.11,SKANDIA,22718600.0,02-2025,...,552839.0,1,2025-06,0,0,552839.0,23093.0,1.97,NaN,0.166282
2,1000185400,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,55528.000000,2025-06-19,520791.11,SKANDIA,22718600.0,02-2025,...,552839.0,1,2025-06,1,1,552839.0,23093.0,1.97,NaN,0.166282
3,1000591190,false,COBRADO,LIQUIDACION_COLOMBIA,128644.546218,2025-09-12,344986.00,SKANDIA,12465987.0,12-2024,...,1930000.0,1,2025-09,0,0,1930000.0,76000.0,1.97,NaN,0.230360
4,1000591190,true,COBRADO,LIQUIDACION_COLOMBIA,214365.000000,2025-09-30,344986.00,SKANDIA,12465987.0,12-2024,...,1930000.0,1,2025-09,1,1,1930000.0,76000.0,1.97,NaN,0.230360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29288,99534,false,COBRADO,LIQUIDACION_COLOMBIA,328455.000000,2022-02-14,287861.00,A&V,9030750.0,06-2021,...,1800000.0,1,2022-02,0,0,1800000.0,390861.0,1.00,NaN,0.000000
29289,99534,false,COBRADO,LIQUIDACION_COLOMBIA,73231.000000,2022-08-10,287861.00,A&V,9030750.0,06-2021,...,2600000.0,1,2022-08,0,0,2600000.0,261436.0,1.97,NaN,0.624195
29290,99534,false,COBRADO,LIQUIDACION_COLOMBIA,146463.000000,2022-09-12,287861.00,A&V,9030750.0,06-2021,...,2600000.0,1,2022-09,1,0,2600000.0,261436.0,1.97,NaN,0.624195
29291,99534,false,COBRADO,LIQUIDACION_COLOMBIA,146464.000000,2022-09-28,287861.00,A&V,9030750.0,06-2021,...,2600000.0,1,2022-09,2,1,2600000.0,261436.0,1.97,NaN,0.624195


In [ ]:
# Calcular la diferencia en días (valor absoluto)
df_resultado['diferencia_dias'] = (df_resultado['Fecha de facturacion'] - df_resultado['Originado']).abs().dt.days

# Filtrar filas donde la diferencia sea menor o igual a 3
df_filtrado = df_resultado[df_resultado['diferencia_dias'] <= 3]

# Eliminar la columna auxiliar si ya no la necesitas
df_filtrado = df_filtrado.drop(columns='diferencia_dias')

In [ ]:
df_filtrado

,Referencia,Credito,Status facturacion,Tipo de comision,Monto,Fecha de facturacion,Apartado Mensual,Vehiculo de ahorro,Deuda inicial Fija,Mes_Año,...,PaB_total,n_pagos_PaB,Mes_año_Originacion,Meses_despues_Originacion,Meses_despues_facturacion,bank_monto,commission_monto,Pri-ult,Monto_Mensualidades,C/A
0,1000185400,false,COBRADO,LIQUIDACION_COLOMBIA,1.940592e+04,2025-06-20,520791.11,SKANDIA,22718600.0,02-2025,...,552839.0,1,2025-06,0,0,552839.0,23093.0,1.97,NaN,0.166282
1,1000185400,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,1.506000e+04,2025-06-19,520791.11,SKANDIA,22718600.0,02-2025,...,552839.0,1,2025-06,0,0,552839.0,23093.0,1.97,NaN,0.166282
2,1000185400,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,5.552800e+04,2025-06-19,520791.11,SKANDIA,22718600.0,02-2025,...,552839.0,1,2025-06,1,1,552839.0,23093.0,1.97,NaN,0.166282
3,1000591190,false,COBRADO,LIQUIDACION_COLOMBIA,1.286445e+05,2025-09-12,344986.00,SKANDIA,12465987.0,12-2024,...,1930000.0,1,2025-09,0,0,1930000.0,76000.0,1.97,NaN,0.230360
5,100076,false,COBRADO,LIQUIDACION_COLOMBIA,1.754700e+05,2022-12-23,359735.00,A&V,17041870.0,06-2021,...,2942000.0,1,2022-12,0,0,2942000.0,208809.0,1.00,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29286,98709164,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,3.215280e+05,2025-09-01,515382.43,SKANDIA,12043200.0,01-2025,...,1800000.0,1,2025-09,2,1,1800000.0,769397.0,2.96,NaN,0.553309
29287,98709164,true,COBRADO,LIQUIDACION_COLOMBIA,1.048366e+06,2025-09-01,515382.43,SKANDIA,12043200.0,01-2025,...,1800000.0,1,2025-09,2,1,1800000.0,769397.0,2.96,NaN,0.553309
29288,99534,false,COBRADO,LIQUIDACION_COLOMBIA,3.284550e+05,2022-02-14,287861.00,A&V,9030750.0,06-2021,...,1800000.0,1,2022-02,0,0,1800000.0,390861.0,1.00,NaN,0.000000
29289,99534,false,COBRADO,LIQUIDACION_COLOMBIA,7.323100e+04,2022-08-10,287861.00,A&V,9030750.0,06-2021,...,2600000.0,1,2022-08,0,0,2600000.0,261436.0,1.97,NaN,0.624195


In [ ]:
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24853 entries, 0 to 29292
Data columns (total 32 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Referencia                 24853 non-null  object        
 1   Credito                    24853 non-null  object        
 2   Status facturacion         24853 non-null  object        
 3   Tipo de comision           24853 non-null  object        
 4   Monto                      24853 non-null  float64       
 5   Fecha de facturacion       24853 non-null  datetime64[ns]
 6   Apartado Mensual           24853 non-null  float64       
 7   Vehiculo de ahorro         24853 non-null  object        
 8   Deuda inicial Fija         24853 non-null  float64       
 9   Mes_Año                    24853 non-null  object        
 10  Mes_Cobro                  24853 non-null  float64       
 11  AM/DB                      24853 non-null  float64       
 12  AM/DB_out

In [ ]:
# --- Requisitos (ejecútalo una vez por sesión) ---
!pip -q install gspread gspread-dataframe

import json
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe
from google.colab import userdata

# 1) Leer tu JSON de servicio desde el secreto MI_JSON
mi_json = userdata.get("MI_JSON")  # <- como acordamos, SIEMPRE leer así
if not mi_json:
    raise RuntimeError("No se encontró el secreto MI_JSON en Colab. Verifica en 'Entorno de ejecución > Configurar claves de usuario'.")

svc_info = json.loads(mi_json)

# (Opcional) Mostrar el correo del service account para compartir la hoja si hace falta
print("Comparte tu Google Sheet con este correo (permiso de lector o editor):")
print(svc_info.get("client_email"))

# 2) Autenticación con gspread usando el JSON en memoria
gc = gspread.service_account_from_dict(svc_info)

# 3) Abrir el Google Sheet por URL
url = "https://docs.google.com/spreadsheets/d/1jcPPhtF2YK3Kr7P_A0Mgh2OqhOfnVWB2to3UPoSH5tE/edit?gid=916870612#gid=916870612"
sh = gc.open_by_url(url)

# 4) Seleccionar la pestaña por GID (916870612)
ws = sh.get_worksheet_by_id(916870612)
if ws is None:
    raise ValueError("No se encontró una pestaña con gid=916870612. Revisa el gid en la URL.")

# 5) Descargar a DataFrame
#    - keep_default_na=False evita que strings como 'NA' se conviertan en NaN sin querer
#    - evaluate_formulas=True trae valores calculados si hay fórmulas
df_mora = get_as_dataframe(
    ws,
    evaluate_formulas=True,
    header=0,
    dtype=None,
    keep_default_na=False,
    na_filter=True
)

# 6) Limpieza básica: eliminar filas completamente vacías
df_mora = df_mora.dropna(how="all").reset_index(drop=True)

# 7) (Opcional) Intentar parsear columnas de fecha comunes
for col in ["FECHA", "Mes_año_Originacion"]:
    if col in df_mora.columns:
        df_mora[col] = pd.to_datetime(df_mora[col], errors="coerce", dayfirst=True)

# 8) Verificación rápida
print("Filas:", len(df_mora), "| Columnas:", len(df_mora.columns))
df_mora.head()

Comparte tu Google Sheet con este correo (permiso de lector o editor):
colabservice@colabaccess-468021.iam.gserviceaccount.com


/tmp/ipython-input-3309028506.py:51: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df_mora[col] = pd.to_datetime(df_mora[col], errors="coerce", dayfirst=True)


Filas: 81473 | Columnas: 15


,REFERENCIA,FECHA,X_COBRAR,PAGO,X_COBRAR_FUTURO,PLAN_PAGADO,PLAN_PAGADO_INCOBRABLE,FECHA_ORIGEN,FECHA_COBRO,RESPONSABLE,FECHA_DE_PAGO,MOROSO,DIAS_EN_MORA,MORA_STATUS,Saldo actual
0,1000185400,2025-06-30,19405.88,89993.915966,70588.24,-70588.035966,-70588.035966,2025-06-19 0:00:00,2025-06-20 0:00:00,Hector Elian Lacera Vega,2025-06-19 0:00:00,False,0,Al día,
1,1000185400,2025-07-31,70588.24,0.000000,0.00,0.204034,0.204034,,,Hector Elian Lacera Vega,2025-07-05 0:00:00,False,0,Cerrado,
2,1000185400,2025-08-31,0.00,0.000000,0.00,0.204034,0.204034,,,Hector Elian Lacera Vega,,False,0,Cerrado,
3,1000591190,2025-09-30,63865.55,343009.546218,64778.99,-622068.996218,-622068.996218,2025-09-11 0:00:00,2025-09-15 0:00:00,Steven Mateo Aroca Garzon,2025-09-11 0:00:00,False,0,Al día,
4,1000591190,2025-10-28,64778.99,0.000000,0.00,-557290.006218,-557290.006218,,,Steven Mateo Aroca Garzon,2025-10-15 0:00:00,False,0,Al día,


In [ ]:
df_mora

,REFERENCIA,FECHA,X_COBRAR,PAGO,X_COBRAR_FUTURO,PLAN_PAGADO,PLAN_PAGADO_INCOBRABLE,FECHA_ORIGEN,FECHA_COBRO,RESPONSABLE,FECHA_DE_PAGO,MOROSO,DIAS_EN_MORA,MORA_STATUS,Saldo actual
0,1000185400,2025-06-30,19405.88,89993.915966,70588.24,-70588.035966,-70588.035966,2025-06-19 0:00:00,2025-06-20 0:00:00,Hector Elian Lacera Vega,2025-06-19 0:00:00,False,0,Al día,
1,1000185400,2025-07-31,70588.24,0.000000,0.00,0.204034,0.204034,,,Hector Elian Lacera Vega,2025-07-05 0:00:00,False,0,Cerrado,
2,1000185400,2025-08-31,0.00,0.000000,0.00,0.204034,0.204034,,,Hector Elian Lacera Vega,,False,0,Cerrado,
3,1000591190,2025-09-30,63865.55,343009.546218,64778.99,-622068.996218,-622068.996218,2025-09-11 0:00:00,2025-09-15 0:00:00,Steven Mateo Aroca Garzon,2025-09-11 0:00:00,False,0,Al día,
4,1000591190,2025-10-28,64778.99,0.000000,0.00,-557290.006218,-557290.006218,,,Steven Mateo Aroca Garzon,2025-10-15 0:00:00,False,0,Al día,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81468,99534,2022-09-30,146463.03,292927.000000,0.00,-1.230000,-1.230000,,2022-09-14 0:00:00,Natalia Valentina Castro Jimenez,2022-09-10 0:00:00,False,0,Cerrado,
81469,99534,2022-10-31,0.00,0.000000,0.00,-1.230000,-1.230000,,,Natalia Valentina Castro Jimenez,,False,0,Cerrado,
81470,9957452,2024-08-31,619327.73,798525.000000,179197.48,-179197.270000,-179197.270000,2024-08-13 0:00:00,2024-08-14 0:00:00,Juan Felipe Hurtado Mercado,2024-08-13 0:00:00,False,0,Al día,
81471,9957452,2024-09-30,179197.48,0.000000,0.00,0.210000,0.210000,,,Juan Felipe Hurtado Mercado,2024-09-05 0:00:00,False,0,Cerrado,


In [ ]:
import pandas as pd
import numpy as np

# --- Copias ---
df_filtrado_ = df_filtrado.copy()
df_mora_ = df_mora.copy()

# --- Alinear tipos de ID: usar string en ambos ---
df_filtrado_['REFERENCIA'] = df_filtrado_['Referencia'].astype(str).str.strip()
df_mora_['REFERENCIA'] = df_mora_['REFERENCIA'].astype(str).str.strip()

# --- 1) Llave mensual en df_filtrado (ignorando día) ---
base = pd.to_datetime(df_filtrado_['Mes_año_Originacion'], dayfirst=True, errors='coerce')
base_period = base.dt.to_period('M')

meses = df_filtrado_['Meses_despues_Originacion'].fillna(0).astype('int64').to_numpy()
target_period = base_period + meses   # Period[M]

df_filtrado_['MES_Y'] = target_period  # llave mensual para el merge

# (Opcional) FECHA "visual": último día del mes, salvo si es mes actual -> hoy
hoy = pd.Timestamp.today().normalize()  # naive
fin_de_mes = target_period.dt.to_timestamp(how='end')  # <-- usar .dt
mismo_mes_actual = (fin_de_mes.dt.month == hoy.month) & (fin_de_mes.dt.year == hoy.year)
df_filtrado_['FECHA'] = fin_de_mes.where(~mismo_mes_actual, hoy)

# --- 2) Llave mensual en df_mora a partir de su FECHA ---
df_mora_['FECHA'] = pd.to_datetime(df_mora_['FECHA'], errors='coerce').dt.normalize()
df_mora_['MES_Y'] = df_mora_['FECHA'].dt.to_period('M')

# Si hay varias filas en el mismo mes por REFERENCIA, quedarnos con la última del mes
df_mora_sorted = df_mora_.sort_values(['REFERENCIA', 'MES_Y', 'FECHA'])
df_mora_mensual = df_mora_sorted.drop_duplicates(['REFERENCIA', 'MES_Y'], keep='last')

# --- 3) Merge por (REFERENCIA, MES_Y) ---
cols_mora = ['REFERENCIA', 'MES_Y', 'MORA_STATUS']
out = df_filtrado_.merge(
    df_mora_mensual[cols_mora],
    how='left',
    on=['REFERENCIA', 'MES_Y'],
    validate='m:1'
)

# --- 4) Limpieza ---
out.drop(columns=['REFERENCIA', 'MES_Y'], inplace=True)

# out = df_filtrado con MORA_STATUS por mes/año



In [ ]:
out

,Referencia,Credito,Status facturacion,Tipo de comision,Monto,Fecha de facturacion,Apartado Mensual,Vehiculo de ahorro,Deuda inicial Fija,Mes_Año,...,Mes_año_Originacion,Meses_despues_Originacion,Meses_despues_facturacion,bank_monto,commission_monto,Pri-ult,Monto_Mensualidades,C/A,FECHA,MORA_STATUS
0,1000185400,false,COBRADO,LIQUIDACION_COLOMBIA,1.940592e+04,2025-06-20,520791.11,SKANDIA,22718600.0,02-2025,...,2025-06,0,0,552839.0,23093.0,1.97,NaN,0.166282,2025-06-30 23:59:59.999999999,Al día
1,1000185400,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,1.506000e+04,2025-06-19,520791.11,SKANDIA,22718600.0,02-2025,...,2025-06,0,0,552839.0,23093.0,1.97,NaN,0.166282,2025-06-30 23:59:59.999999999,Al día
2,1000185400,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,5.552800e+04,2025-06-19,520791.11,SKANDIA,22718600.0,02-2025,...,2025-06,1,1,552839.0,23093.0,1.97,NaN,0.166282,2025-07-31 23:59:59.999999999,Cerrado
3,1000591190,false,COBRADO,LIQUIDACION_COLOMBIA,1.286445e+05,2025-09-12,344986.00,SKANDIA,12465987.0,12-2024,...,2025-09,0,0,1930000.0,76000.0,1.97,NaN,0.230360,2025-09-30 23:59:59.999999999,Al día
4,100076,false,COBRADO,LIQUIDACION_COLOMBIA,1.754700e+05,2022-12-23,359735.00,A&V,17041870.0,06-2021,...,2022-12,0,0,2942000.0,208809.0,1.00,NaN,0.000000,2022-12-31 23:59:59.999999999,Al día
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24848,98709164,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,3.215280e+05,2025-09-01,515382.43,SKANDIA,12043200.0,01-2025,...,2025-09,2,1,1800000.0,769397.0,2.96,NaN,0.553309,2025-11-30 23:59:59.999999999,NaN
24849,98709164,true,COBRADO,LIQUIDACION_COLOMBIA,1.048366e+06,2025-09-01,515382.43,SKANDIA,12043200.0,01-2025,...,2025-09,2,1,1800000.0,769397.0,2.96,NaN,0.553309,2025-11-30 23:59:59.999999999,NaN
24850,99534,false,COBRADO,LIQUIDACION_COLOMBIA,3.284550e+05,2022-02-14,287861.00,A&V,9030750.0,06-2021,...,2022-02,0,0,1800000.0,390861.0,1.00,NaN,0.000000,2022-02-28 23:59:59.999999999,Cerrado
24851,99534,false,COBRADO,LIQUIDACION_COLOMBIA,7.323100e+04,2022-08-10,287861.00,A&V,9030750.0,06-2021,...,2022-08,0,0,2600000.0,261436.0,1.97,NaN,0.624195,2022-08-31 23:59:59.999999999,Mora 1


In [ ]:
# Eliminar la columna FECHA y renombrar resultado
df_filtrado = out.drop(columns=['FECHA'], errors='ignore').copy()

# Verificar resultado
df_filtrado.info()
df_filtrado.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24853 entries, 0 to 24852
Data columns (total 33 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Referencia                 24853 non-null  object        
 1   Credito                    24853 non-null  object        
 2   Status facturacion         24853 non-null  object        
 3   Tipo de comision           24853 non-null  object        
 4   Monto                      24853 non-null  float64       
 5   Fecha de facturacion       24853 non-null  datetime64[ns]
 6   Apartado Mensual           24853 non-null  float64       
 7   Vehiculo de ahorro         24853 non-null  object        
 8   Deuda inicial Fija         24853 non-null  float64       
 9   Mes_Año                    24853 non-null  object        
 10  Mes_Cobro                  24853 non-null  float64       
 11  AM/DB                      24853 non-null  float64       
 12  AM/D

,Referencia,Credito,Status facturacion,Tipo de comision,Monto,Fecha de facturacion,Apartado Mensual,Vehiculo de ahorro,Deuda inicial Fija,Mes_Año,...,n_pagos_PaB,Mes_año_Originacion,Meses_despues_Originacion,Meses_despues_facturacion,bank_monto,commission_monto,Pri-ult,Monto_Mensualidades,C/A,MORA_STATUS
0,1000185400,false,COBRADO,LIQUIDACION_COLOMBIA,19405.915966,2025-06-20,520791.11,SKANDIA,22718600.0,02-2025,...,1,2025-06,0,0,552839.0,23093.0,1.97,NaN,0.166282,Al día
1,1000185400,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,15060.000000,2025-06-19,520791.11,SKANDIA,22718600.0,02-2025,...,1,2025-06,0,0,552839.0,23093.0,1.97,NaN,0.166282,Al día
2,1000185400,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,55528.000000,2025-06-19,520791.11,SKANDIA,22718600.0,02-2025,...,1,2025-06,1,1,552839.0,23093.0,1.97,NaN,0.166282,Cerrado
3,1000591190,false,COBRADO,LIQUIDACION_COLOMBIA,128644.546218,2025-09-12,344986.00,SKANDIA,12465987.0,12-2024,...,1,2025-09,0,0,1930000.0,76000.0,1.97,NaN,0.230360,Al día
4,100076,false,COBRADO,LIQUIDACION_COLOMBIA,175470.000000,2022-12-23,359735.00,A&V,17041870.0,06-2021,...,1,2022-12,0,0,2942000.0,208809.0,1.00,NaN,0.000000,Al día


In [ ]:
if 'commission_monto' in df_filtrado.columns:
    df_filtrado.drop(columns=['commission_monto'], inplace=True)

In [ ]:
df_mora.info()
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81473 entries, 0 to 81472
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   REFERENCIA              81473 non-null  int64         
 1   FECHA                   81473 non-null  datetime64[ns]
 2   X_COBRAR                81473 non-null  float64       
 3   PAGO                    81473 non-null  float64       
 4   X_COBRAR_FUTURO         81473 non-null  float64       
 5   PLAN_PAGADO             81473 non-null  float64       
 6   PLAN_PAGADO_INCOBRABLE  81473 non-null  float64       
 7   FECHA_ORIGEN            81473 non-null  object        
 8   FECHA_COBRO             81473 non-null  object        
 9   RESPONSABLE             81473 non-null  object        
 10  FECHA_DE_PAGO           81473 non-null  object        
 11  MOROSO                  81473 non-null  bool          
 12  DIAS_EN_MORA            81473 non-null  int64 

In [ ]:
df_filtrado

,Referencia,Credito,Status facturacion,Tipo de comision,Monto,Fecha de facturacion,Apartado Mensual,Vehiculo de ahorro,Deuda inicial Fija,Mes_Año,...,PaB_total,n_pagos_PaB,Mes_año_Originacion,Meses_despues_Originacion,Meses_despues_facturacion,bank_monto,Pri-ult,Monto_Mensualidades,C/A,MORA_STATUS
0,1000185400,false,COBRADO,LIQUIDACION_COLOMBIA,1.940592e+04,2025-06-20,520791.11,SKANDIA,22718600.0,02-2025,...,552839.0,1,2025-06,0,0,552839.0,1.97,NaN,0.166282,Al día
1,1000185400,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,1.506000e+04,2025-06-19,520791.11,SKANDIA,22718600.0,02-2025,...,552839.0,1,2025-06,0,0,552839.0,1.97,NaN,0.166282,Al día
2,1000185400,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,5.552800e+04,2025-06-19,520791.11,SKANDIA,22718600.0,02-2025,...,552839.0,1,2025-06,1,1,552839.0,1.97,NaN,0.166282,Cerrado
3,1000591190,false,COBRADO,LIQUIDACION_COLOMBIA,1.286445e+05,2025-09-12,344986.00,SKANDIA,12465987.0,12-2024,...,1930000.0,1,2025-09,0,0,1930000.0,1.97,NaN,0.230360,Al día
4,100076,false,COBRADO,LIQUIDACION_COLOMBIA,1.754700e+05,2022-12-23,359735.00,A&V,17041870.0,06-2021,...,2942000.0,1,2022-12,0,0,2942000.0,1.00,NaN,0.000000,Al día
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24848,98709164,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,3.215280e+05,2025-09-01,515382.43,SKANDIA,12043200.0,01-2025,...,1800000.0,1,2025-09,2,1,1800000.0,2.96,NaN,0.553309,NaN
24849,98709164,true,COBRADO,LIQUIDACION_COLOMBIA,1.048366e+06,2025-09-01,515382.43,SKANDIA,12043200.0,01-2025,...,1800000.0,1,2025-09,2,1,1800000.0,2.96,NaN,0.553309,NaN
24850,99534,false,COBRADO,LIQUIDACION_COLOMBIA,3.284550e+05,2022-02-14,287861.00,A&V,9030750.0,06-2021,...,1800000.0,1,2022-02,0,0,1800000.0,1.00,NaN,0.000000,Cerrado
24851,99534,false,COBRADO,LIQUIDACION_COLOMBIA,7.323100e+04,2022-08-10,287861.00,A&V,9030750.0,06-2021,...,2600000.0,1,2022-08,0,0,2600000.0,1.97,NaN,0.624195,Mora 1


In [ ]:
# ============================
# AJUSTE DE COLUMNAS df_resultado
# ============================

# 1) Eliminar la columna 'bank_monto' si existe
if 'bank_monto' in df_filtrado.columns:
    df_filtrado.drop(columns=['bank_monto'], inplace=True)


# 2) Definir el nuevo orden con 'commission_monto' después de 'requester'
columnas_orden = [
    'Referencia', 'Credito', 'Status facturacion', 'Tipo de comision', 'Monto',
    'Fecha de facturacion', 'Apartado Mensual', 'Vehiculo de ahorro',
    'Deuda inicial Fija', 'Mes_Año', 'Mes_Cobro', 'AM/DB', 'AM/DB_outlier',
    'Rango_AM_DB', 'Rango_de_deuda', 'mes_exacto', 'Originado', 'requester',
    'BANCO', 'amount_promedio', 'amount_total', 'n_pagos', 'PaB_total',
    'n_pagos_PaB', 'Mes_año_Originacion', 'Meses_despues_Originacion',
    'Meses_despues_facturacion', 'C/A', 'Pri-ult', 'Monto_Mensualidades',
    'MORA_STATUS'
]

# 3) Reordenar (las columnas que existan + las extras al final)
columnas_existentes = [c for c in columnas_orden if c in df_filtrado.columns]
otras_columnas = [c for c in df_filtrado.columns if c not in columnas_orden]

df_filtrado = df_filtrado[columnas_existentes + otras_columnas]

print("✅ Columnas ajustadas correctamente. Nuevo orden:")
print(df_filtrado.columns.tolist())


✅ Columnas ajustadas correctamente. Nuevo orden:
['Referencia', 'Credito', 'Status facturacion', 'Tipo de comision', 'Monto', 'Fecha de facturacion', 'Apartado Mensual', 'Vehiculo de ahorro', 'Deuda inicial Fija', 'Mes_Año', 'Mes_Cobro', 'AM/DB', 'AM/DB_outlier', 'Rango_AM_DB', 'Rango_de_deuda', 'mes_exacto', 'Originado', 'requester', 'BANCO', 'amount_promedio', 'amount_total', 'n_pagos', 'PaB_total', 'n_pagos_PaB', 'Mes_año_Originacion', 'Meses_despues_Originacion', 'Meses_despues_facturacion', 'C/A', 'Pri-ult', 'Monto_Mensualidades', 'MORA_STATUS']


In [ ]:
#if 'Monto_Mensualidades' in df_resultado.columns:
#    df_filtrado = df_filtrado.drop(columns=['Monto_Mensualidades'])

In [ ]:
# ---- uploader.py (puedes ponerlo en una celda de Colab o en un .py en tu repo) ----
import os, json, re
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials
from gspread_dataframe import set_with_dataframe

def _load_service_account_credentials():
    """
    Busca credenciales en este orden:
      1) Colab secret userdata['MI_JSON']
      2) Variable de entorno MI_JSON
      3) Archivo local service_account.json
    """
    # 1) Colab
    try:
        from google.colab import userdata  # type: ignore
        creds_str = userdata.get('MI_JSON')
        if creds_str:
            return Credentials.from_service_account_info(
                json.loads(creds_str),
                scopes=["https://www.googleapis.com/auth/spreadsheets"]
            )
    except Exception:
        pass

    # 2) Env var
    mi_json = os.environ.get("MI_JSON")
    if mi_json:
        return Credentials.from_service_account_info(
            json.loads(mi_json),
            scopes=["https://www.googleapis.com/auth/spreadsheets"]
        )

    # 3) Archivo local
    if os.path.exists("service_account.json"):
        with open("service_account.json", "r", encoding="utf-8") as f:
            info = json.load(f)
        return Credentials.from_service_account_info(
            info,
            scopes=["https://www.googleapis.com/auth/spreadsheets"]
        )

    raise RuntimeError("No encontré credenciales. Define MI_JSON (Colab o env var) o aporta service_account.json.")

def _parse_sheet_url(url_or_id: str):
    """
    Devuelve (spreadsheet_id, gid_int|None).
    Acepta URL completa o solo ID.
    """
    if "/d/" in url_or_id:
        spreadsheet_id = url_or_id.split("/d/")[1].split("/")[0]
        m = re.search(r"[?&#]gid=(\d+)", url_or_id)
        gid = int(m.group(1)) if m else None
    else:
        spreadsheet_id = url_or_id
        gid = None
    return spreadsheet_id, gid

def upload_dataframe_to_sheet(
    df: pd.DataFrame,
    spreadsheet_url_or_id: str,
    sheet_name: str | None = None,
    mode: str = "replace"  # "replace" limpia, "append" agrega al final
):
    """
    Sube un DataFrame a una hoja de Google Sheets.
      - Si la URL trae gid, escribe en esa hoja por ID.
      - Si das sheet_name, usa ese nombre (crea la hoja si no existe).
    """
    creds = _load_service_account_credentials()
    gc = gspread.authorize(creds)

    spreadsheet_id, gid = _parse_sheet_url(spreadsheet_url_or_id)
    sh = gc.open_by_key(spreadsheet_id)

    # Resolver worksheet
    ws = None
    if gid is not None:
        # Abrir por gid exacto
        try:
            ws = sh.get_worksheet_by_id(gid)
        except Exception as e:
            raise RuntimeError(f"No pude abrir la hoja con gid={gid}. "
                               f"Verifica permisos y que la hoja exista. Detalle: {e}")
    elif sheet_name:
        try:
            ws = sh.worksheet(sheet_name)
        except gspread.exceptions.WorksheetNotFound:
            ws = sh.add_worksheet(title=sheet_name, rows="100", cols="31")
    else:
        # fallback: primera hoja
        ws = sh.sheet1

    # Escribir
    if mode == "append":
        # Append: conserva encabezados si ya existen; si la hoja está vacía, escribe con encabezados
        values = ws.get_all_values()
        start_row = len(values) + 1 if values else 1
        if not values:
            set_with_dataframe(ws, df, row=1, col=1, include_index=False, include_column_header=True, resize=True)
        else:
            # sin encabezado para no duplicarlo
            set_with_dataframe(ws, df, row=start_row, col=1, include_index=False, include_column_header=False, resize=True)
    else:
        # replace: limpia y escribe
        ws.clear()
        set_with_dataframe(ws, df, include_index=False, include_column_header=True, resize=True)

    # Mostrar el service account para compartir acceso si hace falta
    try:
        print("✅ Subido correctamente.")
        print("👉 Si te da permiso denegado, comparte el Sheet con:", creds.service_account_email)
    except Exception:
        pass

In [ ]:
from __main__ import upload_dataframe_to_sheet  # si está en la misma celda/entorno

# 3) Sube tu DataFrame df_resultado a la hoja del gid de tu URL
url = "https://docs.google.com/spreadsheets/d/1xGSzneJkRqREZupLshG9jaNK8wbP8WpV-yjUs_Lm7AA/edit?gid=1534851414#gid=1534851414"
upload_dataframe_to_sheet(df_filtrado, url, mode="replace")  # o mode="append"

✅ Subido correctamente.
👉 Si te da permiso denegado, comparte el Sheet con: colabservice@colabaccess-468021.iam.gserviceaccount.com


In [ ]:
# 1) Seleccionar columnas requeridas
columnas = ["Referencia", "Originado", "amount_promedio", "amount_total", "PaB_total", "Pri-ult", "n_pagos", "n_pagos_PaB", "C/A"]
df_resultado_filtrado = df_filtrado[columnas].copy()

# 2) Eliminar duplicados (todas las columnas consideradas)
df_resultado_sin_dups = df_resultado_filtrado.drop_duplicates()

# 3) Subir a una hoja NUEVA llamada "Resultado sin duplicados"
#    (no se usa el gid anterior; se mantiene el mismo Spreadsheet)
url = "https://docs.google.com/spreadsheets/d/1xGSzneJkRqREZupLshG9jaNK8wbP8WpV-yjUs_Lm7AA/edit"

upload_dataframe_to_sheet(
    df_resultado_sin_dups,
    url,
    sheet_name="Resultado sin duplicados",  # nueva hoja
    mode="replace"  # o "append" si prefieres agregar sin borrar
)


✅ Subido correctamente.
👉 Si te da permiso denegado, comparte el Sheet con: colabservice@colabaccess-468021.iam.gserviceaccount.com


#CC_PaB

In [ ]:
import os, io, json, time, base64
import pandas as pd, requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def get_secret(name):
    # 1) Colab.userdata si existe
    try:
        from google.colab import userdata as _ud
        v = _ud.get(name)
        if v:
            return v
    except Exception:
        pass
    # 2) Variables de entorno (GitHub/local)
    return os.environ.get(name)

USER_API   = get_secret("USER_API")
SECRET_API = get_secret("SECRET_API")

if not USER_API or not SECRET_API:
    raise ValueError("Falta USER_API o SECRET_API. En Colab: 'Secretos'. En GitHub: Settings → Secrets → Actions.")

BASE_URL = "https://mutatio-api.gobravo.dev"
RESOURCE = "/accounting/flows/download"

def make_session():
    s = requests.Session()
    retry = Retry(
        total=3, connect=2, read=2, backoff_factor=1.2,
        status_forcelist=[502, 503, 504], allowed_methods={"POST"},
        raise_on_status=False
    )
    s.mount("https://", HTTPAdapter(max_retries=retry))
    s.mount("http://",  HTTPAdapter(max_retries=retry))
    return s

session = make_session()

def _leer_csv_flexible(resp) -> pd.DataFrame:
    resp.raise_for_status()
    txt = resp.text or ""
    if not txt.strip():
        return pd.DataFrame()
    first = txt.splitlines()[0]
    sep = ';' if first.count(';') >= first.count(',') else ','
    return pd.read_csv(io.StringIO(txt), sep=sep, dtype=str)

def _decode_jwt_exp(jwt_token: str):
    try:
        parts = jwt_token.split(".")
        if len(parts) != 3:
            return None
        payload_b64 = parts[1] + "==="  # padding
        payload = json.loads(base64.urlsafe_b64decode(payload_b64))
        return payload.get("exp"), payload
    except Exception:
        return None

def get_token(user: str, secret: str) -> str:
    url = f"{BASE_URL}/auth/generate-token"
    r = session.post(url, json={"user": user, "secret": secret},
                     headers={"Content-Type":"application/json","Accept":"application/json"},
                     timeout=(10, 45))
    r.raise_for_status()
    tok = r.json().get("token")
    if not tok:
        raise RuntimeError(f"Auth OK pero no vino 'token'. Respuesta: {r.text[:300]}")
    return tok

class Client:
    def __init__(self, user, secret):
        self.user = user
        self.secret = secret
        self.token = None

    def ensure_token(self):
        if not self.token:
            self.token = get_token(self.user, self.secret)

    def refresh_token(self):
        self.token = get_token(self.user, self.secret)

    def descargar_pagina_flows(self, page: int) -> pd.DataFrame:
        self.ensure_token()
        try_styles = [
            ("query", {"params": {"pageToDownload": str(page)}, "json": []}),
            ("json",  {"params": {}, "json": {"pageToDownload": page}}),
        ]
        last_resp = None

        for style, kwargs in try_styles:
            url = f"{BASE_URL}{RESOURCE}"
            headers = {
                "Authorization": f"Bearer {self.token}",
                "Accept": "text/csv, text/plain",
                "Content-Type": "application/json",
            }
            resp = session.post(url, headers=headers, timeout=(12, 90), **kwargs)
            last_resp = resp

            if resp.status_code == 200:
                return _leer_csv_flexible(resp)

            if resp.status_code == 401:
                # Intento único de refrescar token y reintentar con el mismo estilo
                exp_info = _decode_jwt_exp(self.token)
                if exp_info:
                    exp_ts, payload = exp_info
                    if exp_ts:
                        ahora = int(time.time())
                        print(f"[DEBUG] JWT expira en {exp_ts} (faltan {exp_ts - ahora}s). Payload (recortado): {json.dumps({k:payload[k] for k in payload if k!='exp'})[:200]}")
                print(f"[WARN] 401 con estilo '{style}'. Intento refrescar token y reintentar una vez...")
                self.refresh_token()
                headers["Authorization"] = f"Bearer {self.token}"
                resp2 = session.post(url, headers=headers, timeout=(12, 90), **kwargs)
                last_resp = resp2
                if resp2.status_code == 200:
                    return _leer_csv_flexible(resp2)
                else:
                    # si vuelve a fallar 401/403, seguimos al siguiente estilo o salimos
                    continue

        # Si llegamos aquí, no hubo 200
        msg = (last_resp.text or "")[:500] if last_resp is not None else ""
        raise RuntimeError(f"Fallo al descargar página {page}. "
                           f"status={getattr(last_resp,'status_code',None)} "
                           f"body='{msg}'")

    def descargar_todo_flows(self, max_pages: int = 500) -> pd.DataFrame:
        frames = []
        for page in range(max_pages):
            df = self.descargar_pagina_flows(page)
            if df.empty:
                break
            frames.append(df)
        return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

# --- Uso ---
cli = Client(USER_API, SECRET_API)
flujos_pab = cli.descargar_todo_flows(max_pages=200)

print(f"Filas totales: {len(flujos_pab)} | Columnas: {len(flujos_pab.columns)}")
print(flujos_pab.columns.tolist())

Filas totales: 127436 | Columnas: 43
['Id', 'Empresa', 'Tipo de flujo', 'Se paga con credito', 'Referencia', 'Receptor de pago', 'Cuenta de retiro', 'Numero de cheque', 'Empresa pagadora', 'Tipo de financiamiento', 'Fecha de flujo', 'Monto', 'Status de cuenta por cobrar', 'Fecha de facturacion', 'Fecha de envio de cobro', 'Liquidacion Relacionada', 'Fecha de cobro', 'Banco receptor', 'Saldo', 'Numero de factura', 'Status flujo', 'Fecha de vencimiento', 'Concepto', 'Rubro de contabilidad', 'Area', 'Comision estructurada', 'Fecha Comision estructurada', 'Referencia de crédito', 'Numero de factura (Compra)', 'Id liquidacion relacionada', 'Id Quickbase del flujo', 'Fecha de creacion', 'Fecha de actualizacion', 'Creado por', 'Actualizado por', 'Status Sistema Contable', 'Subido a Sistema Contable por', 'Fecha subida a sistema contable', 'Id de Flujo dividido', 'Documento Soporte', 'Status TE Sistema Contable', 'Subido TE a Sistema Contable por', 'Fecha TE subida a sistema contable']


In [ ]:
resultado_actualizado

,Referencia,Credito,Status facturacion,Tipo de comision,Monto,Fecha de facturacion,Apartado Mensual,Vehiculo de ahorro,Deuda inicial Fija,Mes_Año,Mes_Cobro,AM/DB,AM/DB_outlier,Rango_AM_DB,Rango_de_deuda
0,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,602031.0,2025-06-26,611362.66,SKANDIA,43346600.0,06-2025,0.0,0.014104,0,< 1.5%,35 – 60 M
1,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,627346.0,2025-06-26,611362.66,SKANDIA,43346600.0,06-2025,1.0,0.014104,0,< 1.5%,35 – 60 M
2,1000018033,false,COBRADO,MENSUALIDAD_COLOMBIA,227237.0,2025-08-05,611362.66,SKANDIA,43346600.0,06-2025,2.0,0.014104,0,< 1.5%,35 – 60 M
3,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,3852.0,2025-06-26,611362.66,SKANDIA,43346600.0,06-2025,2.0,0.014104,0,< 1.5%,35 – 60 M
4,1000018033,false,COBRADO,MENSUALIDAD_COLOMBIA,227237.0,2025-09-03,611362.66,SKANDIA,43346600.0,06-2025,3.0,0.014104,0,< 1.5%,35 – 60 M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765853,99990,false,COBRADO,MENSUALIDAD_COLOMBIA,157577.0,2021-12-05,519374.00,A&V,29414564.0,06-2021,6.0,0.017657,0,1.5% – 1.99%,< 35 M
765854,99990,true,COBRADO,LIQUIDACION_COLOMBIA,1210878.0,2021-12-23,519374.00,A&V,29414564.0,06-2021,6.0,0.017657,0,1.5% – 1.99%,< 35 M
765855,99990,true,COBRADO,MENSUALIDAD_COLOMBIA,1575770.0,2021-12-23,519374.00,A&V,29414564.0,06-2021,6.0,0.017657,0,1.5% – 1.99%,< 35 M
765856,99990,true,COBRADO,LIQUIDACION_COLOMBIA,112390.0,2022-07-30,519374.00,A&V,29414564.0,06-2021,13.0,0.017657,0,1.5% – 1.99%,< 35 M


In [ ]:
resultado_actualizado_pab = resultado_actualizado.drop(columns=["Credito", "Status facturacion", "Tipo de comision", "Monto", "Fecha de facturacion", "Mes_Cobro"], errors="ignore")

In [ ]:
resultado_actualizado_pab.drop_duplicates(inplace=True)

In [ ]:
berex_estruct["reference"] = berex_estruct["reference"].astype("object")
flujos_pab["Referencia"] = flujos_pab["Referencia"].astype("object")
resultado_actualizado_pab["Referencia"] = resultado_actualizado_pab["Referencia"].astype("object")

refs_validas = berex_estruct["reference"].unique()

flujos_pab = flujos_pab[flujos_pab["Referencia"].isin(refs_validas)]
resultado_actualizado_pab = resultado_actualizado_pab[resultado_actualizado_pab["Referencia"].isin(refs_validas)]

# (Opcional) Reiniciar índices
flujos_pab.reset_index(drop=True, inplace=True)
resultado_actualizado_pab.reset_index(drop=True, inplace=True)

In [ ]:
# Filtrar las filas donde 'destination' sea 'bank'
berex_bank = berex_estruct[berex_estruct["destination"] == "bank"]

# Crear el nuevo DataFrame CC_PAB con las columnas seleccionadas y renombradas
CC_PAB = berex_bank.rename(columns={
    "reference": "REFERENCIA",
    "amount": "X_COBRAR",
    "bank": "BANCO",
    "payment_date": "FECHA_PAGO",
    "requester": "RESPONSABLE",
    "Originado": "FECHA_ORIGEN"
})[["REFERENCIA", "X_COBRAR", "BANCO", "FECHA_PAGO", "RESPONSABLE", "FECHA_ORIGEN"]]

# Reiniciar el índice
CC_PAB.reset_index(drop=True, inplace=True)

In [ ]:
resultado_actualizado_pab

,Referencia,Apartado Mensual,Vehiculo de ahorro,Deuda inicial Fija,Mes_Año,AM/DB,AM/DB_outlier,Rango_AM_DB,Rango_de_deuda
0,1000185400,520791.11,SKANDIA,22718600.0,02-2025,0.022924,0,2% – 2.5%,< 35 M
1,1000591190,344986.00,SKANDIA,12465987.0,12-2024,0.027674,0,> 2.5%,< 35 M
2,100076,359735.00,A&V,17041870.0,06-2021,0.021109,0,2% – 2.5%,< 35 M
3,1001082965,2530576.88,SKANDIA,196225650.0,07-2024,0.012896,0,< 1.5%,> 100 M
4,1001244516,214729.55,SKANDIA,10192650.0,11-2023,0.021067,0,2% – 2.5%,< 35 M
...,...,...,...,...,...,...,...,...,...
7753,98700329,545221.52,SKANDIA,36990527.0,11-2024,0.014739,0,< 1.5%,35 – 60 M
7754,98702966,1452577.37,SKANDIA,121123008.0,03-2024,0.011993,0,< 1.5%,> 100 M
7755,98709164,515382.43,SKANDIA,12043200.0,01-2025,0.042794,1,> 2.5%,< 35 M
7756,99534,287861.00,A&V,9030750.0,06-2021,0.031876,0,> 2.5%,< 35 M


In [ ]:
flujos_pab

,Id,Empresa,Tipo de flujo,Se paga con credito,Referencia,Receptor de pago,Cuenta de retiro,Numero de cheque,Empresa pagadora,Tipo de financiamiento,...,Creado por,Actualizado por,Status Sistema Contable,Subido a Sistema Contable por,Fecha subida a sistema contable,Id de Flujo dividido,Documento Soporte,Status TE Sistema Contable,Subido TE a Sistema Contable por,Fecha TE subida a sistema contable
0,70037,RESUELVE_TU_DEUDA_COLOMBIA_SAS,FINANCIAMIENTO_A_CLIENTE,No,3007013619,Scotiabank Colpatria,BANCOLOMBIA,8906,RESPALDO_COLOMBIA_SAS,OPERACION_REPARACION_DE_CREDITO,...,Sin registro,andres.giraldo,NO_CARGADO,Sin data,Sin fecha,Sin id,NO,NO_CARGADO,Sin data,Sin fecha
1,70040,RESUELVE_TU_DEUDA_COLOMBIA_SAS,FINANCIAMIENTO_A_CLIENTE,No,3142462909,?xito,IRIS,845500,RESUELVE_TU_DEUDA_COLOMBIA_SAS,OPERACION_REPARACION_DE_CREDITO,...,Sin registro,Sin registro,NO_CARGADO,Sin data,Sin fecha,Sin id,NO,NO_CARGADO,Sin data,Sin fecha
2,70041,RESUELVE_TU_DEUDA_COLOMBIA_SAS,FINANCIAMIENTO_A_CLIENTE,No,3507158039,Banco de Bogota,IRIS,367000,RESUELVE_TU_DEUDA_COLOMBIA_SAS,OPERACION_REPARACION_DE_CREDITO,...,Sin registro,sgonzalez,NO_CARGADO,Sin data,Sin fecha,Sin id,NO,NO_CARGADO,Sin data,Sin fecha
3,70042,RESUELVE_TU_DEUDA_COLOMBIA_SAS,FINANCIAMIENTO_A_CLIENTE,No,3106137498,Banco de Bogot?,BANCOLOMBIA,8909,RESPALDO_COLOMBIA_SAS,OPERACION_REPARACION_DE_CREDITO,...,Sin registro,Sin registro,NO_CARGADO,Sin data,Sin fecha,Sin id,NO,NO_CARGADO,Sin data,Sin fecha
4,70043,RESUELVE_TU_DEUDA_COLOMBIA_SAS,FINANCIAMIENTO_A_CLIENTE,No,3212009494,Bancolombia,BANCOLOMBIA,21803994,RESPALDO_COLOMBIA_SAS,OPERACION_REPARACION_DE_CREDITO,...,Sin registro,sgonzalez,NO_CARGADO,Sin data,Sin fecha,Sin id,NO,NO_CARGADO,Sin data,Sin fecha
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27716,198494,RACE_CAPITAL_COLOMBIA_SAS,FINANCIAMIENTO_A_CLIENTE,Si,79599377,Itaú,BANCOLOMBIA,80733094.95,RACE_CAPITAL_COLOMBIA_SAS,PAGO_A_BANCO,...,giseth.cadena,felipe.contreras,NO_CARGADO,Sin data,Sin fecha,198417,NO,NO_CARGADO,Sin data,Sin fecha
27717,198498,LEVA_CAPITAL_COLOMBIA_SAS,FINANCIAMIENTO_A_CLIENTE,Si,3103019614,Scotiabank Colpatria,BANCOLOMBIA,111291015,LEVA_CAPITAL_COLOMBIA_SAS,PAGO_A_BANCO,...,giseth.cadena,felipe.contreras,NO_CARGADO,Sin data,Sin fecha,198455,NO,NO_CARGADO,Sin data,Sin fecha
27718,198508,LEVA_CAPITAL_COLOMBIA_SAS,FINANCIAMIENTO_A_CLIENTE,Si,3134216,Bancoomeva,BANCOLOMBIA,92641104.3,LEVA_CAPITAL_COLOMBIA_SAS,PAGO_A_BANCO,...,giseth.cadena,felipe.contreras,NO_CARGADO,Sin data,Sin fecha,198434,NO,NO_CARGADO,Sin data,Sin fecha
27719,198510,LEVA_CAPITAL_COLOMBIA_SAS,FINANCIAMIENTO_A_CLIENTE,Si,3168680324,Covinoc,BANCOLOMBIA,111291015,LEVA_CAPITAL_COLOMBIA_SAS,PAGO_A_BANCO,...,giseth.cadena,felipe.contreras,NO_CARGADO,Sin data,Sin fecha,198427,NO,NO_CARGADO,Sin data,Sin fecha


In [ ]:
# 1. Eliminar la columna FECHA_PAGO
CC_PAB = CC_PAB.drop(columns=["FECHA_PAGO"], errors="ignore")

# 2. Agrupar por las columnas clave y sumar X_COBRAR
CC_PAB = (
    CC_PAB
    .groupby(["REFERENCIA", "BANCO", "RESPONSABLE", "FECHA_ORIGEN"], as_index=False)
    .agg({"X_COBRAR": "sum"})
)

# 3. Renombrar la columna resultante
CC_PAB.rename(columns={"X_COBRAR": "X_COBRAR_TOTAL"}, inplace=True)

# 4. (Opcional) Verificar el resultado
print(CC_PAB.head())

   REFERENCIA        BANCO                RESPONSABLE FECHA_ORIGEN  \
0  1000185400   Rapicredit   Hector Elian Lacera Vega   2025-06-19   
1  1000591190    Falabella  Steven Mateo Aroca Garzon   2025-09-11   
2      100076   Davivienda  David Steban Pineda Gallo   2022-12-23   
3      100076   Davivienda                No Asignado   2023-01-11   
4  1001082965  Bancolombia  Alba Yohana Moreno Martin   2024-12-23   

   X_COBRAR_TOTAL  
0        552839.0  
1       1930000.0  
2       2942000.0  
3      10200000.0  
4        590000.0  


In [ ]:
CC_PAB

,REFERENCIA,BANCO,RESPONSABLE,FECHA_ORIGEN,X_COBRAR_TOTAL
0,1000185400,Rapicredit,Hector Elian Lacera Vega,2025-06-19,552839.0
1,1000591190,Falabella,Steven Mateo Aroca Garzon,2025-09-11,1930000.0
2,100076,Davivienda,David Steban Pineda Gallo,2022-12-23,2942000.0
3,100076,Davivienda,No Asignado,2023-01-11,10200000.0
4,1001082965,Bancolombia,Alba Yohana Moreno Martin,2024-12-23,590000.0
...,...,...,...,...,...
13997,98702966,Éxito,Jana Milena Lopez Buitrago,2024-09-30,6094000.0
13998,98709164,Bancolombia,Vivian Caterin Rodriguez Verano,2025-08-29,1800000.0
13999,99534,Colsubsidio,Natalia Valentina Castro Jimenez,2022-08-10,2600000.0
14000,99534,Scotiabank Colpatria,Maria Paula Balaguera Penagos,2022-02-14,1800000.0
